In [ ]:
# Astrophysics Project Made By Aharon Kakoly- "Radial To Rotating Flows"
# The main goal of the project is to provide a statistical tool (by Python) that is able to give a statistical representation of the cosmological population
# of each one of the gases. In other words, it will give an information about how often our sightlines (random sightlines) passes through gas of a particular
# column density."For this specific redshift in the universe we expect tp this amount of each one of the gases consistent with what we expect".
# The tool will do that by creating HI projections for different simulations and snapshots (that is the input of the code).
# The tool output will be two things: a saved .png image of the HI projection of each one of the simulations, and a saved .hdf5 file of the projection.
# This is the final code for my project.

In [ ]:
# Import of important libraries in Python:
import sys, os, time, importlib, glob, pdb
import matplotlib, pylab as pl, numpy as np
from numpy import log10 as log
import h5py, astropy, scipy, scipy.stats
from astropy import units as un, constants as cons
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import statistics as s
import random
from os import listdir
from os.path import isfile, join
import MyTools.AnalysisTools as l
import MyTools.HalosData as HD
import MyTools.Verdict as Verdict
import MyTools.Config as c
import math
import matplotlib.patches as mpatches
from astropy import units as un, constants as cons
import pylab as pl
import seaborn as sns
import palettable
%matplotlib auto

# FIRE studio libraries
from OldFireStudio.firestudio.studios.gas_studio import GasStudio
from OldFireStudio.firestudio.studios.star_studio import StarStudio

# Important constants
X=0.7 # Hydrogen mass fraction
h=0.7 # Hubble parameter in units of 100 km/s Mpc^-1

# Print the config file:
# For CCA
print("For CCA")
print("The simulations names in the config file are:",c.ListOfCCASimulationsNames)
print("The simulations key names in the config file are:",c.ListOfCCAkeysOfTheSimulationsNames)
print("The snapshot numbers in the config file are:",c.ListOfCCASnapshotsNumbers)
print("")
# For Stampede
print("For Stampede")
print("The simulations names in the config file are:",c.ListOfStampedeSimulationsNames)
print("The simulations key names in the config file are:",c.ListOfStampedekeysOfTheSimulationsNames)
print("The snapshot numbers in the config file are:",c.ListOfStampedeSnapshotsNumbers)
print("")

In [ ]:
# Creates the list of the default simulationes from the config file and the relevant paths for them
ListOfCCASimulationsNames=c.ListOfCCASimulationsNames
ListOfStampedeSimulationsNames=c.ListOfStampedeSimulationsNames
ListOfCCASimulationsPaths=[]
ListOfStampedeSimulationsPaths=[]
for i in ListOfCCASimulationsNames:
    SnapDirect=c.CCASnapDirect
    SnapDirect=SnapDirect+i
    ListOfCCASimulationsPaths.append(SnapDirect)
for i in ListOfStampedeSimulationsNames:
    SnapDirect=c.StampedeSnapDirect
    SnapDirect=SnapDirect+i
    ListOfStampedeSimulationsPaths.append(SnapDirect)
print("The simulations paths for CCA are: ",ListOfCCASimulationsPaths)
print("The snapshots numbers for CCA are: ",c.ListOfCCASnapshotsNumbers)
print("")
print("The simulations paths for Stampede are: ",ListOfStampedeSimulationsPaths)
print("The snapshots numbers for Stampede are: ",c.ListOfStampedeSnapshotsNumbers)

In [ ]:
# This block will create the full paths directories for the chosen simulations and sanpshot numbers
ListOfSnapshotsNumbersForCCA=c.ListOfCCASnapshotsNumbers
ListOfSnapshotsNumbersForStampede=c.ListOfStampedeSnapshotsNumbers
ListOfOutputFolderPathsForCCA=[]
ListOfOutputFolderPathsForStampede=[]
for i in ListOfCCASimulationsPaths:
    temp=i#+"/output"
    ListOfOutputFolderPathsForCCA.append(temp)
ListOfFullPathsForCCA=[]
for i in ListOfOutputFolderPathsForCCA:
    OutputFiles=os.listdir(i)
    for j in ListOfSnapshotsNumbersForCCA:
        option1="snapshot_"+j+".hdf5"
        option2="snapdir_"+j
        if(option1 in OutputFiles):
            FullPath=i+"/"+option1
        if(option2 in OutputFiles):
            FullPath=i+"/"+option2
        ListOfFullPathsForCCA.append(FullPath) 
print("The full paths for CCA simulations and snapshots are:")
print(ListOfOutputFolderPathsForCCA)

print("")

for i in ListOfStampedeSimulationsPaths:
    temp=i+"/output"
    ListOfOutputFolderPathsForStampede.append(temp)
ListOfFullPathsForStampede=[]
for i in ListOfOutputFolderPathsForStampede:
    OutputFiles=os.listdir(i)
    for j in ListOfSnapshotsNumbersForStampede:
        option1="snapshot_"+j+".hdf5"
        option2="snapdir_"+j
        if(option1 in OutputFiles):
            FullPath=i+"/"+option1
        if(option2 in OutputFiles):
            FullPath=i+"/"+option2
        ListOfFullPathsForStampede.append(FullPath) 
print("The full paths for Stampede simulations and snapshots are:")
print(ListOfFullPathsForStampede)

In [ ]:
# This block will create HI projection plot for each of the simulations and snapshot numbers of CCA !!!
# Output: will save 2 files: .png image of the plot and .hdf5 file of the projection
plt.clf()
ListOfSnapshotsParts=[]
forloopcount=0
for SnapDirect in ListOfFullPathsForCCA:
    ListOfSnapshotsParts=[]
    if(SnapDirect.endswith('.hdf5')):
        ListOfSnapshotsParts.append(SnapDirect)
    else:
        ListOfFiles=os.listdir(SnapDirect)
        for i in ListOfFiles:
            if (i.endswith('.hdf5')):
                ListOfSnapshotsParts.append(SnapDirect+'/'+i)# Append Only hdf5 Files To The Snapshots Parts List
    print("The list of the snapshot parts is:",ListOfSnapshotsParts)
    print("")
    SimulationKey=c.ListOfCCAkeysOfTheSimulationsNames[forloopcount]
    Snap1=l.Snapshot(ListOfSnapshotsParts[0],[0,0,0],[0,0,0])# SnapName, Not Real Center Coordinates, Not Real Center Velocity
    z=Snap1.Redshift()
    print("The redshift of the given halo is:",z)
    Mvir=HD.HaloMvir(SimulationKey,z)
    Rvir=HD.HaloRvir(SimulationKey,z)
    CenterCoordinates=HD.HaloCenterCoordinates(SimulationKey,z)
    CenterVelocity=HD.HaloCenterVelocity(SimulationKey,z)
    afactor=HD.HaloaFactor(SimulationKey,z)
   
    # This part intended to merge all the data from the different snapshot parts (if there are more than 1 part)
    TotalNumberOfParticles=0
    Masses=[]
    Coordinates=[]
    Temperature=[]
    r=[]
    rs=[]
    Density=[]
    NeutralH=[]
    GasMasses=[]
    BoxesSizes=[]
    SmoothingLengths=[]
    
    for snapshotname in ListOfSnapshotsParts:
        Snap1=l.Snapshot(snapshotname,CenterCoordinates,CenterVelocity,a=afactor)# SnapName, Real Center Coordinates, Real Center Velocity

        NumberOfParticlesOfOneSnapshot=Snap1.number_of_particles()# 0-Gas,1-High Resolution Dark Matter,2-Dummy Particles,3-Dummy Particles,4-Stars,5-Black Holes
        TotalNumberOfParticles=TotalNumberOfParticles+NumberOfParticlesOfOneSnapshot[0]

        MassesOfOneSnapshotGasParticles=Snap1.masses(0)# The Masses Of The Gas Particles
        GasMasses.extend(MassesOfOneSnapshotGasParticles)

        CoordinatesOfOneSnapShot=Snap1.coords(0)# The Coordinates Of The Gas Particles
        Coordinates.extend(CoordinatesOfOneSnapShot)

        TemperatureOfOneSnapShot=Snap1.Ts()# The Temperature Of The Gas Particles
        Temperature.extend(TemperatureOfOneSnapShot)

        rOfOneSnapShot=Snap1.rs()# The r Of The Gas Particles
        r.extend(rOfOneSnapShot)

        DensityOfOneSnapShot=Snap1.rhos()# The Density Of The Gas Particles
        Density.extend(DensityOfOneSnapShot)

        NeutralHOfOneSnapshot=Snap1.neutralHydrogenAbundance(0)# Returns The Neutral Hydrogen Fraction
        NeutralH.extend(NeutralHOfOneSnapshot)
        
        SmoothingLengthOfOneSnapshot=Snap1.SmoothingLength()
        SmoothingLengths.extend(SmoothingLengthOfOneSnapshot)
        
    Coordinates=np.array(Coordinates)
    Temperature=np.array(Temperature)
    r=np.array(r)
    Density=np.array(Density)
    NeutralH=np.array(NeutralH)
    GasMasses=np.array(GasMasses)
    SmoothingLengths=np.array(SmoothingLengths)
    
    print("The total number of gas particles is:",TotalNumberOfParticles)
    print("r min is:",r.min())
    print("r max is:",r.max())
          
    snapdir=ListOfOutputFolderPathsForCCA[forloopcount]
    iSnapshot=ListOfSnapshotsNumbersForCCA[forloopcount]
    r_max=200
    z_width=200
    filedir='/home1/08289/tg875885/radial_to_rotating_flows/Aharon/Outputs2'
    SnapDirect=SnapDirect.replace("/","-")
    SnapDirect=SnapDirect.replace(".hdf5","")
    projection_output_filename=SnapDirect
    
    # Create input dictionary for FIRE studio from snapshot and snapdict
    studiodic = {}
    studiodic['Coordinates']    = Coordinates
    studiodic['Masses']         = GasMasses*NeutralH
    studiodic['BoxSize']        = Snap1.BoxSize()
    studiodic['SmoothingLength']= SmoothingLengths 
    
    # Create N_HI projection 
    mystudio=GasStudio(
             snapdir, 
             snapnum=int(iSnapshot),
             snapdict=studiodic,
             datadir=filedir,
             frame_half_width=r_max,
             frame_depth=z_width,
             quantity_name='Masses',
             take_log_of_quantity=False, 
             galaxy_extractor=False,
             pixels=1200,
             single_image='Density',
             overwrite=True,
             savefig=False,      
             use_hsml=True,
             intermediate_file_name=projection_output_filename,
             )
    
    # This part intended to plot the HI projection with Firestudio and save .png and .hdf5 files
    # .hdf5 file will contain the following things: 
    # A grid of column densities, The side length/the positions the column densities are at, The redshift, The simulation halo mass (its viral mass),
    # The simulation name, The path to the file name should contain reference to the simulation itself and to the snapshot number. 
    fig,ax=plt.subplots()
    NHImap,_=mystudio.projectImage([])
    NHImap+=log((X*un.Msun*un.pc**-2/cons.m_p/1e10*h).to('cm**-2').value) # Units fix
    NHImap=np.where(NHImap<-1000,0,NHImap)
    NHImap=NHImap.T
    Xs=np.linspace(-r_max,r_max,NHImap.shape[0])
    Ys=np.linspace(-r_max,r_max,NHImap.shape[1])
    # This part calculates the S of DLA in Rvir/ the total S of Rvir (it will be useful in the calculations)
    def CalculateTheDistanceBetweenTwoPoints(x1,y1,x2,y2): # Function that calculates the distance (R⊥) between 2 points (x1,y1,x2,y2)
        r1=np.sqrt(x1**2+y1**2)
        r2=np.sqrt(x2**2+y2**2)
        Distance=np.abs(r1-r2)
        return Distance
    CountOfDLAInRvir=0
    CountOfTotalSightsInRvir=0
    ListOfIndexes=np.argwhere(NHImap)
    for i in range(0,len(ListOfIndexes)-1):
        xIndex=ListOfIndexes[i][0]
        yIndex=ListOfIndexes[i][1]
        d=CalculateTheDistanceBetweenTwoPoints(Xs[xIndex],Ys[yIndex],0,0)
        if(d<Rvir):
            CountOfTotalSightsInRvir=CountOfTotalSightsInRvir+1
            if(NHImap[xIndex][yIndex]>20.3):
                CountOfDLAInRvir=CountOfDLAInRvir+1
    print("The total sightlines in Rvir area is:",CountOfTotalSightsInRvir)
    print("The count of DLA sightlines in Rvir area is:",CountOfDLAInRvir)
    SDLADivideSRvir=CountOfDLAInRvir/CountOfTotalSightsInRvir
    print("S of DLA in Rvir / S Rvir=",SDLADivideSRvir)
    # This part prepares the graph
    plt.rcParams['mathtext.fontset']='dejavuserif'
    plt.pcolormesh(Xs,Ys,NHImap,cmap='viridis',norm=pl.Normalize(13.5,21.5))
    cbar=pl.colorbar()
    cbar.set_label("log "+r"$N_{\rm HI}$"+" "+"[cm"+r"$^{-2}$"+"]",fontsize=14,fontname="serif")
    plt.xlabel("X [kpc]",fontsize=12,fontname="serif")
    plt.ylabel("Y [kpc]",fontsize=12,fontname="serif")
    title="z= "+str(round(z,2))+"   ;   M= "+str(Mvir)+r" $M_{☉}$"+"\n"+r"$R_{vir}$"+" Area: "+r"$\frac{S_{DLA}}{S_{Total}}=$"+str(round(SDLADivideSRvir,2))
    plt.title(title)
    a_circle=plt.Circle((0,0),Rvir,fill=False,color='white') # Draw white circle in Rvir of the halo
    ax.add_patch(a_circle) # Adds the circle to the plot
    plt.savefig('/home1/08289/tg875885/radial_to_rotating_flows/Aharon/Outputs2/'+SnapDirect+'.png')
    plt.show()
    plt.clf()
    plt.cla()
    plt.close()
    
    xlist=Coordinates[:,0]
    ylist=Coordinates[:,1]
    data=Verdict.Dict({'hist': NHImap,'x_bins': Xs,'y_bins': Ys})
    #data.to_hdf5('/home1/08289/tg875885/radial_to_rotating_flows/Aharon/Outputs2/'+SnapDirect+'.hdf5')
    print("DONE 1")
    ######################################################################################################################################
    ######################################################################################################################################
    # log(N_HI)[R⊥] - R⊥ is the distance between 2 sightlines (1 is DLA sighline and 1 is random sightline, N_HI of the random sightline)
    plt.rcParams['mathtext.fontset']='dejavuserif'
    plt.clf()
    hist=NHImap
    x_bins=Xs
    y_bins=Ys
    def CalculateTheDistanceBetweenTwoSightLines(x1,y1,x2,y2): # Function that calculates the distance (R⊥) between 2 points (x1,y1,x2,y2)
            r1=np.sqrt(x1**2+y1**2)
            r2=np.sqrt(x2**2+y2**2)
            Distance=np.abs(r1-r2)
            return Distance
    ListOfRBetweenTwoSightLines=[] # Will be the x values for the N_HI against R⊥ plot
    ListOfN_HIOfTheAnotherLine=[] # Will be the y values for the N_HI against R⊥ plot
    ListOfDLAIndexes=np.argwhere(hist>20.3) # Each element in this list has 2 indexs, first one for the x indexes and second one for the y indexes

    # This part will random 100 pairs in each one of the bins: [0,10], [10,20], [20,30] ...
    def RandomSightinesForSpecificBin(R_perpLower,R_perpUpper): # Function that generate 100 pairs of sightlines for a specific given bin 
        CountOfRandom100=0
        while(CountOfRandom100<100):
            DLARandomIndex=random.randint(0,len(ListOfDLAIndexes)-1) # Choose random index from the ListOfDLAIndexes list
            xDLAIndex=ListOfDLAIndexes[DLARandomIndex][0] # Find the relevant x index 
            yDLAIndex=ListOfDLAIndexes[DLARandomIndex][1] # Find the relevant y index
            # Next part will be for the Another random sighline
            xAnotherIndex=random.randint(0,hist.shape[0]-1) # Find the relevant x index for the Another sighline
            yAnotherIndex=random.randint(0,hist.shape[1]-1) # Find the relevant y index for the Another sighline
            while(hist[xAnotherIndex][yAnotherIndex]==0): # Make sure that the random sighline has no 0 value for the colmun density]
                xAnotherIndex=random.randint(0,hist.shape[0]-1) # Find the relevant x index for the Another sighline
                yAnotherIndex=random.randint(0,hist.shape[1]-1) # Find the relevant y index for the Another sighline
            R_perp=CalculateTheDistanceBetweenTwoSightLines(x_bins[xDLAIndex],y_bins[yDLAIndex],x_bins[xAnotherIndex],y_bins[yAnotherIndex])
            if(R_perp<R_perpUpper and R_perp>R_perpLower):
                if(hist[xAnotherIndex][yAnotherIndex]>20.3):
                    ListOfN_HIOfTheAnotherLine.append(max(hist[xAnotherIndex][yAnotherIndex],hist[xDLAIndex][yDLAIndex])) # Insert the N_HI of the Another sightline to the final y list
                                                                                                                          # If both of them are DLAs it will take the highst N_HI
                else:
                    ListOfN_HIOfTheAnotherLine.append(hist[xAnotherIndex][yAnotherIndex])
                ListOfRBetweenTwoSightLines.append(R_perp) # Insert the R⊥ between the two sightlines to the final x list
                CountOfRandom100=CountOfRandom100+1
    ProjectionLimit=200            
    R_perpLower=0
    R_perpUpper=10
    count=0
    while(R_perpUpper<ProjectionLimit+10): # Generate all of the sightlines in the projection limit (100 pairs for each bin, each bin=10kpc)
        RandomSightinesForSpecificBin(R_perpLower,R_perpUpper)
        print("done",count)
        count=count+1
        R_perpLower=R_perpLower+10
        R_perpUpper=R_perpUpper+10

    fig=plt.figure(constrained_layout=True) # Creates the figure
    ax=fig.add_gridspec(5,3) # Creates the figure structure
    #########################################################################################################################################
    # Covering Factor for DLA
    ax2=fig.add_subplot(ax[0,0:3])
    # This part calculates the covering factor in each range
    Count_0_10=0 # For the total points in the R_perpendicular area
    RSum_0_10=0 # To calculate the mean of the R in this area
    Count2_0_10=0 # Only for the points with HI>10**17.2

    Count_10_20=0 # For the total points in the R_perpendicular area
    RSum_10_20=0 # To calculate the mean of the R in this area
    Count2_10_20=0 # Only for the points with HI>10**17.2

    Count_20_30=0 # For the total points in the R_perpendicular area
    RSum_20_30=0 # To calculate the mean of the R in this area
    Count2_20_30=0 # Only for the points with HI>10**17.2

    Count_30_40=0 # For the total points in the R_perpendicular area
    RSum_30_40=0 # To calculate the mean of the R in this area
    Count2_30_40=0 # Only for the points with HI>10**17.2

    Count_40_50=0 # For the total points in the R_perpendicular area
    RSum_40_50=0 # To calculate the mean of the R in this area
    Count2_40_50=0 # Only for the points with HI>10**17.2

    Count_50_60=0 # For the total points in the R_perpendicular area
    RSum_50_60=0 # To calculate the mean of the R in this area
    Count2_50_60=0 # Only for the points with HI>10**17.2

    Count_60_70=0 # For the total points in the R_perpendicular area
    RSum_60_70=0 # To calculate the mean of the R in this area
    Count2_60_70=0 # Only for the points with HI>10**17.2

    Count_70_80=0 # For the total points in the R_perpendicular area
    RSum_70_80=0 # To calculate the mean of the R in this area
    Count2_70_80=0 # Only for the points with HI>10**17.2

    Count_80_90=0 # For the total points in the R_perpendicular area
    RSum_80_90=0 # To calculate the mean of the R in this area
    Count2_80_90=0 # Only for the points with HI>10**17.2

    Count_90_100=0 # For the total points in the R_perpendicular area
    RSum_90_100=0 # To calculate the mean of the R in this area
    Count2_90_100=0 # Only for the points with HI>10**17.2

    Count_100_120=0 # For the total points in the R_perpendicular area
    RSum_100_120=0 # To calculate the mean of the R in this area
    Count2_100_120=0 # Only for the points with HI>10**17.2

    Count_120_140=0 # For the total points in the R_perpendicular area
    RSum_120_140=0 # To calculate the mean of the R in this area
    Count2_120_140=0 # Only for the points with HI>10**17.2

    Count_140_160=0 # For the total points in the R_perpendicular area
    RSum_140_160=0 # To calculate the mean of the R in this area
    Count2_140_160=0 # Only for the points with HI>10**17.2

    Count_160_180=0 # For the total points in the R_perpendicular area
    RSum_160_180=0 # To calculate the mean of the R in this area
    Count2_160_180=0 # Only for the points with HI>10**17.2

    Count_180_200=0 # For the total points in the R_perpendicular area
    RSum_180_200=0 # To calculate the mean of the R in this area
    Count2_180_200=0 # Only for the points with HI>10**17.2

    # This for loop calculates for each R_perp area how many total points there are, how many points are at least DLAs and what is the sum of R in each area (later to calculate the mean)
    for i in range(0,len(ListOfRBetweenTwoSightLines)):
        if(ListOfRBetweenTwoSightLines[i]<10):
            Count_0_10=Count_0_10+1
            RSum_0_10=RSum_0_10+ListOfRBetweenTwoSightLines[i]
            if(ListOfN_HIOfTheAnotherLine[i]>20.3):
                Count2_0_10=Count2_0_10+1

        if(ListOfRBetweenTwoSightLines[i]>10 and ListOfRBetweenTwoSightLines[i]<20):
            Count_10_20=Count_10_20+1
            RSum_10_20=RSum_10_20+ListOfRBetweenTwoSightLines[i]
            if(ListOfN_HIOfTheAnotherLine[i]>20.3):
                Count2_10_20=Count2_10_20+1

        if(ListOfRBetweenTwoSightLines[i]>20 and ListOfRBetweenTwoSightLines[i]<30):
            Count_20_30=Count_20_30+1
            RSum_20_30=RSum_20_30+ListOfRBetweenTwoSightLines[i]
            if(ListOfN_HIOfTheAnotherLine[i]>20.3):
                Count2_20_30=Count2_20_30+1

        if(ListOfRBetweenTwoSightLines[i]>30 and ListOfRBetweenTwoSightLines[i]<40):
            Count_30_40=Count_30_40+1
            RSum_30_40=RSum_30_40+ListOfRBetweenTwoSightLines[i]
            if(ListOfN_HIOfTheAnotherLine[i]>20.3):
                Count2_30_40=Count2_30_40+1    

        if(ListOfRBetweenTwoSightLines[i]>40 and ListOfRBetweenTwoSightLines[i]<50):
            Count_40_50=Count_40_50+1
            RSum_40_50=RSum_40_50+ListOfRBetweenTwoSightLines[i]
            if(ListOfN_HIOfTheAnotherLine[i]>20.3):
                Count2_40_50=Count2_40_50+1

        if(ListOfRBetweenTwoSightLines[i]>50 and ListOfRBetweenTwoSightLines[i]<60):
            Count_50_60=Count_50_60+1
            RSum_50_60=RSum_50_60+ListOfRBetweenTwoSightLines[i]
            if(ListOfN_HIOfTheAnotherLine[i]>20.3):
                Count2_50_60=Count2_50_60+1

        if(ListOfRBetweenTwoSightLines[i]>60 and ListOfRBetweenTwoSightLines[i]<70):
            Count_60_70=Count_60_70+1
            RSum_60_70=RSum_60_70+ListOfRBetweenTwoSightLines[i]
            if(ListOfN_HIOfTheAnotherLine[i]>20.3):
                Count2_60_70=Count2_60_70+1

        if(ListOfRBetweenTwoSightLines[i]>70 and ListOfRBetweenTwoSightLines[i]<80):
            Count_70_80=Count_70_80+1
            RSum_70_80=RSum_70_80+ListOfRBetweenTwoSightLines[i]
            if(ListOfN_HIOfTheAnotherLine[i]>20.3):
                Count2_70_80=Count2_70_80+1

        if(ListOfRBetweenTwoSightLines[i]>80 and ListOfRBetweenTwoSightLines[i]<90):
            Count_80_90=Count_80_90+1
            RSum_80_90=RSum_80_90+ListOfRBetweenTwoSightLines[i]
            if(ListOfN_HIOfTheAnotherLine[i]>20.3):
                Count2_80_90=Count2_80_90+1    

        if(ListOfRBetweenTwoSightLines[i]>90 and ListOfRBetweenTwoSightLines[i]<100):
            Count_90_100=Count_90_100+1
            RSum_90_100=RSum_90_100+ListOfRBetweenTwoSightLines[i]
            if(ListOfN_HIOfTheAnotherLine[i]>20.3):
                Count2_90_100=Count2_90_100+1

        if(ListOfRBetweenTwoSightLines[i]>100 and ListOfRBetweenTwoSightLines[i]<120):
            Count_100_120=Count_100_120+1
            RSum_100_120=RSum_100_120+ListOfRBetweenTwoSightLines[i]
            if(ListOfN_HIOfTheAnotherLine[i]>20.3):
                Count2_100_120=Count2_100_120+1

        if(ListOfRBetweenTwoSightLines[i]>120 and ListOfRBetweenTwoSightLines[i]<140):
            Count_120_140=Count_120_140+1
            RSum_120_140=RSum_120_140+ListOfRBetweenTwoSightLines[i]
            if(ListOfN_HIOfTheAnotherLine[i]>20.3):
                Count2_120_140=Count2_120_140+1

        if(ListOfRBetweenTwoSightLines[i]>140 and ListOfRBetweenTwoSightLines[i]<160):
            Count_140_160=Count_140_160+1
            RSum_140_160=RSum_140_160+ListOfRBetweenTwoSightLines[i]
            if(ListOfN_HIOfTheAnotherLine[i]>20.3):
                Count2_140_160=Count2_140_160+1

        if(ListOfRBetweenTwoSightLines[i]>160 and ListOfRBetweenTwoSightLines[i]<180):
            Count_160_180=Count_160_180+1
            RSum_160_180=RSum_160_180+ListOfRBetweenTwoSightLines[i]
            if(ListOfN_HIOfTheAnotherLine[i]>20.3):
                Count2_160_180=Count2_160_180+1    

        if(ListOfRBetweenTwoSightLines[i]>180 and ListOfRBetweenTwoSightLines[i]<200):
            Count_180_200=Count_180_200+1
            RSum_180_200=RSum_180_200+ListOfRBetweenTwoSightLines[i]
            if(ListOfN_HIOfTheAnotherLine[i]>20.3):
                Count2_180_200=Count2_180_200+1

    # This if part calculates the final x (that will be the maen of R in specific area) and y (the covering factor- how many point from all the pints are at least LLAs)
    if(Count_0_10!=0):
        A1x_0_10=RSum_0_10/Count_0_10
        A1y_0_10=Count2_0_10/Count_0_10
    else:
        A1x_0_10=5
        A1y_0_10=0

    if(Count_10_20!=0):
        A1x_10_20=RSum_10_20/Count_10_20
        A1y_10_20=Count2_10_20/Count_10_20
    else:
        A1x_10_20=10+5
        A1y_10_20=0    

    if(Count_20_30!=0):
        A1x_20_30=RSum_20_30/Count_20_30
        A1y_20_30=Count2_20_30/Count_20_30
    else:
        A1x_20_30=20+5
        A1y_20_30=0

    if(Count_30_40!=0):
        A1x_30_40=RSum_30_40/Count_30_40
        A1y_30_40=Count2_30_40/Count_30_40
    else:
        A1x_30_40=30+5
        A1y_30_40=0

    if(Count_40_50!=0):
        A1x_40_50=RSum_40_50/Count_40_50
        A1y_40_50=Count2_40_50/Count_40_50
    else:
        A1x_40_50=40+5
        A1y_40_50=0

    if(Count_50_60!=0):
        A1x_50_60=RSum_50_60/Count_50_60
        A1y_50_60=Count2_50_60/Count_50_60
    else:
        A1x_50_60=50+5
        A1y_50_60=0

    if(Count_60_70!=0):
        A1x_60_70=RSum_60_70/Count_60_70
        A1y_60_70=Count2_60_70/Count_60_70
    else:
        A1x_60_70=60+5
        A1y_60_70=0

    if(Count_70_80!=0):
        A1x_70_80=RSum_70_80/Count_70_80
        A1y_70_80=Count2_70_80/Count_70_80
    else:
        A1x_70_80=70+5
        A1y_70_80=0    

    if(Count_80_90!=0):
        A1x_80_90=RSum_80_90/Count_80_90
        A1y_80_90=Count2_80_90/Count_80_90
    else:
        A1x_80_90=80+5
        A1y_80_90=0

    if(Count_90_100!=0):
        A1x_90_100=RSum_90_100/Count_90_100
        A1y_90_100=Count2_90_100/Count_90_100
    else:
        A1x_90_100=90+5
        A1y_90_100=0

    if(Count_100_120!=0):
        A1x_100_120=RSum_100_120/Count_100_120
        A1y_100_120=Count2_100_120/Count_100_120
    else:
        A1x_100_120=90+10
        A1y_100_120=0 

    if(Count_120_140!=0):
        A1x_120_140=RSum_120_140/Count_120_140
        A1y_120_140=Count2_120_140/Count_120_140
    else:
        A1x_120_140=120+10
        A1y_120_140=0

    if(Count_140_160!=0):
        A1x_140_160=RSum_140_160/Count_140_160
        A1y_140_160=Count2_140_160/Count_140_160
    else:
        A1x_140_160=140+10
        A1y_140_160=0

    if(Count_160_180!=0):
        A1x_160_180=RSum_160_180/Count_160_180
        A1y_160_180=Count2_160_180/Count_160_180
    else:
        A1x_160_180=160+10
        A1y_160_180=0

    if(Count_180_200!=0):
        A1x_180_200=RSum_180_200/Count_180_200
        A1y_180_200=Count2_180_200/Count_180_200
    else:
        A1x_180_200=180+10
        A1y_180_200=0

    # This part prepears the plot axies and labels
    A1xList=[A1x_0_10,A1x_10_20,A1x_20_30,A1x_30_40,A1x_40_50,A1x_50_60,A1x_60_70,A1x_70_80,A1x_80_90,A1x_90_100,
            A1x_100_120,A1x_120_140,A1x_140_160,A1x_160_180,A1x_180_200] # List of x for the covering factor plot

    A1yList=[A1y_0_10,A1y_10_20,A1y_20_30,A1y_30_40,A1y_40_50,A1y_50_60,A1y_60_70,A1y_70_80,A1y_80_90,A1y_90_100,
            A1y_100_120,A1y_120_140,A1y_140_160,A1y_160_180,A1y_180_200] # List of y for the covering factor plot

    #print("X values (the mean of R_perp in each bin) for the covering factor plot are: ",AxList)
    #print("Y values (the covering factor in each bin) for the covering factor plot are ",AyList)
    plt.scatter(A1xList,A1yList,color="black") # Plot the points in the covering factor plot       
    plt.xlim([0,200])
    plt.ylim([0.0,1.0])
    plt.ylabel(r"$C_f$"+" (DLA)",fontsize=12,fontname="serif")
    plt.xticks([25,50,75,100,125,150,175,200], [" "," "," "," "," "," "," "," "])

    # Next part will plot the horizontal and vertical lines in the covering factor graph for the x errors
    plt.hlines(A1y_0_10,xmin=0,xmax=10,color='black',linestyle='-')
    plt.vlines(10,ymin=A1y_0_10-0.1,ymax=A1y_0_10+0.1,color='black',linestyle='-')

    plt.hlines(A1y_10_20,xmin=10,xmax=20,color='black',linestyle='-')
    plt.vlines(10,ymin=A1y_10_20-0.1,ymax=A1y_10_20+0.1,color='black',linestyle='-')
    plt.vlines(20,ymin=A1y_10_20-0.1,ymax=A1y_10_20+0.1,color='black',linestyle='-')

    plt.hlines(A1y_20_30,xmin=20,xmax=30,color='black',linestyle='-')
    plt.vlines(20,ymin=A1y_20_30-0.1,ymax=A1y_20_30+0.1,color='black',linestyle='-')
    plt.vlines(30,ymin=A1y_20_30-0.1,ymax=A1y_20_30+0.1,color='black',linestyle='-')

    plt.hlines(A1y_30_40,xmin=30,xmax=40,color='black',linestyle='-')
    plt.vlines(30,ymin=A1y_30_40-0.1,ymax=A1y_30_40+0.1,color='black',linestyle='-')
    plt.vlines(40,ymin=A1y_30_40-0.1,ymax=A1y_30_40+0.1,color='black',linestyle='-')

    plt.hlines(A1y_40_50,xmin=40,xmax=50,color='black',linestyle='-')
    plt.vlines(40,ymin=A1y_40_50-0.1,ymax=A1y_40_50+0.1,color='black',linestyle='-')
    plt.vlines(50,ymin=A1y_40_50-0.1,ymax=A1y_40_50+0.1,color='black',linestyle='-')

    plt.hlines(A1y_50_60,xmin=50,xmax=60,color='black',linestyle='-')
    plt.vlines(50,ymin=A1y_50_60-0.1,ymax=A1y_50_60+0.1,color='black',linestyle='-')
    plt.vlines(60,ymin=A1y_50_60-0.1,ymax=A1y_50_60+0.1,color='black',linestyle='-')

    plt.hlines(A1y_60_70,xmin=60,xmax=70,color='black',linestyle='-')
    plt.vlines(60,ymin=A1y_60_70-0.1,ymax=A1y_60_70+0.1,color='black',linestyle='-')
    plt.vlines(70,ymin=A1y_60_70-0.1,ymax=A1y_60_70+0.1,color='black',linestyle='-')

    plt.hlines(A1y_70_80,xmin=70,xmax=80,color='black',linestyle='-')
    plt.vlines(70,ymin=A1y_70_80-0.1,ymax=A1y_70_80+0.1,color='black',linestyle='-')
    plt.vlines(80,ymin=A1y_70_80-0.1,ymax=A1y_70_80+0.1,color='black',linestyle='-')

    plt.hlines(A1y_80_90,xmin=80,xmax=90,color='black',linestyle='-')
    plt.vlines(80,ymin=A1y_80_90-0.1,ymax=A1y_80_90+0.1,color='black',linestyle='-')
    plt.vlines(90,ymin=A1y_80_90-0.1,ymax=A1y_80_90+0.1,color='black',linestyle='-')

    plt.hlines(A1y_90_100,xmin=90,xmax=100,color='black',linestyle='-')
    plt.vlines(90,ymin=A1y_90_100-0.1,ymax=A1y_90_100+0.1,color='black',linestyle='-')
    plt.vlines(100,ymin=A1y_90_100-0.1,ymax=A1y_90_100+0.1,color='black',linestyle='-')

    plt.hlines(A1y_100_120,xmin=100,xmax=120,color='black',linestyle='-')
    plt.vlines(100,ymin=A1y_100_120-0.1,ymax=A1y_100_120+0.1,color='black',linestyle='-')
    plt.vlines(120,ymin=A1y_100_120-0.1,ymax=A1y_100_120+0.1,color='black',linestyle='-')

    plt.hlines(A1y_120_140,xmin=120,xmax=140,color='black',linestyle='-')
    plt.vlines(120,ymin=A1y_120_140-0.1,ymax=A1y_120_140+0.1,color='black',linestyle='-')
    plt.vlines(140,ymin=A1y_120_140-0.1,ymax=A1y_120_140+0.1,color='black',linestyle='-')

    plt.hlines(A1y_140_160,xmin=140,xmax=160,color='black',linestyle='-')
    plt.vlines(140,ymin=A1y_140_160-0.1,ymax=A1y_140_160+0.1,color='black',linestyle='-')
    plt.vlines(160,ymin=A1y_140_160-0.1,ymax=A1y_140_160+0.1,color='black',linestyle='-')

    plt.hlines(A1y_160_180,xmin=160,xmax=180,color='black',linestyle='-')
    plt.vlines(160,ymin=A1y_160_180-0.1,ymax=A1y_160_180+0.1,color='black',linestyle='-')
    plt.vlines(180,ymin=A1y_160_180-0.1,ymax=A1y_160_180+0.1,color='black',linestyle='-')

    plt.hlines(A1y_180_200,xmin=180,xmax=200,color='black',linestyle='-')
    plt.vlines(180,ymin=A1y_180_200-0.1,ymax=A1y_180_200+0.1,color='black',linestyle='-')
    plt.vlines(200,ymin=A1y_180_200-0.1,ymax=A1y_180_200+0.1,color='black',linestyle='-')
    #########################################################################################################################################
    # Covering Factor for LLS
    ax1=fig.add_subplot(ax[1,0:3]) # Grid for the covering factor plot
    # This part calculates the covering factor in each range
    Count_0_10=0 # For the total points in the R_perpendicular area
    RSum_0_10=0 # To calculate the mean of the R in this area
    Count2_0_10=0 # Only for the points with HI>10**17.2

    Count_10_20=0 # For the total points in the R_perpendicular area
    RSum_10_20=0 # To calculate the mean of the R in this area
    Count2_10_20=0 # Only for the points with HI>10**17.2

    Count_20_30=0 # For the total points in the R_perpendicular area
    RSum_20_30=0 # To calculate the mean of the R in this area
    Count2_20_30=0 # Only for the points with HI>10**17.2

    Count_30_40=0 # For the total points in the R_perpendicular area
    RSum_30_40=0 # To calculate the mean of the R in this area
    Count2_30_40=0 # Only for the points with HI>10**17.2

    Count_40_50=0 # For the total points in the R_perpendicular area
    RSum_40_50=0 # To calculate the mean of the R in this area
    Count2_40_50=0 # Only for the points with HI>10**17.2

    Count_50_60=0 # For the total points in the R_perpendicular area
    RSum_50_60=0 # To calculate the mean of the R in this area
    Count2_50_60=0 # Only for the points with HI>10**17.2

    Count_60_70=0 # For the total points in the R_perpendicular area
    RSum_60_70=0 # To calculate the mean of the R in this area
    Count2_60_70=0 # Only for the points with HI>10**17.2

    Count_70_80=0 # For the total points in the R_perpendicular area
    RSum_70_80=0 # To calculate the mean of the R in this area
    Count2_70_80=0 # Only for the points with HI>10**17.2

    Count_80_90=0 # For the total points in the R_perpendicular area
    RSum_80_90=0 # To calculate the mean of the R in this area
    Count2_80_90=0 # Only for the points with HI>10**17.2

    Count_90_100=0 # For the total points in the R_perpendicular area
    RSum_90_100=0 # To calculate the mean of the R in this area
    Count2_90_100=0 # Only for the points with HI>10**17.2

    Count_100_120=0 # For the total points in the R_perpendicular area
    RSum_100_120=0 # To calculate the mean of the R in this area
    Count2_100_120=0 # Only for the points with HI>10**17.2

    Count_120_140=0 # For the total points in the R_perpendicular area
    RSum_120_140=0 # To calculate the mean of the R in this area
    Count2_120_140=0 # Only for the points with HI>10**17.2

    Count_140_160=0 # For the total points in the R_perpendicular area
    RSum_140_160=0 # To calculate the mean of the R in this area
    Count2_140_160=0 # Only for the points with HI>10**17.2

    Count_160_180=0 # For the total points in the R_perpendicular area
    RSum_160_180=0 # To calculate the mean of the R in this area
    Count2_160_180=0 # Only for the points with HI>10**17.2

    Count_180_200=0 # For the total points in the R_perpendicular area
    RSum_180_200=0 # To calculate the mean of the R in this area
    Count2_180_200=0 # Only for the points with HI>10**17.2

    # This for loop calculates for each R_perp area how many total points there are, how many points are at least LLAs and what is the sum of R in each area (later to calculate the mean)
    for i in range(0,len(ListOfRBetweenTwoSightLines)):
        if(ListOfRBetweenTwoSightLines[i]<10):
            Count_0_10=Count_0_10+1
            RSum_0_10=RSum_0_10+ListOfRBetweenTwoSightLines[i]
            if(ListOfN_HIOfTheAnotherLine[i]>17.2):
                Count2_0_10=Count2_0_10+1

        if(ListOfRBetweenTwoSightLines[i]>10 and ListOfRBetweenTwoSightLines[i]<20):
            Count_10_20=Count_10_20+1
            RSum_10_20=RSum_10_20+ListOfRBetweenTwoSightLines[i]
            if(ListOfN_HIOfTheAnotherLine[i]>17.2):
                Count2_10_20=Count2_10_20+1

        if(ListOfRBetweenTwoSightLines[i]>20 and ListOfRBetweenTwoSightLines[i]<30):
            Count_20_30=Count_20_30+1
            RSum_20_30=RSum_20_30+ListOfRBetweenTwoSightLines[i]
            if(ListOfN_HIOfTheAnotherLine[i]>17.2):
                Count2_20_30=Count2_20_30+1

        if(ListOfRBetweenTwoSightLines[i]>30 and ListOfRBetweenTwoSightLines[i]<40):
            Count_30_40=Count_30_40+1
            RSum_30_40=RSum_30_40+ListOfRBetweenTwoSightLines[i]
            if(ListOfN_HIOfTheAnotherLine[i]>17.2):
                Count2_30_40=Count2_30_40+1    

        if(ListOfRBetweenTwoSightLines[i]>40 and ListOfRBetweenTwoSightLines[i]<50):
            Count_40_50=Count_40_50+1
            RSum_40_50=RSum_40_50+ListOfRBetweenTwoSightLines[i]
            if(ListOfN_HIOfTheAnotherLine[i]>17.2):
                Count2_40_50=Count2_40_50+1

        if(ListOfRBetweenTwoSightLines[i]>50 and ListOfRBetweenTwoSightLines[i]<60):
            Count_50_60=Count_50_60+1
            RSum_50_60=RSum_50_60+ListOfRBetweenTwoSightLines[i]
            if(ListOfN_HIOfTheAnotherLine[i]>17.2):
                Count2_50_60=Count2_50_60+1

        if(ListOfRBetweenTwoSightLines[i]>60 and ListOfRBetweenTwoSightLines[i]<70):
            Count_60_70=Count_60_70+1
            RSum_60_70=RSum_60_70+ListOfRBetweenTwoSightLines[i]
            if(ListOfN_HIOfTheAnotherLine[i]>17.2):
                Count2_60_70=Count2_60_70+1

        if(ListOfRBetweenTwoSightLines[i]>70 and ListOfRBetweenTwoSightLines[i]<80):
            Count_70_80=Count_70_80+1
            RSum_70_80=RSum_70_80+ListOfRBetweenTwoSightLines[i]
            if(ListOfN_HIOfTheAnotherLine[i]>17.2):
                Count2_70_80=Count2_70_80+1

        if(ListOfRBetweenTwoSightLines[i]>80 and ListOfRBetweenTwoSightLines[i]<90):
            Count_80_90=Count_80_90+1
            RSum_80_90=RSum_80_90+ListOfRBetweenTwoSightLines[i]
            if(ListOfN_HIOfTheAnotherLine[i]>17.2):
                Count2_80_90=Count2_80_90+1    

        if(ListOfRBetweenTwoSightLines[i]>90 and ListOfRBetweenTwoSightLines[i]<100):
            Count_90_100=Count_90_100+1
            RSum_90_100=RSum_90_100+ListOfRBetweenTwoSightLines[i]
            if(ListOfN_HIOfTheAnotherLine[i]>17.2):
                Count2_90_100=Count2_90_100+1

        if(ListOfRBetweenTwoSightLines[i]>100 and ListOfRBetweenTwoSightLines[i]<120):
            Count_100_120=Count_100_120+1
            RSum_100_120=RSum_100_120+ListOfRBetweenTwoSightLines[i]
            if(ListOfN_HIOfTheAnotherLine[i]>17.2):
                Count2_100_120=Count2_100_120+1

        if(ListOfRBetweenTwoSightLines[i]>120 and ListOfRBetweenTwoSightLines[i]<140):
            Count_120_140=Count_120_140+1
            RSum_120_140=RSum_120_140+ListOfRBetweenTwoSightLines[i]
            if(ListOfN_HIOfTheAnotherLine[i]>17.2):
                Count2_120_140=Count2_120_140+1

        if(ListOfRBetweenTwoSightLines[i]>140 and ListOfRBetweenTwoSightLines[i]<160):
            Count_140_160=Count_140_160+1
            RSum_140_160=RSum_140_160+ListOfRBetweenTwoSightLines[i]
            if(ListOfN_HIOfTheAnotherLine[i]>17.2):
                Count2_140_160=Count2_140_160+1

        if(ListOfRBetweenTwoSightLines[i]>160 and ListOfRBetweenTwoSightLines[i]<180):
            Count_160_180=Count_160_180+1
            RSum_160_180=RSum_160_180+ListOfRBetweenTwoSightLines[i]
            if(ListOfN_HIOfTheAnotherLine[i]>17.2):
                Count2_160_180=Count2_160_180+1    

        if(ListOfRBetweenTwoSightLines[i]>180 and ListOfRBetweenTwoSightLines[i]<200):
            Count_180_200=Count_180_200+1
            RSum_180_200=RSum_180_200+ListOfRBetweenTwoSightLines[i]
            if(ListOfN_HIOfTheAnotherLine[i]>17.2):
                Count2_180_200=Count2_180_200+1

    # This if part calculates the final x (that will be the maen of R in specific area) and y (the covering factor- how many point from all the pints are at least LLAs)
    if(Count_0_10!=0):
        A2x_0_10=RSum_0_10/Count_0_10
        A2y_0_10=Count2_0_10/Count_0_10
    else:
        A2x_0_10=5
        A2y_0_10=0

    if(Count_10_20!=0):
        A2x_10_20=RSum_10_20/Count_10_20
        A2y_10_20=Count2_10_20/Count_10_20
    else:
        A2x_10_20=10+5
        A2y_10_20=0    

    if(Count_20_30!=0):
        A2x_20_30=RSum_20_30/Count_20_30
        A2y_20_30=Count2_20_30/Count_20_30
    else:
        A2x_20_30=20+5
        A2y_20_30=0

    if(Count_30_40!=0):
        A2x_30_40=RSum_30_40/Count_30_40
        A2y_30_40=Count2_30_40/Count_30_40
    else:
        A2x_30_40=30+5
        A2y_30_40=0

    if(Count_40_50!=0):
        A2x_40_50=RSum_40_50/Count_40_50
        A2y_40_50=Count2_40_50/Count_40_50
    else:
        A2x_40_50=40+5
        A2y_40_50=0

    if(Count_50_60!=0):
        A2x_50_60=RSum_50_60/Count_50_60
        A2y_50_60=Count2_50_60/Count_50_60
    else:
        A2x_50_60=50+5
        A2y_50_60=0

    if(Count_60_70!=0):
        A2x_60_70=RSum_60_70/Count_60_70
        A2y_60_70=Count2_60_70/Count_60_70
    else:
        A2x_60_70=60+5
        A2y_60_70=0

    if(Count_70_80!=0):
        A2x_70_80=RSum_70_80/Count_70_80
        A2y_70_80=Count2_70_80/Count_70_80
    else:
        A2x_70_80=70+5
        A2y_70_80=0    

    if(Count_80_90!=0):
        A2x_80_90=RSum_80_90/Count_80_90
        A2y_80_90=Count2_80_90/Count_80_90
    else:
        A2x_80_90=80+5
        A2y_80_90=0

    if(Count_90_100!=0):
        A2x_90_100=RSum_90_100/Count_90_100
        A2y_90_100=Count2_90_100/Count_90_100
    else:
        A2x_90_100=90+5
        A2y_90_100=0

    if(Count_100_120!=0):
        A2x_100_120=RSum_100_120/Count_100_120
        A2y_100_120=Count2_100_120/Count_100_120
    else:
        A2x_100_120=90+10
        A2y_100_120=0 

    if(Count_120_140!=0):
        A2x_120_140=RSum_120_140/Count_120_140
        A2y_120_140=Count2_120_140/Count_120_140
    else:
        A2x_120_140=120+10
        A2y_120_140=0

    if(Count_140_160!=0):
        A2x_140_160=RSum_140_160/Count_140_160
        A2y_140_160=Count2_140_160/Count_140_160
    else:
        A2x_140_160=140+10
        A2y_140_160=0

    if(Count_160_180!=0):
        A2x_160_180=RSum_160_180/Count_160_180
        A2y_160_180=Count2_160_180/Count_160_180
    else:
        A2x_160_180=160+10
        A2y_160_180=0

    if(Count_180_200!=0):
        A2x_180_200=RSum_180_200/Count_180_200
        A2y_180_200=Count2_180_200/Count_180_200
    else:
        A2x_180_200=180+10
        A2y_180_200=0

    # This part prepears the plot axies and labels
    A2xList=[A2x_0_10,A2x_10_20,A2x_20_30,A2x_30_40,A2x_40_50,A2x_50_60,A2x_60_70,A2x_70_80,A2x_80_90,A2x_90_100,
            A2x_100_120,A2x_120_140,A2x_140_160,A2x_160_180,A2x_180_200] # List of x for the covering factor plot

    A2yList=[A2y_0_10,A2y_10_20,A2y_20_30,A2y_30_40,A2y_40_50,A2y_50_60,A2y_60_70,A2y_70_80,A2y_80_90,A2y_90_100,
            A2y_100_120,A2y_120_140,A2y_140_160,A2y_160_180,A2y_180_200] # List of y for the covering factor plot

    plt.scatter(A2xList,A2yList,color="black") # Plot the points in the covering factor plot       
    plt.xlim([0,200])
    plt.ylim([0.0,1.0])
    plt.ylabel(r"$C_f$"+" (LLS)",fontsize=12,fontname="serif")
    plt.xticks([25,50,75,100,125,150,175,200], [" "," "," "," "," "," "," "," "])

    # Next part will plot the horizontal and vertical lines in the covering factor graph for the x errors
    plt.hlines(A2y_0_10,xmin=0,xmax=10,color='black',linestyle='-')
    plt.vlines(10,ymin=A2y_0_10-0.1,ymax=A2y_0_10+0.1,color='black',linestyle='-')

    plt.hlines(A2y_10_20,xmin=10,xmax=20,color='black',linestyle='-')
    plt.vlines(10,ymin=A2y_10_20-0.1,ymax=A2y_10_20+0.1,color='black',linestyle='-')
    plt.vlines(20,ymin=A2y_10_20-0.1,ymax=A2y_10_20+0.1,color='black',linestyle='-')

    plt.hlines(A2y_20_30,xmin=20,xmax=30,color='black',linestyle='-')
    plt.vlines(20,ymin=A2y_20_30-0.1,ymax=A2y_20_30+0.1,color='black',linestyle='-')
    plt.vlines(30,ymin=A2y_20_30-0.1,ymax=A2y_20_30+0.1,color='black',linestyle='-')

    plt.hlines(A2y_30_40,xmin=30,xmax=40,color='black',linestyle='-')
    plt.vlines(30,ymin=A2y_30_40-0.1,ymax=A2y_30_40+0.1,color='black',linestyle='-')
    plt.vlines(40,ymin=A2y_30_40-0.1,ymax=A2y_30_40+0.1,color='black',linestyle='-')

    plt.hlines(A2y_40_50,xmin=40,xmax=50,color='black',linestyle='-')
    plt.vlines(40,ymin=A2y_40_50-0.1,ymax=A2y_40_50+0.1,color='black',linestyle='-')
    plt.vlines(50,ymin=A2y_40_50-0.1,ymax=A2y_40_50+0.1,color='black',linestyle='-')

    plt.hlines(A2y_50_60,xmin=50,xmax=60,color='black',linestyle='-')
    plt.vlines(50,ymin=A2y_50_60-0.1,ymax=A2y_50_60+0.1,color='black',linestyle='-')
    plt.vlines(60,ymin=A2y_50_60-0.1,ymax=A2y_50_60+0.1,color='black',linestyle='-')

    plt.hlines(A2y_60_70,xmin=60,xmax=70,color='black',linestyle='-')
    plt.vlines(60,ymin=A2y_60_70-0.1,ymax=A2y_60_70+0.1,color='black',linestyle='-')
    plt.vlines(70,ymin=A2y_60_70-0.1,ymax=A2y_60_70+0.1,color='black',linestyle='-')

    plt.hlines(A2y_70_80,xmin=70,xmax=80,color='black',linestyle='-')
    plt.vlines(70,ymin=A2y_70_80-0.1,ymax=A2y_70_80+0.1,color='black',linestyle='-')
    plt.vlines(80,ymin=A2y_70_80-0.1,ymax=A2y_70_80+0.1,color='black',linestyle='-')

    plt.hlines(A2y_80_90,xmin=80,xmax=90,color='black',linestyle='-')
    plt.vlines(80,ymin=A2y_80_90-0.1,ymax=A2y_80_90+0.1,color='black',linestyle='-')
    plt.vlines(90,ymin=A2y_80_90-0.1,ymax=A2y_80_90+0.1,color='black',linestyle='-')

    plt.hlines(A2y_90_100,xmin=90,xmax=100,color='black',linestyle='-')
    plt.vlines(90,ymin=A2y_90_100-0.1,ymax=A2y_90_100+0.1,color='black',linestyle='-')
    plt.vlines(100,ymin=A2y_90_100-0.1,ymax=A2y_90_100+0.1,color='black',linestyle='-')

    plt.hlines(A2y_100_120,xmin=100,xmax=120,color='black',linestyle='-')
    plt.vlines(100,ymin=A2y_100_120-0.1,ymax=A2y_100_120+0.1,color='black',linestyle='-')
    plt.vlines(120,ymin=A2y_100_120-0.1,ymax=A2y_100_120+0.1,color='black',linestyle='-')

    plt.hlines(A2y_120_140,xmin=120,xmax=140,color='black',linestyle='-')
    plt.vlines(120,ymin=A2y_120_140-0.1,ymax=A2y_120_140+0.1,color='black',linestyle='-')
    plt.vlines(140,ymin=A2y_120_140-0.1,ymax=A2y_120_140+0.1,color='black',linestyle='-')

    plt.hlines(A2y_140_160,xmin=140,xmax=160,color='black',linestyle='-')
    plt.vlines(140,ymin=A2y_140_160-0.1,ymax=A2y_140_160+0.1,color='black',linestyle='-')
    plt.vlines(160,ymin=A2y_140_160-0.1,ymax=A2y_140_160+0.1,color='black',linestyle='-')

    plt.hlines(A2y_160_180,xmin=160,xmax=180,color='black',linestyle='-')
    plt.vlines(160,ymin=A2y_160_180-0.1,ymax=A2y_160_180+0.1,color='black',linestyle='-')
    plt.vlines(180,ymin=A2y_160_180-0.1,ymax=A2y_160_180+0.1,color='black',linestyle='-')

    plt.hlines(A2y_180_200,xmin=180,xmax=200,color='black',linestyle='-')
    plt.vlines(180,ymin=A2y_180_200-0.1,ymax=A2y_180_200+0.1,color='black',linestyle='-')
    plt.vlines(200,ymin=A2y_180_200-0.1,ymax=A2y_180_200+0.1,color='black',linestyle='-')

    # Next part will handle the main graph of N_HI
    ax1 = fig.add_subplot(ax[2:,0:3]) # Plot for the main graph
    plt.scatter(ListOfRBetweenTwoSightLines,ListOfN_HIOfTheAnotherLine) # Plot the N_HI against R⊥
    plt.xlabel(r"$R_⊥$"+" [kpc]"+" ("+r"$R_\perp$"+" from DLA)",fontsize=12,fontname="serif")
    plt.ylabel("log "+r"$N_{\rm HI}$"+" "+"[cm"+r"$^{-2}$"+"]",fontsize=12,fontname="serif")
    #plt.yticks([14,15,16,17,18,19,20,21,22],["14","15","16","17","18","19","20","21","22"])
    plt.autoscale(enable=True,axis='y',tight=True) # Allows the axhspan to go up till the edge
    plt.axhspan(20.3,22,color='gray',alpha=0.5,label="DLA") # Mark the area of the DLAs
    plt.axhspan(17.2,20.3,color='lightgray',alpha=0.5,label="LLS") # Mark the area of the LLSs
    lightgray_patch=mpatches.Patch(color='lightgray',label='LLS') # Creates patch for the LLS
    gray_patch=mpatches.Patch(color='gray',label='DLA') # Creates patch for the DLA
    plt.legend(handles=[gray_patch,lightgray_patch],loc='lower right')
    plt.xlim([0,200])
    plt.ylim([13.5,21.5])
    #plt.savefig('/home1/08289/tg875885/radial_to_rotating_flows/Aharon/Outputs2/'+SnapDirect+"~log(N_HI)[R⊥].png")
    plt.show()
    plt.clf()
    plt.cla()
    plt.close()
    print("DONE 2")
    ######################################################################################################################################
    ######################################################################################################################################
    # log(N_HI)[R⊥] - R⊥ is the distance between a random sightline and the center of the halo (N_HI is of the random sightline)
    plt.rcParams['mathtext.fontset']='dejavuserif'
    plt.clf()
    hist=NHImap
    x_bins=Xs
    y_bins=Ys
    def CalculateTheDistanceBetweenTwoPoints(x1,y1,x2,y2): # Function that calculates the distance between 2 points (x1,y1,x2,y2)
            r1=np.sqrt(x1**2+y1**2)
            r2=np.sqrt(x2**2+y2**2)
            Distance=np.abs(r1-r2)
            return Distance
    ListOfRBetweenSightlineAndTheCenter=[] # Will be the x values for the N_HI against R⊥ plot
    ListOfN_HIOfTheSightline=[] # Will be the y values for the N_HI against R⊥ plot

    # This part will random 100 pairs in each one of the bins: [0,10], [10,20], [20,30], [30,40] ...
    def RandomSightinesForSpecificBin(R_perpLower,R_perpUpper): # Function that generate 100 pairs of sightlines for a specific given bin 
        CountOfRandom100=0
        while(CountOfRandom100<100):
            xSightlineIndex=random.randint(0,hist.shape[0]-1) # Find the relevant x index for the sighline
            ySightlineIndex=random.randint(0,hist.shape[1]-1) # Find the relevant y index for the sighline
            while(hist[xSightlineIndex][ySightlineIndex]==0): # Make sure that the random sighline has no 0 value for the colmun density
                xSightlineIndex=random.randint(0,hist.shape[0]-1) # Find the relevant x index for the sighline
                ySightlineIndex=random.randint(0,hist.shape[1]-1) # Find the relevant y index for the sighline
            R_perp=CalculateTheDistanceBetweenTwoPoints(0,0,x_bins[xSightlineIndex],y_bins[ySightlineIndex])
            if(R_perp<R_perpUpper and R_perp>R_perpLower):
                ListOfN_HIOfTheSightline.append(hist[xSightlineIndex][ySightlineIndex]) # Insert the N_HI of the sightline to the final y list
                ListOfRBetweenSightlineAndTheCenter.append(R_perp) # Insert the R⊥ between the sightline and the center to the final x list
                CountOfRandom100=CountOfRandom100+1       
    ProjectionLimit=200            
    R_perpLower=0
    R_perpUpper=10
    count=0
    while(R_perpUpper<ProjectionLimit+10): # Generate all of the sightlines in the projection limit (100 pairs for each bin, each bin=10kpc)
        RandomSightinesForSpecificBin(R_perpLower,R_perpUpper)
        print("done",count)
        count=count+1
        R_perpLower=R_perpLower+10
        R_perpUpper=R_perpUpper+10

    fig=plt.figure(constrained_layout=True) # Creates the figure
    ax=fig.add_gridspec(5,3) # Creates the figure structure
    #########################################################################################################################################
    # Covering Factor for DLA
    ax2=fig.add_subplot(ax[0,0:3]) # Grid for the covering factor plot
    # This part calculates the covering factor in each range
    Count_0_10=0 # For the total points in the R_perpendicular area
    RSum_0_10=0 # To calculate the mean of the R in this area
    Count2_0_10=0 # Only for the points with HI>10**17.2

    Count_10_20=0 # For the total points in the R_perpendicular area
    RSum_10_20=0 # To calculate the mean of the R in this area
    Count2_10_20=0 # Only for the points with HI>10**17.2

    Count_20_30=0 # For the total points in the R_perpendicular area
    RSum_20_30=0 # To calculate the mean of the R in this area
    Count2_20_30=0 # Only for the points with HI>10**17.2

    Count_30_40=0 # For the total points in the R_perpendicular area
    RSum_30_40=0 # To calculate the mean of the R in this area
    Count2_30_40=0 # Only for the points with HI>10**17.2

    Count_40_50=0 # For the total points in the R_perpendicular area
    RSum_40_50=0 # To calculate the mean of the R in this area
    Count2_40_50=0 # Only for the points with HI>10**17.2

    Count_50_60=0 # For the total points in the R_perpendicular area
    RSum_50_60=0 # To calculate the mean of the R in this area
    Count2_50_60=0 # Only for the points with HI>10**17.2

    Count_60_70=0 # For the total points in the R_perpendicular area
    RSum_60_70=0 # To calculate the mean of the R in this area
    Count2_60_70=0 # Only for the points with HI>10**17.2

    Count_70_80=0 # For the total points in the R_perpendicular area
    RSum_70_80=0 # To calculate the mean of the R in this area
    Count2_70_80=0 # Only for the points with HI>10**17.2

    Count_80_90=0 # For the total points in the R_perpendicular area
    RSum_80_90=0 # To calculate the mean of the R in this area
    Count2_80_90=0 # Only for the points with HI>10**17.2

    Count_90_100=0 # For the total points in the R_perpendicular area
    RSum_90_100=0 # To calculate the mean of the R in this area
    Count2_90_100=0 # Only for the points with HI>10**17.2

    Count_100_120=0 # For the total points in the R_perpendicular area
    RSum_100_120=0 # To calculate the mean of the R in this area
    Count2_100_120=0 # Only for the points with HI>10**17.2

    Count_120_140=0 # For the total points in the R_perpendicular area
    RSum_120_140=0 # To calculate the mean of the R in this area
    Count2_120_140=0 # Only for the points with HI>10**17.2

    Count_140_160=0 # For the total points in the R_perpendicular area
    RSum_140_160=0 # To calculate the mean of the R in this area
    Count2_140_160=0 # Only for the points with HI>10**17.2

    Count_160_180=0 # For the total points in the R_perpendicular area
    RSum_160_180=0 # To calculate the mean of the R in this area
    Count2_160_180=0 # Only for the points with HI>10**17.2

    Count_180_200=0 # For the total points in the R_perpendicular area
    RSum_180_200=0 # To calculate the mean of the R in this area
    Count2_180_200=0 # Only for the points with HI>10**17.2

    # This for loop calculates for each R_perp area how many total points there are, how many points are at least DLAs and what is the sum of R in each area (later to calculate the mean)
    for i in range(0,len(ListOfRBetweenSightlineAndTheCenter)):
        if(ListOfRBetweenSightlineAndTheCenter[i]<10):
            Count_0_10=Count_0_10+1
            RSum_0_10=RSum_0_10+ListOfRBetweenSightlineAndTheCenter[i]
            if(ListOfN_HIOfTheSightline[i]>20.3):
                Count2_0_10=Count2_0_10+1

        if(ListOfRBetweenSightlineAndTheCenter[i]>10 and ListOfRBetweenSightlineAndTheCenter[i]<20):
            Count_10_20=Count_10_20+1
            RSum_10_20=RSum_10_20+ListOfRBetweenSightlineAndTheCenter[i]
            if(ListOfN_HIOfTheSightline[i]>20.3):
                Count2_10_20=Count2_10_20+1

        if(ListOfRBetweenSightlineAndTheCenter[i]>20 and ListOfRBetweenSightlineAndTheCenter[i]<30):
            Count_20_30=Count_20_30+1
            RSum_20_30=RSum_20_30+ListOfRBetweenSightlineAndTheCenter[i]
            if(ListOfN_HIOfTheSightline[i]>20.3):
                Count2_20_30=Count2_20_30+1

        if(ListOfRBetweenSightlineAndTheCenter[i]>30 and ListOfRBetweenSightlineAndTheCenter[i]<40):
            Count_30_40=Count_30_40+1
            RSum_30_40=RSum_30_40+ListOfRBetweenSightlineAndTheCenter[i]
            if(ListOfN_HIOfTheSightline[i]>20.3):
                Count2_30_40=Count2_30_40+1    

        if(ListOfRBetweenSightlineAndTheCenter[i]>40 and ListOfRBetweenSightlineAndTheCenter[i]<50):
            Count_40_50=Count_40_50+1
            RSum_40_50=RSum_40_50+ListOfRBetweenSightlineAndTheCenter[i]
            if(ListOfN_HIOfTheSightline[i]>20.3):
                Count2_40_50=Count2_40_50+1

        if(ListOfRBetweenSightlineAndTheCenter[i]>50 and ListOfRBetweenSightlineAndTheCenter[i]<60):
            Count_50_60=Count_50_60+1
            RSum_50_60=RSum_50_60+ListOfRBetweenSightlineAndTheCenter[i]
            if(ListOfN_HIOfTheSightline[i]>20.3):
                Count2_50_60=Count2_50_60+1

        if(ListOfRBetweenSightlineAndTheCenter[i]>60 and ListOfRBetweenSightlineAndTheCenter[i]<70):
            Count_60_70=Count_60_70+1
            RSum_60_70=RSum_60_70+ListOfRBetweenSightlineAndTheCenter[i]
            if(ListOfN_HIOfTheSightline[i]>20.3):
                Count2_60_70=Count2_60_70+1

        if(ListOfRBetweenSightlineAndTheCenter[i]>70 and ListOfRBetweenSightlineAndTheCenter[i]<80):
            Count_70_80=Count_70_80+1
            RSum_70_80=RSum_70_80+ListOfRBetweenSightlineAndTheCenter[i]
            if(ListOfN_HIOfTheSightline[i]>20.3):
                Count2_70_80=Count2_70_80+1

        if(ListOfRBetweenSightlineAndTheCenter[i]>80 and ListOfRBetweenSightlineAndTheCenter[i]<90):
            Count_80_90=Count_80_90+1
            RSum_80_90=RSum_80_90+ListOfRBetweenSightlineAndTheCenter[i]
            if(ListOfN_HIOfTheSightline[i]>20.3):
                Count2_80_90=Count2_80_90+1    

        if(ListOfRBetweenSightlineAndTheCenter[i]>90 and ListOfRBetweenSightlineAndTheCenter[i]<100):
            Count_90_100=Count_90_100+1
            RSum_90_100=RSum_90_100+ListOfRBetweenSightlineAndTheCenter[i]
            if(ListOfN_HIOfTheSightline[i]>20.3):
                Count2_90_100=Count2_90_100+1

        if(ListOfRBetweenSightlineAndTheCenter[i]>100 and ListOfRBetweenSightlineAndTheCenter[i]<120):
            Count_100_120=Count_100_120+1
            RSum_100_120=RSum_100_120+ListOfRBetweenSightlineAndTheCenter[i]
            if(ListOfN_HIOfTheSightline[i]>20.3):
                Count2_100_120=Count2_100_120+1       

        if(ListOfRBetweenSightlineAndTheCenter[i]>120 and ListOfRBetweenSightlineAndTheCenter[i]<140):
            Count_120_140=Count_120_140+1
            RSum_120_140=RSum_120_140+ListOfRBetweenSightlineAndTheCenter[i]
            if(ListOfN_HIOfTheSightline[i]>20.3):
                Count2_120_140=Count2_120_140+1    

        if(ListOfRBetweenSightlineAndTheCenter[i]>140 and ListOfRBetweenSightlineAndTheCenter[i]<160):
            Count_140_160=Count_140_160+1
            RSum_140_160=RSum_140_160+ListOfRBetweenSightlineAndTheCenter[i]
            if(ListOfN_HIOfTheSightline[i]>20.3):
                Count2_140_160=Count2_140_160+1

        if(ListOfRBetweenSightlineAndTheCenter[i]>160 and ListOfRBetweenSightlineAndTheCenter[i]<180):
            Count_160_180=Count_160_180+1
            RSum_160_180=RSum_160_180+ListOfRBetweenSightlineAndTheCenter[i]
            if(ListOfN_HIOfTheSightline[i]>20.3):
                Count2_160_180=Count2_160_180+1

        if(ListOfRBetweenSightlineAndTheCenter[i]>180 and ListOfRBetweenSightlineAndTheCenter[i]<200):
            Count_180_200=Count_180_200+1
            RSum_180_200=RSum_180_200+ListOfRBetweenSightlineAndTheCenter[i]
            if(ListOfN_HIOfTheSightline[i]>20.3):
                Count2_180_200=Count2_180_200+1       

    # This if part calculates the final x (that will be the maen of R in specific area) and y (the covering factor- how many point from all the pints are at least LLAs)
    if(Count_0_10!=0):
        B1x_0_10=RSum_0_10/Count_0_10
        B1y_0_10=Count2_0_10/Count_0_10
    else:
        B1x_0_10=5
        B1y_0_10=0

    if(Count_10_20!=0):
        B1x_10_20=RSum_10_20/Count_10_20
        B1y_10_20=Count2_10_20/Count_10_20
    else:
        B1x_10_20=10+5
        B1y_10_20=0    

    if(Count_20_30!=0):
        B1x_20_30=RSum_20_30/Count_20_30
        B1y_20_30=Count2_20_30/Count_20_30
    else:
        B1x_20_30=20+5
        B1y_20_30=0

    if(Count_30_40!=0):
        B1x_30_40=RSum_30_40/Count_30_40
        B1y_30_40=Count2_30_40/Count_30_40
    else:
        B1x_30_40=30+5
        B1y_30_40=0

    if(Count_40_50!=0):
        B1x_40_50=RSum_40_50/Count_40_50
        B1y_40_50=Count2_40_50/Count_40_50
    else:
        B1x_40_50=40+5
        B1y_40_50=0

    if(Count_50_60!=0):
        B1x_50_60=RSum_50_60/Count_50_60
        B1y_50_60=Count2_50_60/Count_50_60
    else:
        B1x_50_60=50+5
        B1y_50_60=0

    if(Count_60_70!=0):
        B1x_60_70=RSum_60_70/Count_60_70
        B1y_60_70=Count2_60_70/Count_60_70
    else:
        B1x_60_70=60+5
        B1y_60_70=0

    if(Count_70_80!=0):
        B1x_70_80=RSum_70_80/Count_70_80
        B1y_70_80=Count2_70_80/Count_70_80
    else:
        B1x_70_80=70+5
        B1y_70_80=0    

    if(Count_80_90!=0):
        B1x_80_90=RSum_80_90/Count_80_90
        B1y_80_90=Count2_80_90/Count_80_90
    else:
        B1x_80_90=80+5
        B1y_80_90=0

    if(Count_90_100!=0):
        B1x_90_100=RSum_90_100/Count_90_100
        B1y_90_100=Count2_90_100/Count_90_100
    else:
        B1x_90_100=90+5
        B1y_90_100=0

    if(Count_100_120!=0):
        B1x_100_120=RSum_100_120/Count_100_120
        B1y_100_120=Count2_100_120/Count_100_120
    else:
        B1x_100_120=90+10
        B1y_100_120=0 

    if(Count_120_140!=0):
        B1x_120_140=RSum_120_140/Count_120_140
        B1y_120_140=Count2_120_140/Count_120_140
    else:
        B1x_120_140=120+10
        B1y_120_140=0

    if(Count_140_160!=0):
        B1x_140_160=RSum_140_160/Count_140_160
        B1y_140_160=Count2_140_160/Count_140_160
    else:
        B1x_140_160=140+10
        B1y_140_160=0

    if(Count_160_180!=0):
        B1x_160_180=RSum_160_180/Count_160_180
        B1y_160_180=Count2_160_180/Count_160_180
    else:
        B1x_160_180=160+10
        B1y_160_180=0

    if(Count_180_200!=0):
        B1x_180_200=RSum_180_200/Count_180_200
        B1y_180_200=Count2_180_200/Count_180_200
    else:
        B1x_180_200=180+10
        B1y_180_200=0

    # This part prepears the plot axies and labels
    B1xList=[B1x_0_10,B1x_10_20,B1x_20_30,B1x_30_40,B1x_40_50,B1x_50_60,B1x_60_70,B1x_70_80,B1x_80_90,B1x_90_100,
            B1x_100_120,B1x_120_140,B1x_140_160,B1x_160_180,B1x_180_200] # List of x for the covering factor plot

    B1yList=[B1y_0_10,B1y_10_20,B1y_20_30,B1y_30_40,B1y_40_50,B1y_50_60,B1y_60_70,B1y_70_80,B1y_80_90,B1y_90_100,
            B1y_100_120,B1y_120_140,B1y_140_160,B1y_160_180,B1y_180_200] # List of y for the covering factor plot

    #print("X values (the mean of R_perp in each bin) for the covering factor plot are: ",AxList)
    #print("Y values (the covering factor in each bin) for the covering factor plot are ",AyList)
    plt.scatter(B1xList,B1yList,color="black") # Plot the points in the covering factor plot       
    plt.xlim([0,200])
    plt.ylim([0.0,1.0])
    plt.ylabel(r"$C_f$"+" (DLA)",fontsize=12,fontname="serif")
    plt.xticks([25,50,75,100,125,150,175,200], [" "," "," "," "," "," "," "," "])

    # Next part will plot the horizontal and vertical lines in the covering factor graph for the x errors
    plt.hlines(B1y_0_10,xmin=0,xmax=10,color='black',linestyle='-')
    plt.vlines(10,ymin=B1y_0_10-0.1,ymax=B1y_0_10+0.1,color='black',linestyle='-')

    plt.hlines(B1y_10_20,xmin=10,xmax=20,color='black',linestyle='-')
    plt.vlines(10,ymin=B1y_10_20-0.1,ymax=B1y_10_20+0.1,color='black',linestyle='-')
    plt.vlines(20,ymin=B1y_10_20-0.1,ymax=B1y_10_20+0.1,color='black',linestyle='-')

    plt.hlines(B1y_20_30,xmin=20,xmax=30,color='black',linestyle='-')
    plt.vlines(20,ymin=B1y_20_30-0.1,ymax=B1y_20_30+0.1,color='black',linestyle='-')
    plt.vlines(30,ymin=B1y_20_30-0.1,ymax=B1y_20_30+0.1,color='black',linestyle='-')

    plt.hlines(B1y_30_40,xmin=30,xmax=40,color='black',linestyle='-')
    plt.vlines(30,ymin=B1y_30_40-0.1,ymax=B1y_30_40+0.1,color='black',linestyle='-')
    plt.vlines(40,ymin=B1y_30_40-0.1,ymax=B1y_30_40+0.1,color='black',linestyle='-')

    plt.hlines(B1y_40_50,xmin=40,xmax=50,color='black',linestyle='-')
    plt.vlines(40,ymin=B1y_40_50-0.1,ymax=B1y_40_50+0.1,color='black',linestyle='-')
    plt.vlines(50,ymin=B1y_40_50-0.1,ymax=B1y_40_50+0.1,color='black',linestyle='-')

    plt.hlines(B1y_50_60,xmin=50,xmax=60,color='black',linestyle='-')
    plt.vlines(50,ymin=B1y_50_60-0.1,ymax=B1y_50_60+0.1,color='black',linestyle='-')
    plt.vlines(60,ymin=B1y_50_60-0.1,ymax=B1y_50_60+0.1,color='black',linestyle='-')

    plt.hlines(B1y_60_70,xmin=60,xmax=70,color='black',linestyle='-')
    plt.vlines(60,ymin=B1y_60_70-0.1,ymax=B1y_60_70+0.1,color='black',linestyle='-')
    plt.vlines(70,ymin=B1y_60_70-0.1,ymax=B1y_60_70+0.1,color='black',linestyle='-')

    plt.hlines(B1y_70_80,xmin=70,xmax=80,color='black',linestyle='-')
    plt.vlines(70,ymin=B1y_70_80-0.1,ymax=B1y_70_80+0.1,color='black',linestyle='-')
    plt.vlines(80,ymin=B1y_70_80-0.1,ymax=B1y_70_80+0.1,color='black',linestyle='-')

    plt.hlines(B1y_80_90,xmin=80,xmax=90,color='black',linestyle='-')
    plt.vlines(80,ymin=B1y_80_90-0.1,ymax=B1y_80_90+0.1,color='black',linestyle='-')
    plt.vlines(90,ymin=B1y_80_90-0.1,ymax=B1y_80_90+0.1,color='black',linestyle='-')

    plt.hlines(B1y_90_100,xmin=90,xmax=100,color='black',linestyle='-')
    plt.vlines(90,ymin=B1y_90_100-0.1,ymax=B1y_90_100+0.1,color='black',linestyle='-')
    plt.vlines(100,ymin=B1y_90_100-0.1,ymax=B1y_90_100+0.1,color='black',linestyle='-')

    plt.hlines(B1y_100_120,xmin=100,xmax=120,color='black',linestyle='-')
    plt.vlines(100,ymin=B1y_100_120-0.1,ymax=B1y_100_120+0.1,color='black',linestyle='-')
    plt.vlines(120,ymin=B1y_100_120-0.1,ymax=B1y_100_120+0.1,color='black',linestyle='-')

    plt.hlines(B1y_120_140,xmin=120,xmax=140,color='black',linestyle='-')
    plt.vlines(120,ymin=B1y_120_140-0.1,ymax=B1y_120_140+0.1,color='black',linestyle='-')
    plt.vlines(140,ymin=B1y_120_140-0.1,ymax=B1y_120_140+0.1,color='black',linestyle='-')

    plt.hlines(B1y_140_160,xmin=140,xmax=160,color='black',linestyle='-')
    plt.vlines(140,ymin=B1y_140_160-0.1,ymax=B1y_140_160+0.1,color='black',linestyle='-')
    plt.vlines(160,ymin=B1y_140_160-0.1,ymax=B1y_140_160+0.1,color='black',linestyle='-')

    plt.hlines(B1y_160_180,xmin=160,xmax=180,color='black',linestyle='-')
    plt.vlines(160,ymin=B1y_160_180-0.1,ymax=B1y_160_180+0.1,color='black',linestyle='-')
    plt.vlines(180,ymin=B1y_160_180-0.1,ymax=B1y_160_180+0.1,color='black',linestyle='-')

    plt.hlines(B1y_180_200,xmin=180,xmax=200,color='black',linestyle='-')
    plt.vlines(180,ymin=B1y_180_200-0.1,ymax=B1y_180_200+0.1,color='black',linestyle='-')
    plt.vlines(200,ymin=B1y_180_200-0.1,ymax=B1y_180_200+0.1,color='black',linestyle='-')
    #########################################################################################################################################
    # Covering Factor for LLS
    ax1=fig.add_subplot(ax[1,0:3]) # Grid for the covering factor plot
    # This part calculates the covering factor in each range
    Count_0_10=0 # For the total points in the R_perpendicular area
    RSum_0_10=0 # To calculate the mean of the R in this area
    Count2_0_10=0 # Only for the points with HI>10**17.2

    Count_10_20=0 # For the total points in the R_perpendicular area
    RSum_10_20=0 # To calculate the mean of the R in this area
    Count2_10_20=0 # Only for the points with HI>10**17.2

    Count_20_30=0 # For the total points in the R_perpendicular area
    RSum_20_30=0 # To calculate the mean of the R in this area
    Count2_20_30=0 # Only for the points with HI>10**17.2

    Count_30_40=0 # For the total points in the R_perpendicular area
    RSum_30_40=0 # To calculate the mean of the R in this area
    Count2_30_40=0 # Only for the points with HI>10**17.2

    Count_40_50=0 # For the total points in the R_perpendicular area
    RSum_40_50=0 # To calculate the mean of the R in this area
    Count2_40_50=0 # Only for the points with HI>10**17.2

    Count_50_60=0 # For the total points in the R_perpendicular area
    RSum_50_60=0 # To calculate the mean of the R in this area
    Count2_50_60=0 # Only for the points with HI>10**17.2

    Count_60_70=0 # For the total points in the R_perpendicular area
    RSum_60_70=0 # To calculate the mean of the R in this area
    Count2_60_70=0 # Only for the points with HI>10**17.2

    Count_70_80=0 # For the total points in the R_perpendicular area
    RSum_70_80=0 # To calculate the mean of the R in this area
    Count2_70_80=0 # Only for the points with HI>10**17.2

    Count_80_90=0 # For the total points in the R_perpendicular area
    RSum_80_90=0 # To calculate the mean of the R in this area
    Count2_80_90=0 # Only for the points with HI>10**17.2

    Count_90_100=0 # For the total points in the R_perpendicular area
    RSum_90_100=0 # To calculate the mean of the R in this area
    Count2_90_100=0 # Only for the points with HI>10**17.2

    Count_100_120=0 # For the total points in the R_perpendicular area
    RSum_100_120=0 # To calculate the mean of the R in this area
    Count2_100_120=0 # Only for the points with HI>10**17.2

    Count_120_140=0 # For the total points in the R_perpendicular area
    RSum_120_140=0 # To calculate the mean of the R in this area
    Count2_120_140=0 # Only for the points with HI>10**17.2

    Count_140_160=0 # For the total points in the R_perpendicular area
    RSum_140_160=0 # To calculate the mean of the R in this area
    Count2_140_160=0 # Only for the points with HI>10**17.2

    Count_160_180=0 # For the total points in the R_perpendicular area
    RSum_160_180=0 # To calculate the mean of the R in this area
    Count2_160_180=0 # Only for the points with HI>10**17.2

    Count_180_200=0 # For the total points in the R_perpendicular area
    RSum_180_200=0 # To calculate the mean of the R in this area
    Count2_180_200=0 # Only for the points with HI>10**17.2

    # This for loop calculates for each R_perp area how many total points there are, how many points are at least LLAs and what is the sum of R in each area (later to calculate the mean)
    for i in range(0,len(ListOfRBetweenSightlineAndTheCenter)):
        if(ListOfRBetweenSightlineAndTheCenter[i]<10):
            Count_0_10=Count_0_10+1
            RSum_0_10=RSum_0_10+ListOfRBetweenSightlineAndTheCenter[i]
            if(ListOfN_HIOfTheSightline[i]>17.2):
                Count2_0_10=Count2_0_10+1

        if(ListOfRBetweenSightlineAndTheCenter[i]>10 and ListOfRBetweenSightlineAndTheCenter[i]<20):
            Count_10_20=Count_10_20+1
            RSum_10_20=RSum_10_20+ListOfRBetweenSightlineAndTheCenter[i]
            if(ListOfN_HIOfTheSightline[i]>17.2):
                Count2_10_20=Count2_10_20+1

        if(ListOfRBetweenSightlineAndTheCenter[i]>20 and ListOfRBetweenSightlineAndTheCenter[i]<30):
            Count_20_30=Count_20_30+1
            RSum_20_30=RSum_20_30+ListOfRBetweenSightlineAndTheCenter[i]
            if(ListOfN_HIOfTheSightline[i]>17.2):
                Count2_20_30=Count2_20_30+1

        if(ListOfRBetweenSightlineAndTheCenter[i]>30 and ListOfRBetweenSightlineAndTheCenter[i]<40):
            Count_30_40=Count_30_40+1
            RSum_30_40=RSum_30_40+ListOfRBetweenSightlineAndTheCenter[i]
            if(ListOfN_HIOfTheSightline[i]>17.2):
                Count2_30_40=Count2_30_40+1    

        if(ListOfRBetweenSightlineAndTheCenter[i]>40 and ListOfRBetweenSightlineAndTheCenter[i]<50):
            Count_40_50=Count_40_50+1
            RSum_40_50=RSum_40_50+ListOfRBetweenSightlineAndTheCenter[i]
            if(ListOfN_HIOfTheSightline[i]>17.2):
                Count2_40_50=Count2_40_50+1

        if(ListOfRBetweenSightlineAndTheCenter[i]>50 and ListOfRBetweenSightlineAndTheCenter[i]<60):
            Count_50_60=Count_50_60+1
            RSum_50_60=RSum_50_60+ListOfRBetweenSightlineAndTheCenter[i]
            if(ListOfN_HIOfTheSightline[i]>17.2):
                Count2_50_60=Count2_50_60+1

        if(ListOfRBetweenSightlineAndTheCenter[i]>60 and ListOfRBetweenSightlineAndTheCenter[i]<70):
            Count_60_70=Count_60_70+1
            RSum_60_70=RSum_60_70+ListOfRBetweenSightlineAndTheCenter[i]
            if(ListOfN_HIOfTheSightline[i]>17.2):
                Count2_60_70=Count2_60_70+1

        if(ListOfRBetweenSightlineAndTheCenter[i]>70 and ListOfRBetweenSightlineAndTheCenter[i]<80):
            Count_70_80=Count_70_80+1
            RSum_70_80=RSum_70_80+ListOfRBetweenSightlineAndTheCenter[i]
            if(ListOfN_HIOfTheSightline[i]>17.2):
                Count2_70_80=Count2_70_80+1

        if(ListOfRBetweenSightlineAndTheCenter[i]>80 and ListOfRBetweenSightlineAndTheCenter[i]<90):
            Count_80_90=Count_80_90+1
            RSum_80_90=RSum_80_90+ListOfRBetweenSightlineAndTheCenter[i]
            if(ListOfN_HIOfTheSightline[i]>17.2):
                Count2_80_90=Count2_80_90+1    

        if(ListOfRBetweenSightlineAndTheCenter[i]>90 and ListOfRBetweenSightlineAndTheCenter[i]<100):
            Count_90_100=Count_90_100+1
            RSum_90_100=RSum_90_100+ListOfRBetweenSightlineAndTheCenter[i]
            if(ListOfN_HIOfTheSightline[i]>17.2):
                Count2_90_100=Count2_90_100+1

        if(ListOfRBetweenSightlineAndTheCenter[i]>100 and ListOfRBetweenSightlineAndTheCenter[i]<120):
            Count_100_120=Count_100_120+1
            RSum_100_120=RSum_100_120+ListOfRBetweenSightlineAndTheCenter[i]
            if(ListOfN_HIOfTheSightline[i]>17.2):
                Count2_100_120=Count2_100_120+1       

        if(ListOfRBetweenSightlineAndTheCenter[i]>120 and ListOfRBetweenSightlineAndTheCenter[i]<140):
            Count_120_140=Count_120_140+1
            RSum_120_140=RSum_120_140+ListOfRBetweenSightlineAndTheCenter[i]
            if(ListOfN_HIOfTheSightline[i]>17.2):
                Count2_120_140=Count2_120_140+1    

        if(ListOfRBetweenSightlineAndTheCenter[i]>140 and ListOfRBetweenSightlineAndTheCenter[i]<160):
            Count_140_160=Count_140_160+1
            RSum_140_160=RSum_140_160+ListOfRBetweenSightlineAndTheCenter[i]
            if(ListOfN_HIOfTheSightline[i]>17.2):
                Count2_140_160=Count2_140_160+1

        if(ListOfRBetweenSightlineAndTheCenter[i]>160 and ListOfRBetweenSightlineAndTheCenter[i]<180):
            Count_160_180=Count_160_180+1
            RSum_160_180=RSum_160_180+ListOfRBetweenSightlineAndTheCenter[i]
            if(ListOfN_HIOfTheSightline[i]>17.2):
                Count2_160_180=Count2_160_180+1

        if(ListOfRBetweenSightlineAndTheCenter[i]>180 and ListOfRBetweenSightlineAndTheCenter[i]<200):
            Count_180_200=Count_180_200+1
            RSum_180_200=RSum_180_200+ListOfRBetweenSightlineAndTheCenter[i]
            if(ListOfN_HIOfTheSightline[i]>17.2):
                Count2_180_200=Count2_180_200+1     

    # This if part calculates the final x (that will be the maen of R in specific area) and y (the covering factor- how many point from all the pints are at least LLAs)
    if(Count_0_10!=0):
        B2x_0_10=RSum_0_10/Count_0_10
        B2y_0_10=Count2_0_10/Count_0_10
    else:
        B2x_0_10=5
        B2y_0_10=0

    if(Count_10_20!=0):
        B2x_10_20=RSum_10_20/Count_10_20
        B2y_10_20=Count2_10_20/Count_10_20
    else:
        B2x_10_20=10+5
        B2y_10_20=0    

    if(Count_20_30!=0):
        B2x_20_30=RSum_20_30/Count_20_30
        B2y_20_30=Count2_20_30/Count_20_30
    else:
        B2x_20_30=20+5
        B2y_20_30=0

    if(Count_30_40!=0):
        B2x_30_40=RSum_30_40/Count_30_40
        B2y_30_40=Count2_30_40/Count_30_40
    else:
        B2x_30_40=30+5
        B2y_30_40=0

    if(Count_40_50!=0):
        B2x_40_50=RSum_40_50/Count_40_50
        B2y_40_50=Count2_40_50/Count_40_50
    else:
        B2x_40_50=40+5
        B2y_40_50=0

    if(Count_50_60!=0):
        B2x_50_60=RSum_50_60/Count_50_60
        B2y_50_60=Count2_50_60/Count_50_60
    else:
        B2x_50_60=50+5
        B2y_50_60=0

    if(Count_60_70!=0):
        B2x_60_70=RSum_60_70/Count_60_70
        B2y_60_70=Count2_60_70/Count_60_70
    else:
        B2x_60_70=60+5
        B2y_60_70=0

    if(Count_70_80!=0):
        B2x_70_80=RSum_70_80/Count_70_80
        B2y_70_80=Count2_70_80/Count_70_80
    else:
        B2x_70_80=70+5
        B2y_70_80=0    

    if(Count_80_90!=0):
        B2x_80_90=RSum_80_90/Count_80_90
        B2y_80_90=Count2_80_90/Count_80_90
    else:
        B2x_80_90=80+5
        B2y_80_90=0

    if(Count_90_100!=0):
        B2x_90_100=RSum_90_100/Count_90_100
        B2y_90_100=Count2_90_100/Count_90_100
    else:
        B2x_90_100=90+5
        B2y_90_100=0

    if(Count_100_120!=0):
        B2x_100_120=RSum_100_120/Count_100_120
        B2y_100_120=Count2_100_120/Count_100_120
    else:
        B2x_100_120=90+10
        B2y_100_120=0 

    if(Count_120_140!=0):
        B2x_120_140=RSum_120_140/Count_120_140
        B2y_120_140=Count2_120_140/Count_120_140
    else:
        B2x_120_140=120+10
        B2y_120_140=0

    if(Count_140_160!=0):
        B2x_140_160=RSum_140_160/Count_140_160
        B2y_140_160=Count2_140_160/Count_140_160
    else:
        B2x_140_160=140+10
        B2y_140_160=0

    if(Count_160_180!=0):
        B2x_160_180=RSum_160_180/Count_160_180
        B2y_160_180=Count2_160_180/Count_160_180
    else:
        B2x_160_180=160+10
        B2y_160_180=0

    if(Count_180_200!=0):
        B2x_180_200=RSum_180_200/Count_180_200
        B2y_180_200=Count2_180_200/Count_180_200
    else:
        B2x_180_200=180+10
        B2y_180_200=0

    # This part prepears the plot axies and labels
    B2xList=[B2x_0_10,B2x_10_20,B2x_20_30,B2x_30_40,B2x_40_50,B2x_50_60,B2x_60_70,B2x_70_80,B2x_80_90,B2x_90_100,
            B2x_100_120,B2x_120_140,B2x_140_160,B2x_160_180,B2x_180_200] # List of x for the covering factor plot

    B2yList=[B2y_0_10,B2y_10_20,B2y_20_30,B2y_30_40,B2y_40_50,B2y_50_60,B2y_60_70,B2y_70_80,B2y_80_90,B2y_90_100,
            B2y_100_120,B2y_120_140,B2y_140_160,B2y_160_180,B2y_180_200] # List of y for the covering factor plot

    #print("X values (the mean of R_perp in each bin) for the covering factor plot are: ",AxList)
    #print("Y values (the covering factor in each bin) for the covering factor plot are ",AyList)
    plt.scatter(B2xList,B2yList,color="black") # Plot the points in the covering factor plot       
    plt.xlim([0,200])
    plt.ylim([0.0,1.0])
    plt.ylabel(r"$C_f$"+" (LLS)",fontsize=12,fontname="serif")
    plt.xticks([25,50,75,100,125,150,175,200], [" "," "," "," "," "," "," "," "])

    # Next part will plot the horizontal and vertical lines in the covering factor graph for the x errors
    plt.hlines(B2y_0_10,xmin=0,xmax=10,color='black',linestyle='-')
    plt.vlines(10,ymin=B2y_0_10-0.1,ymax=B2y_0_10+0.1,color='black',linestyle='-')

    plt.hlines(B2y_10_20,xmin=10,xmax=20,color='black',linestyle='-')
    plt.vlines(10,ymin=B2y_10_20-0.1,ymax=B2y_10_20+0.1,color='black',linestyle='-')
    plt.vlines(20,ymin=B2y_10_20-0.1,ymax=B2y_10_20+0.1,color='black',linestyle='-')

    plt.hlines(B2y_20_30,xmin=20,xmax=30,color='black',linestyle='-')
    plt.vlines(20,ymin=B2y_20_30-0.1,ymax=B2y_20_30+0.1,color='black',linestyle='-')
    plt.vlines(30,ymin=B2y_20_30-0.1,ymax=B2y_20_30+0.1,color='black',linestyle='-')

    plt.hlines(B2y_30_40,xmin=30,xmax=40,color='black',linestyle='-')
    plt.vlines(30,ymin=B2y_30_40-0.1,ymax=B2y_30_40+0.1,color='black',linestyle='-')
    plt.vlines(40,ymin=B2y_30_40-0.1,ymax=B2y_30_40+0.1,color='black',linestyle='-')

    plt.hlines(B2y_40_50,xmin=40,xmax=50,color='black',linestyle='-')
    plt.vlines(40,ymin=B2y_40_50-0.1,ymax=B2y_40_50+0.1,color='black',linestyle='-')
    plt.vlines(50,ymin=B2y_40_50-0.1,ymax=B2y_40_50+0.1,color='black',linestyle='-')

    plt.hlines(B2y_50_60,xmin=50,xmax=60,color='black',linestyle='-')
    plt.vlines(50,ymin=B2y_50_60-0.1,ymax=B2y_50_60+0.1,color='black',linestyle='-')
    plt.vlines(60,ymin=B2y_50_60-0.1,ymax=B2y_50_60+0.1,color='black',linestyle='-')

    plt.hlines(B2y_60_70,xmin=60,xmax=70,color='black',linestyle='-')
    plt.vlines(60,ymin=B2y_60_70-0.1,ymax=B2y_60_70+0.1,color='black',linestyle='-')
    plt.vlines(70,ymin=B2y_60_70-0.1,ymax=B2y_60_70+0.1,color='black',linestyle='-')

    plt.hlines(B2y_70_80,xmin=70,xmax=80,color='black',linestyle='-')
    plt.vlines(70,ymin=B2y_70_80-0.1,ymax=B2y_70_80+0.1,color='black',linestyle='-')
    plt.vlines(80,ymin=B2y_70_80-0.1,ymax=B2y_70_80+0.1,color='black',linestyle='-')

    plt.hlines(B2y_80_90,xmin=80,xmax=90,color='black',linestyle='-')
    plt.vlines(80,ymin=B2y_80_90-0.1,ymax=B2y_80_90+0.1,color='black',linestyle='-')
    plt.vlines(90,ymin=B2y_80_90-0.1,ymax=B2y_80_90+0.1,color='black',linestyle='-')

    plt.hlines(B2y_90_100,xmin=90,xmax=100,color='black',linestyle='-')
    plt.vlines(90,ymin=B2y_90_100-0.1,ymax=B2y_90_100+0.1,color='black',linestyle='-')
    plt.vlines(100,ymin=B2y_90_100-0.1,ymax=B2y_90_100+0.1,color='black',linestyle='-')

    plt.hlines(B2y_100_120,xmin=100,xmax=120,color='black',linestyle='-')
    plt.vlines(100,ymin=B2y_100_120-0.1,ymax=B2y_100_120+0.1,color='black',linestyle='-')
    plt.vlines(120,ymin=B2y_100_120-0.1,ymax=B2y_100_120+0.1,color='black',linestyle='-')

    plt.hlines(B2y_120_140,xmin=120,xmax=140,color='black',linestyle='-')
    plt.vlines(120,ymin=B2y_120_140-0.1,ymax=B2y_120_140+0.1,color='black',linestyle='-')
    plt.vlines(140,ymin=B2y_120_140-0.1,ymax=B2y_120_140+0.1,color='black',linestyle='-')

    plt.hlines(B2y_140_160,xmin=140,xmax=160,color='black',linestyle='-')
    plt.vlines(140,ymin=B2y_140_160-0.1,ymax=B2y_140_160+0.1,color='black',linestyle='-')
    plt.vlines(160,ymin=B2y_140_160-0.1,ymax=B2y_140_160+0.1,color='black',linestyle='-')

    plt.hlines(B2y_160_180,xmin=160,xmax=180,color='black',linestyle='-')
    plt.vlines(160,ymin=B2y_160_180-0.1,ymax=B2y_160_180+0.1,color='black',linestyle='-')
    plt.vlines(180,ymin=B2y_160_180-0.1,ymax=B2y_160_180+0.1,color='black',linestyle='-')

    plt.hlines(B2y_180_200,xmin=180,xmax=200,color='black',linestyle='-')
    plt.vlines(180,ymin=B2y_180_200-0.1,ymax=B2y_180_200+0.1,color='black',linestyle='-')
    plt.vlines(200,ymin=B2y_180_200-0.1,ymax=B2y_180_200+0.1,color='black',linestyle='-')

    # Next part will handle the main graph of N_HI
    ax1 = fig.add_subplot(ax[2:,0:3]) # Plot for the main graph
    plt.scatter(ListOfRBetweenSightlineAndTheCenter,ListOfN_HIOfTheSightline) # Plot the N_HI against R⊥
    plt.xlabel(r"$R_⊥$"+" [kpc]"+" ("+r"$R_\perp$"+" from center)",fontsize=12,fontname="serif")
    plt.ylabel("log "+r"$N_{\rm HI}$"+" "+"[cm"+r"$^{-2}$"+"]",fontsize=12,fontname="serif")
    plt.autoscale(enable=True,axis='y',tight=True) # Allows the axhspan to go up till the edge
    plt.axhspan(20.3,23,color='gray',alpha=0.5,label="DLA") # Mark the area of the DLAs
    plt.axhspan(17.2,20.3,color='lightgray',alpha=0.5,label="LLS") # Mark the area of the LLSs
    lightgray_patch=mpatches.Patch(color='lightgray',label='LLS') # Creates patch for the LLS
    gray_patch=mpatches.Patch(color='gray',label='DLA') # Creates patch for the DLA
    plt.legend(handles=[gray_patch,lightgray_patch],loc='lower right')
    plt.xlim([0,200])
    plt.ylim([13.5,21.5])
    #plt.savefig('/home1/08289/tg875885/radial_to_rotating_flows/Aharon/Outputs2/'+SnapDirect+"~log(N_HI)[R⊥]~FromCenter.png")
    plt.show()
    plt.clf()
    plt.cla()
    plt.close()
    print("DONE 3")
    ######################################################################################################################################
    ######################################################################################################################################
    # This block plot median and 16th, 18th percentile for N_HI agains R_perp for both types of R_perp and C_f factor above it
    plt.clf()
    # Special graph colors
    rperp_dla_color1=palettable.cartocolors.qualitative.Safe_2.mpl_colors[0]
    rperp_dla_color2=palettable.cartocolors.qualitative.Safe_2.mpl_colors[1]
    fig=plt.figure(constrained_layout=True) # Creates the figure
    ax=fig.add_gridspec(5,3) # Creates the figure structure
    ########################################################################################################################################
    # C_f (of DLA) Graph for both R_perp
    ax2=fig.add_subplot(ax[0,0:3]) # Grid for the covering factor plot
    plt.xlim([0,200])
    plt.ylim([0.0,1.0])
    plt.ylabel(r"$C_f$"+" (DLA)",fontsize=12,fontname="serif")
    plt.xticks([25,50,75,100,125,150,175,200], [" "," "," "," "," "," "," "," "])

    # For A1- R_perp from DLA
    plt.scatter(A1xList,A1yList,color=rperp_dla_color1) # Plot the points in the covering factor plot     
    # Next part will plot the horizontal and vertical lines in the covering factor graph for the x errors
    plt.hlines(A1y_0_10,xmin=0,xmax=10,color=rperp_dla_color1,linestyle='-')
    plt.vlines(10,ymin=A1y_0_10-0.1,ymax=A1y_0_10+0.1,color=rperp_dla_color1,linestyle='-')

    plt.hlines(A1y_10_20,xmin=10,xmax=20,color=rperp_dla_color1,linestyle='-')
    plt.vlines(10,ymin=A1y_10_20-0.1,ymax=A1y_10_20+0.1,color=rperp_dla_color1,linestyle='-')
    plt.vlines(20,ymin=A1y_10_20-0.1,ymax=A1y_10_20+0.1,color=rperp_dla_color1,linestyle='-')

    plt.hlines(A1y_20_30,xmin=20,xmax=30,color=rperp_dla_color1,linestyle='-')
    plt.vlines(20,ymin=A1y_20_30-0.1,ymax=A1y_20_30+0.1,color=rperp_dla_color1,linestyle='-')
    plt.vlines(30,ymin=A1y_20_30-0.1,ymax=A1y_20_30+0.1,color=rperp_dla_color1,linestyle='-')

    plt.hlines(A1y_30_40,xmin=30,xmax=40,color=rperp_dla_color1,linestyle='-')
    plt.vlines(30,ymin=A1y_30_40-0.1,ymax=A1y_30_40+0.1,color=rperp_dla_color1,linestyle='-')
    plt.vlines(40,ymin=A1y_30_40-0.1,ymax=A1y_30_40+0.1,color=rperp_dla_color1,linestyle='-')

    plt.hlines(A1y_40_50,xmin=40,xmax=50,color=rperp_dla_color1,linestyle='-')
    plt.vlines(40,ymin=A1y_40_50-0.1,ymax=A1y_40_50+0.1,color=rperp_dla_color1,linestyle='-')
    plt.vlines(50,ymin=A1y_40_50-0.1,ymax=A1y_40_50+0.1,color=rperp_dla_color1,linestyle='-')

    plt.hlines(A1y_50_60,xmin=50,xmax=60,color=rperp_dla_color1,linestyle='-')
    plt.vlines(50,ymin=A1y_50_60-0.1,ymax=A1y_50_60+0.1,color=rperp_dla_color1,linestyle='-')
    plt.vlines(60,ymin=A1y_50_60-0.1,ymax=A1y_50_60+0.1,color=rperp_dla_color1,linestyle='-')

    plt.hlines(A1y_60_70,xmin=60,xmax=70,color=rperp_dla_color1,linestyle='-')
    plt.vlines(60,ymin=A1y_60_70-0.1,ymax=A1y_60_70+0.1,color=rperp_dla_color1,linestyle='-')
    plt.vlines(70,ymin=A1y_60_70-0.1,ymax=A1y_60_70+0.1,color=rperp_dla_color1,linestyle='-')

    plt.hlines(A1y_70_80,xmin=70,xmax=80,color=rperp_dla_color1,linestyle='-')
    plt.vlines(70,ymin=A1y_70_80-0.1,ymax=A1y_70_80+0.1,color=rperp_dla_color1,linestyle='-')
    plt.vlines(80,ymin=A1y_70_80-0.1,ymax=A1y_70_80+0.1,color=rperp_dla_color1,linestyle='-')

    plt.hlines(A1y_80_90,xmin=80,xmax=90,color=rperp_dla_color1,linestyle='-')
    plt.vlines(80,ymin=A1y_80_90-0.1,ymax=A1y_80_90+0.1,color=rperp_dla_color1,linestyle='-')
    plt.vlines(90,ymin=A1y_80_90-0.1,ymax=A1y_80_90+0.1,color=rperp_dla_color1,linestyle='-')

    plt.hlines(A1y_90_100,xmin=90,xmax=100,color=rperp_dla_color1,linestyle='-')
    plt.vlines(90,ymin=A1y_90_100-0.1,ymax=A1y_90_100+0.1,color=rperp_dla_color1,linestyle='-')
    plt.vlines(100,ymin=A1y_90_100-0.1,ymax=A1y_90_100+0.1,color=rperp_dla_color1,linestyle='-')

    plt.hlines(A1y_100_120,xmin=100,xmax=120,color=rperp_dla_color1,linestyle='-')
    plt.vlines(100,ymin=A1y_100_120-0.1,ymax=A1y_100_120+0.1,color=rperp_dla_color1,linestyle='-')
    plt.vlines(120,ymin=A1y_100_120-0.1,ymax=A1y_100_120+0.1,color=rperp_dla_color1,linestyle='-')

    plt.hlines(A1y_120_140,xmin=120,xmax=140,color=rperp_dla_color1,linestyle='-')
    plt.vlines(120,ymin=A1y_120_140-0.1,ymax=A1y_120_140+0.1,color=rperp_dla_color1,linestyle='-')
    plt.vlines(140,ymin=A1y_120_140-0.1,ymax=A1y_120_140+0.1,color=rperp_dla_color1,linestyle='-')

    plt.hlines(A1y_140_160,xmin=140,xmax=160,color=rperp_dla_color1,linestyle='-')
    plt.vlines(140,ymin=A1y_140_160-0.1,ymax=A1y_140_160+0.1,color=rperp_dla_color1,linestyle='-')
    plt.vlines(160,ymin=A1y_140_160-0.1,ymax=A1y_140_160+0.1,color=rperp_dla_color1,linestyle='-')

    plt.hlines(A1y_160_180,xmin=160,xmax=180,color=rperp_dla_color1,linestyle='-')
    plt.vlines(160,ymin=A1y_160_180-0.1,ymax=A1y_160_180+0.1,color=rperp_dla_color1,linestyle='-')
    plt.vlines(180,ymin=A1y_160_180-0.1,ymax=A1y_160_180+0.1,color=rperp_dla_color1,linestyle='-')

    plt.hlines(A1y_180_200,xmin=180,xmax=200,color=rperp_dla_color1,linestyle='-')
    plt.vlines(180,ymin=A1y_180_200-0.1,ymax=A1y_180_200+0.1,color=rperp_dla_color1,linestyle='-')
    plt.vlines(200,ymin=A1y_180_200-0.1,ymax=A1y_180_200+0.1,color=rperp_dla_color1,linestyle='-')

    # For B1- R_perp from center
    plt.scatter(B1xList,B1yList,color=rperp_dla_color2) # Plot the points in the covering factor plot 
    # Next part will plot the horizontal and vertical lines in the covering factor graph for the x errors
    plt.hlines(B1y_0_10,xmin=0,xmax=10,color=rperp_dla_color2,linestyle='-')
    plt.vlines(10,ymin=B1y_0_10-0.1,ymax=B1y_0_10+0.1,color=rperp_dla_color2,linestyle='-')

    plt.hlines(B1y_10_20,xmin=10,xmax=20,color=rperp_dla_color2,linestyle='-')
    plt.vlines(10,ymin=B1y_10_20-0.1,ymax=B1y_10_20+0.1,color=rperp_dla_color2,linestyle='-')
    plt.vlines(20,ymin=B1y_10_20-0.1,ymax=B1y_10_20+0.1,color=rperp_dla_color2,linestyle='-')

    plt.hlines(B1y_20_30,xmin=20,xmax=30,color=rperp_dla_color2,linestyle='-')
    plt.vlines(20,ymin=B1y_20_30-0.1,ymax=B1y_20_30+0.1,color=rperp_dla_color2,linestyle='-')
    plt.vlines(30,ymin=B1y_20_30-0.1,ymax=B1y_20_30+0.1,color=rperp_dla_color2,linestyle='-')

    plt.hlines(B1y_30_40,xmin=30,xmax=40,color=rperp_dla_color2,linestyle='-')
    plt.vlines(30,ymin=B1y_30_40-0.1,ymax=B1y_30_40+0.1,color=rperp_dla_color2,linestyle='-')
    plt.vlines(40,ymin=B1y_30_40-0.1,ymax=B1y_30_40+0.1,color=rperp_dla_color2,linestyle='-')

    plt.hlines(B1y_40_50,xmin=40,xmax=50,color=rperp_dla_color2,linestyle='-')
    plt.vlines(40,ymin=B1y_40_50-0.1,ymax=B1y_40_50+0.1,color=rperp_dla_color2,linestyle='-')
    plt.vlines(50,ymin=B1y_40_50-0.1,ymax=B1y_40_50+0.1,color=rperp_dla_color2,linestyle='-')

    plt.hlines(B1y_50_60,xmin=50,xmax=60,color=rperp_dla_color2,linestyle='-')
    plt.vlines(50,ymin=B1y_50_60-0.1,ymax=B1y_50_60+0.1,color=rperp_dla_color2,linestyle='-')
    plt.vlines(60,ymin=B1y_50_60-0.1,ymax=B1y_50_60+0.1,color=rperp_dla_color2,linestyle='-')

    plt.hlines(B1y_60_70,xmin=60,xmax=70,color=rperp_dla_color2,linestyle='-')
    plt.vlines(60,ymin=B1y_60_70-0.1,ymax=B1y_60_70+0.1,color=rperp_dla_color2,linestyle='-')
    plt.vlines(70,ymin=B1y_60_70-0.1,ymax=B1y_60_70+0.1,color=rperp_dla_color2,linestyle='-')

    plt.hlines(B1y_70_80,xmin=70,xmax=80,color=rperp_dla_color2,linestyle='-')
    plt.vlines(70,ymin=B1y_70_80-0.1,ymax=B1y_70_80+0.1,color=rperp_dla_color2,linestyle='-')
    plt.vlines(80,ymin=B1y_70_80-0.1,ymax=B1y_70_80+0.1,color=rperp_dla_color2,linestyle='-')

    plt.hlines(B1y_80_90,xmin=80,xmax=90,color=rperp_dla_color2,linestyle='-')
    plt.vlines(80,ymin=B1y_80_90-0.1,ymax=B1y_80_90+0.1,color=rperp_dla_color2,linestyle='-')
    plt.vlines(90,ymin=B1y_80_90-0.1,ymax=B1y_80_90+0.1,color=rperp_dla_color2,linestyle='-')

    plt.hlines(B1y_90_100,xmin=90,xmax=100,color=rperp_dla_color2,linestyle='-')
    plt.vlines(90,ymin=B1y_90_100-0.1,ymax=B1y_90_100+0.1,color=rperp_dla_color2,linestyle='-')
    plt.vlines(100,ymin=B1y_90_100-0.1,ymax=B1y_90_100+0.1,color=rperp_dla_color2,linestyle='-')

    plt.hlines(B1y_100_120,xmin=100,xmax=120,color=rperp_dla_color2,linestyle='-')
    plt.vlines(100,ymin=B1y_100_120-0.1,ymax=B1y_100_120+0.1,color=rperp_dla_color2,linestyle='-')
    plt.vlines(120,ymin=B1y_100_120-0.1,ymax=B1y_100_120+0.1,color=rperp_dla_color2,linestyle='-')

    plt.hlines(B1y_120_140,xmin=120,xmax=140,color=rperp_dla_color2,linestyle='-')
    plt.vlines(120,ymin=B1y_120_140-0.1,ymax=B1y_120_140+0.1,color=rperp_dla_color2,linestyle='-')
    plt.vlines(140,ymin=B1y_120_140-0.1,ymax=B1y_120_140+0.1,color=rperp_dla_color2,linestyle='-')

    plt.hlines(B1y_140_160,xmin=140,xmax=160,color=rperp_dla_color2,linestyle='-')
    plt.vlines(140,ymin=B1y_140_160-0.1,ymax=B1y_140_160+0.1,color=rperp_dla_color2,linestyle='-')
    plt.vlines(160,ymin=B1y_140_160-0.1,ymax=B1y_140_160+0.1,color=rperp_dla_color2,linestyle='-')

    plt.hlines(B1y_160_180,xmin=160,xmax=180,color=rperp_dla_color2,linestyle='-')
    plt.vlines(160,ymin=B1y_160_180-0.1,ymax=B1y_160_180+0.1,color=rperp_dla_color2,linestyle='-')
    plt.vlines(180,ymin=B1y_160_180-0.1,ymax=B1y_160_180+0.1,color=rperp_dla_color2,linestyle='-')

    plt.hlines(B1y_180_200,xmin=180,xmax=200,color=rperp_dla_color2,linestyle='-')
    plt.vlines(180,ymin=B1y_180_200-0.1,ymax=B1y_180_200+0.1,color=rperp_dla_color2,linestyle='-')
    plt.vlines(200,ymin=B1y_180_200-0.1,ymax=B1y_180_200+0.1,color=rperp_dla_color2,linestyle='-')
    ########################################################################################################################################
    # C_f (of LLS) Graph for both R_perp
    ax1=fig.add_subplot(ax[1,0:3]) # Grid for the covering factor plot
    plt.xlim([0,200])
    plt.ylim([0.0,1.0])
    plt.ylabel(r"$C_f$"+" (LLS)",fontsize=12,fontname="serif")
    plt.xticks([25,50,75,100,125,150,175,200], [" "," "," "," "," "," "," "," "])

    # For A2- R_perp from DLA
    plt.scatter(A2xList,A2yList,color=rperp_dla_color1) # Plot the points in the covering factor plot       
    # Next part will plot the horizontal and vertical lines in the covering factor graph for the x errors
    plt.hlines(A2y_0_10,xmin=0,xmax=10,color=rperp_dla_color1,linestyle='-')
    plt.vlines(10,ymin=A2y_0_10-0.1,ymax=A2y_0_10+0.1,color=rperp_dla_color1,linestyle='-')

    plt.hlines(A2y_10_20,xmin=10,xmax=20,color=rperp_dla_color1,linestyle='-')
    plt.vlines(10,ymin=A2y_10_20-0.1,ymax=A2y_10_20+0.1,color=rperp_dla_color1,linestyle='-')
    plt.vlines(20,ymin=A2y_10_20-0.1,ymax=A2y_10_20+0.1,color=rperp_dla_color1,linestyle='-')

    plt.hlines(A2y_20_30,xmin=20,xmax=30,color=rperp_dla_color1,linestyle='-')
    plt.vlines(20,ymin=A2y_20_30-0.1,ymax=A2y_20_30+0.1,color=rperp_dla_color1,linestyle='-')
    plt.vlines(30,ymin=A2y_20_30-0.1,ymax=A2y_20_30+0.1,color=rperp_dla_color1,linestyle='-')

    plt.hlines(A2y_30_40,xmin=30,xmax=40,color=rperp_dla_color1,linestyle='-')
    plt.vlines(30,ymin=A2y_30_40-0.1,ymax=A2y_30_40+0.1,color=rperp_dla_color1,linestyle='-')
    plt.vlines(40,ymin=A2y_30_40-0.1,ymax=A2y_30_40+0.1,color=rperp_dla_color1,linestyle='-')

    plt.hlines(A2y_40_50,xmin=40,xmax=50,color=rperp_dla_color1,linestyle='-')
    plt.vlines(40,ymin=A2y_40_50-0.1,ymax=A2y_40_50+0.1,color=rperp_dla_color1,linestyle='-')
    plt.vlines(50,ymin=A2y_40_50-0.1,ymax=A2y_40_50+0.1,color=rperp_dla_color1,linestyle='-')

    plt.hlines(A2y_50_60,xmin=50,xmax=60,color=rperp_dla_color1,linestyle='-')
    plt.vlines(50,ymin=A2y_50_60-0.1,ymax=A2y_50_60+0.1,color=rperp_dla_color1,linestyle='-')
    plt.vlines(60,ymin=A2y_50_60-0.1,ymax=A2y_50_60+0.1,color=rperp_dla_color1,linestyle='-')

    plt.hlines(A2y_60_70,xmin=60,xmax=70,color=rperp_dla_color1,linestyle='-')
    plt.vlines(60,ymin=A2y_60_70-0.1,ymax=A2y_60_70+0.1,color=rperp_dla_color1,linestyle='-')
    plt.vlines(70,ymin=A2y_60_70-0.1,ymax=A2y_60_70+0.1,color=rperp_dla_color1,linestyle='-')

    plt.hlines(A2y_70_80,xmin=70,xmax=80,color=rperp_dla_color1,linestyle='-')
    plt.vlines(70,ymin=A2y_70_80-0.1,ymax=A2y_70_80+0.1,color=rperp_dla_color1,linestyle='-')
    plt.vlines(80,ymin=A2y_70_80-0.1,ymax=A2y_70_80+0.1,color=rperp_dla_color1,linestyle='-')

    plt.hlines(A2y_80_90,xmin=80,xmax=90,color=rperp_dla_color1,linestyle='-')
    plt.vlines(80,ymin=A2y_80_90-0.1,ymax=A2y_80_90+0.1,color=rperp_dla_color1,linestyle='-')
    plt.vlines(90,ymin=A2y_80_90-0.1,ymax=A2y_80_90+0.1,color=rperp_dla_color1,linestyle='-')

    plt.hlines(A2y_90_100,xmin=90,xmax=100,color=rperp_dla_color1,linestyle='-')
    plt.vlines(90,ymin=A2y_90_100-0.1,ymax=A2y_90_100+0.1,color=rperp_dla_color1,linestyle='-')
    plt.vlines(100,ymin=A2y_90_100-0.1,ymax=A2y_90_100+0.1,color=rperp_dla_color1,linestyle='-')

    plt.hlines(A2y_100_120,xmin=100,xmax=120,color=rperp_dla_color1,linestyle='-')
    plt.vlines(100,ymin=A2y_100_120-0.1,ymax=A2y_100_120+0.1,color=rperp_dla_color1,linestyle='-')
    plt.vlines(120,ymin=A2y_100_120-0.1,ymax=A2y_100_120+0.1,color=rperp_dla_color1,linestyle='-')

    plt.hlines(A2y_120_140,xmin=120,xmax=140,color=rperp_dla_color1,linestyle='-')
    plt.vlines(120,ymin=A2y_120_140-0.1,ymax=A2y_120_140+0.1,color=rperp_dla_color1,linestyle='-')
    plt.vlines(140,ymin=A2y_120_140-0.1,ymax=A2y_120_140+0.1,color=rperp_dla_color1,linestyle='-')

    plt.hlines(A2y_140_160,xmin=140,xmax=160,color=rperp_dla_color1,linestyle='-')
    plt.vlines(140,ymin=A2y_140_160-0.1,ymax=A2y_140_160+0.1,color=rperp_dla_color1,linestyle='-')
    plt.vlines(160,ymin=A2y_140_160-0.1,ymax=A2y_140_160+0.1,color=rperp_dla_color1,linestyle='-')

    plt.hlines(A2y_160_180,xmin=160,xmax=180,color=rperp_dla_color1,linestyle='-')
    plt.vlines(160,ymin=A2y_160_180-0.1,ymax=A2y_160_180+0.1,color=rperp_dla_color1,linestyle='-')
    plt.vlines(180,ymin=A2y_160_180-0.1,ymax=A2y_160_180+0.1,color=rperp_dla_color1,linestyle='-')

    # For B2- R_perp from center
    plt.scatter(B2xList,B2yList,color=rperp_dla_color2) # Plot the points in the covering factor plot       
    # Next part will plot the horizontal and vertical lines in the covering factor graph for the x errors
    plt.hlines(B2y_0_10,xmin=0,xmax=10,color=rperp_dla_color2,linestyle='-')
    plt.vlines(10,ymin=B2y_0_10-0.1,ymax=B2y_0_10+0.1,color=rperp_dla_color2,linestyle='-')

    plt.hlines(B2y_10_20,xmin=10,xmax=20,color=rperp_dla_color2,linestyle='-')
    plt.vlines(10,ymin=B2y_10_20-0.1,ymax=B2y_10_20+0.1,color=rperp_dla_color2,linestyle='-')
    plt.vlines(20,ymin=B2y_10_20-0.1,ymax=B2y_10_20+0.1,color=rperp_dla_color2,linestyle='-')

    plt.hlines(B2y_20_30,xmin=20,xmax=30,color=rperp_dla_color2,linestyle='-')
    plt.vlines(20,ymin=B2y_20_30-0.1,ymax=B2y_20_30+0.1,color=rperp_dla_color2,linestyle='-')
    plt.vlines(30,ymin=B2y_20_30-0.1,ymax=B2y_20_30+0.1,color=rperp_dla_color2,linestyle='-')

    plt.hlines(B2y_30_40,xmin=30,xmax=40,color=rperp_dla_color2,linestyle='-')
    plt.vlines(30,ymin=B2y_30_40-0.1,ymax=B2y_30_40+0.1,color=rperp_dla_color2,linestyle='-')
    plt.vlines(40,ymin=B2y_30_40-0.1,ymax=B2y_30_40+0.1,color=rperp_dla_color2,linestyle='-')

    plt.hlines(B2y_40_50,xmin=40,xmax=50,color=rperp_dla_color2,linestyle='-')
    plt.vlines(40,ymin=B2y_40_50-0.1,ymax=B2y_40_50+0.1,color=rperp_dla_color2,linestyle='-')
    plt.vlines(50,ymin=B2y_40_50-0.1,ymax=B2y_40_50+0.1,color=rperp_dla_color2,linestyle='-')

    plt.hlines(B2y_50_60,xmin=50,xmax=60,color=rperp_dla_color2,linestyle='-')
    plt.vlines(50,ymin=B2y_50_60-0.1,ymax=B2y_50_60+0.1,color=rperp_dla_color2,linestyle='-')
    plt.vlines(60,ymin=B2y_50_60-0.1,ymax=B2y_50_60+0.1,color=rperp_dla_color2,linestyle='-')

    plt.hlines(B2y_60_70,xmin=60,xmax=70,color=rperp_dla_color2,linestyle='-')
    plt.vlines(60,ymin=B2y_60_70-0.1,ymax=B2y_60_70+0.1,color=rperp_dla_color2,linestyle='-')
    plt.vlines(70,ymin=B2y_60_70-0.1,ymax=B2y_60_70+0.1,color=rperp_dla_color2,linestyle='-')

    plt.hlines(B2y_70_80,xmin=70,xmax=80,color=rperp_dla_color2,linestyle='-')
    plt.vlines(70,ymin=B2y_70_80-0.1,ymax=B2y_70_80+0.1,color=rperp_dla_color2,linestyle='-')
    plt.vlines(80,ymin=B2y_70_80-0.1,ymax=B2y_70_80+0.1,color=rperp_dla_color2,linestyle='-')

    plt.hlines(B2y_80_90,xmin=80,xmax=90,color=rperp_dla_color2,linestyle='-')
    plt.vlines(80,ymin=B2y_80_90-0.1,ymax=B2y_80_90+0.1,color=rperp_dla_color2,linestyle='-')
    plt.vlines(90,ymin=B2y_80_90-0.1,ymax=B2y_80_90+0.1,color=rperp_dla_color2,linestyle='-')

    plt.hlines(B2y_90_100,xmin=90,xmax=100,color=rperp_dla_color2,linestyle='-')
    plt.vlines(90,ymin=B2y_90_100-0.1,ymax=B2y_90_100+0.1,color=rperp_dla_color2,linestyle='-')
    plt.vlines(100,ymin=B2y_90_100-0.1,ymax=B2y_90_100+0.1,color=rperp_dla_color2,linestyle='-')

    plt.hlines(B2y_100_120,xmin=100,xmax=120,color=rperp_dla_color2,linestyle='-')
    plt.vlines(100,ymin=B2y_100_120-0.1,ymax=B2y_100_120+0.1,color=rperp_dla_color2,linestyle='-')
    plt.vlines(120,ymin=B2y_100_120-0.1,ymax=B2y_100_120+0.1,color=rperp_dla_color2,linestyle='-')

    plt.hlines(B2y_120_140,xmin=120,xmax=140,color=rperp_dla_color2,linestyle='-')
    plt.vlines(120,ymin=B2y_120_140-0.1,ymax=B2y_120_140+0.1,color=rperp_dla_color2,linestyle='-')
    plt.vlines(140,ymin=B2y_120_140-0.1,ymax=B2y_120_140+0.1,color=rperp_dla_color2,linestyle='-')

    plt.hlines(B2y_140_160,xmin=140,xmax=160,color=rperp_dla_color2,linestyle='-')
    plt.vlines(140,ymin=B2y_140_160-0.1,ymax=B2y_140_160+0.1,color=rperp_dla_color2,linestyle='-')
    plt.vlines(160,ymin=B2y_140_160-0.1,ymax=B2y_140_160+0.1,color=rperp_dla_color2,linestyle='-')

    plt.hlines(B2y_160_180,xmin=160,xmax=180,color=rperp_dla_color2,linestyle='-')
    plt.vlines(160,ymin=B2y_160_180-0.1,ymax=B2y_160_180+0.1,color=rperp_dla_color2,linestyle='-')
    plt.vlines(180,ymin=B2y_160_180-0.1,ymax=B2y_160_180+0.1,color=rperp_dla_color2,linestyle='-')

    plt.hlines(B2y_180_200,xmin=180,xmax=200,color=rperp_dla_color2,linestyle='-')
    plt.vlines(180,ymin=B2y_180_200-0.1,ymax=B2y_180_200+0.1,color=rperp_dla_color2,linestyle='-')
    plt.vlines(200,ymin=B2y_180_200-0.1,ymax=B2y_180_200+0.1,color=rperp_dla_color2,linestyle='-')

    # Next part will handle the main graph of Median, 16th, 18th
    ax1 = fig.add_subplot(ax[2:,0:3]) # Plot for the main graph
    low_interval=lambda x: np.nanpercentile(x,16,axis=0)
    high_interval=lambda x: np.nanpercentile(x,84,axis=0)

    Abinsmin=np.min(ListOfRBetweenTwoSightLines)
    Abinsmax=np.max(ListOfRBetweenTwoSightLines)
    Bbinsmin=np.min(ListOfRBetweenSightlineAndTheCenter)
    Bbinsmax=np.max(ListOfRBetweenSightlineAndTheCenter)

    binsmin=np.min([Abinsmin,Bbinsmin])
    binsmax=np.max([Abinsmax,Bbinsmax])
    #bins=np.linspace(binsmin,binsmax,12)
    bins=np.array([5,15,25,35,45,55,65,75,85,95,110,130,150,170,190,210])

    #Abins=np.linspace(np.min(ListOfRBetweenTwoSightLines),np.max(ListOfRBetweenTwoSightLines),12)
    Abin_means,Abin_edges,Abinnumber=scipy.stats.binned_statistic(ListOfRBetweenTwoSightLines,ListOfN_HIOfTheAnotherLine,statistic='mean',bins=bins)
    Abin_means2,Abin_edges2,Abinnumber2=scipy.stats.binned_statistic(ListOfRBetweenTwoSightLines,ListOfN_HIOfTheAnotherLine,statistic=low_interval,bins=bins)
    Abin_means3,Abin_edges3,Abinnumber3=scipy.stats.binned_statistic(ListOfRBetweenTwoSightLines,ListOfN_HIOfTheAnotherLine,statistic=high_interval,bins=bins)
    print("Abin_means shape is: ",Abin_means.shape)
    print("Abin_means list is: ",Abin_means)
    print("Abin_edges shape is: ",Abin_edges.shape)
    print("Abin_edges list is: ",Abin_edges)
    plt.plot(Abin_edges[0:15],Abin_means,color=rperp_dla_color1,label=r"$R_\perp$"+" from DLA",linewidth=3)
    plt.fill_between(Abin_edges[0:15],Abin_means2,Abin_means3,color=rperp_dla_color1,alpha=0.5)
    Amax=np.max(Abin_edges[0:15])

    #Bbins=np.linspace(np.min(ListOfRBetweenSightlineAndTheCenter),np.max(ListOfRBetweenSightlineAndTheCenter),12)
    Bbin_means,Bbin_edges,Bbinnumber=scipy.stats.binned_statistic(ListOfRBetweenSightlineAndTheCenter,ListOfN_HIOfTheSightline,statistic='mean',bins=bins)
    Bbin_means2,Bbin_edges2,Bbinnumber2=scipy.stats.binned_statistic(ListOfRBetweenSightlineAndTheCenter,ListOfN_HIOfTheSightline,statistic=low_interval,bins=bins)
    Bbin_means3,Bbin_edges3,Bbinnumber3=scipy.stats.binned_statistic(ListOfRBetweenSightlineAndTheCenter,ListOfN_HIOfTheSightline,statistic=high_interval,bins=bins)
    print("Bbin_means shape is: ",Bbin_means.shape)
    print("Bbin_means list is: ",Bbin_means)
    print("Bbin_edges shape is: ",Bbin_edges.shape)
    print("Bbin_edges list is: ",Bbin_edges)
    plt.plot(Bbin_edges[0:15],Bbin_means,color=rperp_dla_color2,label=r"$R_\perp$"+" from center",linewidth=3)
    plt.fill_between(Bbin_edges[0:15],Bbin_means2,Bbin_means3,color=rperp_dla_color2,alpha=0.25)
    Bmax=np.max(Bbin_edges[0:15])

    xmax=np.max([Amax,Bmax])
    plt.xlim([0,200])
    plt.xlabel(r"$R_⊥$"+" [kpc]",fontsize=12,fontname="serif")
    plt.ylabel(r"$\log N_{\rm HI}$"+" "+"[cm"+r"$^{-2}$"+"]",fontsize=12,fontname="serif")
    plt.hlines(20.3,xmin=0,xmax=200,color='red',linestyle='--',label="DLA")
    plt.hlines(17.2,xmin=0,xmax=200,color='black',linestyle='--',label="LLS")
    plt.legend(loc='upper right')
    plt.ylim([13.5,21.5])
    plt.savefig('/home1/08289/tg875885/radial_to_rotating_flows/Aharon/Outputs2/'+SnapDirect+"~log(N_HI)[R⊥]~MedianOfBothRperp")
    print("DONE 4")
    break

In [ ]:
# This block will create HI projection plot for each of the simulations and snapshot numbers of Stampede !!! 
# Output: will save 2 files: .png image of the plot and .hdf5 file of the projection
plt.clf()
ListOfSnapshotsParts=[]
forloopcount=0
for SnapDirect in ListOfFullPathsForStampede:
    ListOfSnapshotsParts=[]
    if(SnapDirect.endswith('.hdf5')):
        ListOfSnapshotsParts.append(SnapDirect)
    else:
        ListOfFiles=os.listdir(SnapDirect)
        for i in ListOfFiles:
            if (i.endswith('.hdf5')):
                ListOfSnapshotsParts.append(SnapDirect+'/'+i)# Append Only hdf5 Files To The Snapshots Parts List
    print("The list of the snapshot parts is:",ListOfSnapshotsParts)
    print("")
    SimulationKey=c.ListOfStampedekeysOfTheSimulationsNames[forloopcount]
    Snap1=l.Snapshot(ListOfSnapshotsParts[0],[0,0,0],[0,0,0])# SnapName, Not Real Center Coordinates, Not Real Center Velocity
    z=Snap1.Redshift()
    print("The redshift of the given halo is:",z)
    Mvir=HD.HaloMvir(SimulationKey,z)
    Rvir=HD.HaloRvir(SimulationKey,z)
    CenterCoordinates=HD.HaloCenterCoordinates(SimulationKey,z)
    CenterVelocity=HD.HaloCenterVelocity(SimulationKey,z)
    afactor=HD.HaloaFactor(SimulationKey,z)

    # This part intended to merge all the data from the different snapshot parts (if there are more than 1 part)
    TotalNumberOfParticles=0
    Masses=[]
    Coordinates=[]
    Temperature=[]
    r=[]
    rs=[]
    Density=[]
    NeutralH=[]
    GasMasses=[]
    BoxesSizes=[]
    SmoothingLengths=[]
    
    for snapshotname in ListOfSnapshotsParts:
        Snap1=l.Snapshot(snapshotname,CenterCoordinates,CenterVelocity,a=afactor)# SnapName, Real Center Coordinates, Real Center Velocity

        NumberOfParticlesOfOneSnapshot=Snap1.number_of_particles()# 0-Gas,1-High Resolution Dark Matter,2-Dummy Particles,3-Dummy Particles,4-Stars,5-Black Holes
        TotalNumberOfParticles=TotalNumberOfParticles+NumberOfParticlesOfOneSnapshot[0]

        MassesOfOneSnapshotGasParticles=Snap1.masses(0)# The Masses Of The Gas Particles
        GasMasses.extend(MassesOfOneSnapshotGasParticles)

        CoordinatesOfOneSnapShot=Snap1.coords(0)# The Coordinates Of The Gas Particles
        Coordinates.extend(CoordinatesOfOneSnapShot)

        TemperatureOfOneSnapShot=Snap1.Ts()# The Temperature Of The Gas Particles
        Temperature.extend(TemperatureOfOneSnapShot)

        rOfOneSnapShot=Snap1.rs()# The r Of The Gas Particles
        r.extend(rOfOneSnapShot)

        DensityOfOneSnapShot=Snap1.rhos()# The Density Of The Gas Particles
        Density.extend(DensityOfOneSnapShot)

        NeutralHOfOneSnapshot=Snap1.neutralHydrogenAbundance(0)# Returns The Neutral Hydrogen Fraction
        NeutralH.extend(NeutralHOfOneSnapshot)
        
        SmoothingLengthOfOneSnapshot=Snap1.SmoothingLength()
        SmoothingLengths.extend(SmoothingLengthOfOneSnapshot)
        
    Coordinates=np.array(Coordinates)
    Temperature=np.array(Temperature)
    r=np.array(r)
    Density=np.array(Density)
    NeutralH=np.array(NeutralH)
    GasMasses=np.array(GasMasses)
    SmoothingLengths=np.array(SmoothingLengths)
    
    print("The total number of gas particles is:",TotalNumberOfParticles)
    print("r min is:",r.min())
    print("r max is:",r.max())
          
    snapdir=ListOfOutputFolderPathsForStampede[forloopcount]
    iSnapshot=ListOfSnapshotsNumbersForStampede[forloopcount]
    r_max=200
    z_width=200
    filedir='/home1/08289/tg875885/radial_to_rotating_flows/Aharon/Outputs2'
    SnapDirect=SnapDirect.replace("/","-")
    SnapDirect=SnapDirect.replace(".hdf5","")
    projection_output_filename=SnapDirect
    
    # Create input dictionary for FIRE studio from snapshot and snapdict
    studiodic = {}
    studiodic['Coordinates']    = Coordinates
    studiodic['Masses']         = GasMasses*NeutralH
    studiodic['BoxSize']        = Snap1.BoxSize()
    studiodic['SmoothingLength']= SmoothingLengths 
    
    # Create N_HI projection 
    mystudio=GasStudio(
             snapdir, 
             snapnum=int(iSnapshot),
             snapdict=studiodic,
             datadir=filedir,
             frame_half_width=r_max,
             frame_depth=z_width,
             quantity_name='Masses',
             take_log_of_quantity=False, 
             galaxy_extractor=False,
             pixels=1200,
             single_image='Density',
             overwrite=True,
             savefig=False,      
             use_hsml=True,
             intermediate_file_name=projection_output_filename,
             )
    
    # This part intended to plot the HI projection with Firestudio and save .png and .hdf5 files
    # .hdf5 file will contain the following things: 
    # A grid of column densities, The side length/the positions the column densities are at, The redshift, The simulation halo mass (its viral mass),
    # The simulation name, The path to the file name should contain reference to the simulation itself and to the snapshot number. 
    fig,ax=plt.subplots()
    NHImap,_=mystudio.projectImage([])
    NHImap+=log((X*un.Msun*un.pc**-2/cons.m_p/1e10*h).to('cm**-2').value) # Units fix
    #NHImap=np.where(NHImap<-1000,0,NHImap)
    NHImap=NHImap.T
    Xs=np.linspace(-r_max,r_max,NHImap.shape[0])
    Ys=np.linspace(-r_max,r_max,NHImap.shape[1])
    # This part calculates the S of DLA in Rvir/ the total S of Rvir (it will be useful in the calculations)
    def CalculateTheDistanceBetweenTwoPoints(x1,y1,x2,y2): # Function that calculates the distance (R⊥) between 2 points (x1,y1,x2,y2)
        r1=np.sqrt(x1**2+y1**2)
        r2=np.sqrt(x2**2+y2**2)
        Distance=np.abs(r1-r2)
        return Distance
    CountOfDLAInRvir=0
    CountOfTotalSightsInRvir=0
    ListOfIndexes=np.argwhere(NHImap)
    for i in range(0,len(ListOfIndexes)-1):
        xIndex=ListOfIndexes[i][0]
        yIndex=ListOfIndexes[i][1]
        d=CalculateTheDistanceBetweenTwoPoints(Xs[xIndex],Ys[yIndex],0,0)
        if(d<Rvir):
            CountOfTotalSightsInRvir=CountOfTotalSightsInRvir+1
            if(NHImap[xIndex][yIndex]>20.3):
                CountOfDLAInRvir=CountOfDLAInRvir+1
    print("The total sightlines in Rvir area is:",CountOfTotalSightsInRvir)
    print("The count of DLA sightlines in Rvir area is:",CountOfDLAInRvir)
    SDLADivideSRvir=CountOfDLAInRvir/CountOfTotalSightsInRvir
    print("S of DLA in Rvir / S Rvir=",SDLADivideSRvir)
    # This part prepares the graph
    plt.rcParams['mathtext.fontset']='dejavuserif'
    plt.pcolormesh(Xs,Ys,NHImap,cmap='viridis',norm=pl.Normalize(13.5,21.5))
    cbar=pl.colorbar()
    cbar.set_label("log "+r"$N_{\rm HI}$"+" "+"[cm"+r"$^{-2}$"+"]",fontsize=14,fontname="serif")
    plt.xlabel("X [kpc]",fontsize=12,fontname="serif")
    plt.ylabel("Y [kpc]",fontsize=12,fontname="serif")
    title="z= "+str(round(z,2))+"   ;   M= "+str(Mvir)+r" $M_{☉}$"+"\n"+r"$R_{vir}$"+" Area: "+r"$\frac{S_{DLA}}{S_{Total}}=$"+str(round(SDLADivideSRvir,2))
    plt.title(title)
    a_circle=plt.Circle((0,0),Rvir,fill=False,color='white') # Draw white circle in Rvir of the halo
    ax.add_patch(a_circle) # Adds the circle to the plot
    plt.savefig('/home1/08289/tg875885/radial_to_rotating_flows/Aharon/Outputs2/'+SnapDirect+'.png')
    plt.show()
    plt.clf()
    plt.cla()
    plt.close()
    xlist=Coordinates[:,0]
    ylist=Coordinates[:,1]
    data=Verdict.Dict({'hist': NHImap,'x_bins': Xs,'y_bins': Ys})
    #data.to_hdf5('/home1/08289/tg875885/radial_to_rotating_flows/Aharon/Outputs2/'+SnapDirect+'.hdf5')
    print("DONE 1")
    ######################################################################################################################################
    ######################################################################################################################################
    # log(N_HI)[R⊥] - R⊥ is the distance between 2 sightlines (1 is DLA sighline and 1 is random sightline, N_HI of the random sightline)
    plt.rcParams['mathtext.fontset']='dejavuserif'
    plt.clf()
    hist=NHImap
    x_bins=Xs
    y_bins=Ys
    def CalculateTheDistanceBetweenTwoSightLines(x1,y1,x2,y2): # Function that calculates the distance (R⊥) between 2 points (x1,y1,x2,y2)
            r1=np.sqrt(x1**2+y1**2)
            r2=np.sqrt(x2**2+y2**2)
            Distance=np.abs(r1-r2)
            return Distance
    ListOfRBetweenTwoSightLines=[] # Will be the x values for the N_HI against R⊥ plot
    ListOfN_HIOfTheAnotherLine=[] # Will be the y values for the N_HI against R⊥ plot
    ListOfDLAIndexes=np.argwhere(hist>20.3) # Each element in this list has 2 indexs, first one for the x indexes and second one for the y indexes

    # This part will random 100 pairs in each one of the bins: [0,10], [10,20], [20,30] ...
    def RandomSightinesForSpecificBin(R_perpLower,R_perpUpper): # Function that generate 100 pairs of sightlines for a specific given bin 
        CountOfRandom100=0
        while(CountOfRandom100<100):
            DLARandomIndex=random.randint(0,len(ListOfDLAIndexes)-1) # Choose random index from the ListOfDLAIndexes list
            xDLAIndex=ListOfDLAIndexes[DLARandomIndex][0] # Find the relevant x index 
            yDLAIndex=ListOfDLAIndexes[DLARandomIndex][1] # Find the relevant y index
            # Next part will be for the Another random sighline
            xAnotherIndex=random.randint(0,hist.shape[0]-1) # Find the relevant x index for the Another sighline
            yAnotherIndex=random.randint(0,hist.shape[1]-1) # Find the relevant y index for the Another sighline
            while(hist[xAnotherIndex][yAnotherIndex]==0): # Make sure that the random sighline has no 0 value for the colmun density]
                xAnotherIndex=random.randint(0,hist.shape[0]-1) # Find the relevant x index for the Another sighline
                yAnotherIndex=random.randint(0,hist.shape[1]-1) # Find the relevant y index for the Another sighline
            R_perp=CalculateTheDistanceBetweenTwoSightLines(x_bins[xDLAIndex],y_bins[yDLAIndex],x_bins[xAnotherIndex],y_bins[yAnotherIndex])
            if(R_perp<R_perpUpper and R_perp>R_perpLower):
                if(hist[xAnotherIndex][yAnotherIndex]>20.3):
                    ListOfN_HIOfTheAnotherLine.append(max(hist[xAnotherIndex][yAnotherIndex],hist[xDLAIndex][yDLAIndex])) # Insert the N_HI of the Another sightline to the final y list
                                                                                                                          # If both of them are DLAs it will take the highst N_HI
                else:
                    ListOfN_HIOfTheAnotherLine.append(hist[xAnotherIndex][yAnotherIndex])
                ListOfRBetweenTwoSightLines.append(R_perp) # Insert the R⊥ between the two sightlines to the final x list
                CountOfRandom100=CountOfRandom100+1
    ProjectionLimit=200            
    R_perpLower=0
    R_perpUpper=10
    count=0
    while(R_perpUpper<ProjectionLimit+10): # Generate all of the sightlines in the projection limit (100 pairs for each bin, each bin=10kpc)
        RandomSightinesForSpecificBin(R_perpLower,R_perpUpper)
        print("done",count)
        count=count+1
        R_perpLower=R_perpLower+10
        R_perpUpper=R_perpUpper+10

    fig=plt.figure(constrained_layout=True) # Creates the figure
    ax=fig.add_gridspec(5,3) # Creates the figure structure
    #########################################################################################################################################
    # Covering Factor for DLA
    ax2=fig.add_subplot(ax[0,0:3])
    # This part calculates the covering factor in each range
    Count_0_10=0 # For the total points in the R_perpendicular area
    RSum_0_10=0 # To calculate the mean of the R in this area
    Count2_0_10=0 # Only for the points with HI>10**17.2

    Count_10_20=0 # For the total points in the R_perpendicular area
    RSum_10_20=0 # To calculate the mean of the R in this area
    Count2_10_20=0 # Only for the points with HI>10**17.2

    Count_20_30=0 # For the total points in the R_perpendicular area
    RSum_20_30=0 # To calculate the mean of the R in this area
    Count2_20_30=0 # Only for the points with HI>10**17.2

    Count_30_40=0 # For the total points in the R_perpendicular area
    RSum_30_40=0 # To calculate the mean of the R in this area
    Count2_30_40=0 # Only for the points with HI>10**17.2

    Count_40_50=0 # For the total points in the R_perpendicular area
    RSum_40_50=0 # To calculate the mean of the R in this area
    Count2_40_50=0 # Only for the points with HI>10**17.2

    Count_50_60=0 # For the total points in the R_perpendicular area
    RSum_50_60=0 # To calculate the mean of the R in this area
    Count2_50_60=0 # Only for the points with HI>10**17.2

    Count_60_70=0 # For the total points in the R_perpendicular area
    RSum_60_70=0 # To calculate the mean of the R in this area
    Count2_60_70=0 # Only for the points with HI>10**17.2

    Count_70_80=0 # For the total points in the R_perpendicular area
    RSum_70_80=0 # To calculate the mean of the R in this area
    Count2_70_80=0 # Only for the points with HI>10**17.2

    Count_80_90=0 # For the total points in the R_perpendicular area
    RSum_80_90=0 # To calculate the mean of the R in this area
    Count2_80_90=0 # Only for the points with HI>10**17.2

    Count_90_100=0 # For the total points in the R_perpendicular area
    RSum_90_100=0 # To calculate the mean of the R in this area
    Count2_90_100=0 # Only for the points with HI>10**17.2

    Count_100_120=0 # For the total points in the R_perpendicular area
    RSum_100_120=0 # To calculate the mean of the R in this area
    Count2_100_120=0 # Only for the points with HI>10**17.2

    Count_120_140=0 # For the total points in the R_perpendicular area
    RSum_120_140=0 # To calculate the mean of the R in this area
    Count2_120_140=0 # Only for the points with HI>10**17.2

    Count_140_160=0 # For the total points in the R_perpendicular area
    RSum_140_160=0 # To calculate the mean of the R in this area
    Count2_140_160=0 # Only for the points with HI>10**17.2

    Count_160_180=0 # For the total points in the R_perpendicular area
    RSum_160_180=0 # To calculate the mean of the R in this area
    Count2_160_180=0 # Only for the points with HI>10**17.2

    Count_180_200=0 # For the total points in the R_perpendicular area
    RSum_180_200=0 # To calculate the mean of the R in this area
    Count2_180_200=0 # Only for the points with HI>10**17.2

    # This for loop calculates for each R_perp area how many total points there are, how many points are at least DLAs and what is the sum of R in each area (later to calculate the mean)
    for i in range(0,len(ListOfRBetweenTwoSightLines)):
        if(ListOfRBetweenTwoSightLines[i]<10):
            Count_0_10=Count_0_10+1
            RSum_0_10=RSum_0_10+ListOfRBetweenTwoSightLines[i]
            if(ListOfN_HIOfTheAnotherLine[i]>20.3):
                Count2_0_10=Count2_0_10+1

        if(ListOfRBetweenTwoSightLines[i]>10 and ListOfRBetweenTwoSightLines[i]<20):
            Count_10_20=Count_10_20+1
            RSum_10_20=RSum_10_20+ListOfRBetweenTwoSightLines[i]
            if(ListOfN_HIOfTheAnotherLine[i]>20.3):
                Count2_10_20=Count2_10_20+1

        if(ListOfRBetweenTwoSightLines[i]>20 and ListOfRBetweenTwoSightLines[i]<30):
            Count_20_30=Count_20_30+1
            RSum_20_30=RSum_20_30+ListOfRBetweenTwoSightLines[i]
            if(ListOfN_HIOfTheAnotherLine[i]>20.3):
                Count2_20_30=Count2_20_30+1

        if(ListOfRBetweenTwoSightLines[i]>30 and ListOfRBetweenTwoSightLines[i]<40):
            Count_30_40=Count_30_40+1
            RSum_30_40=RSum_30_40+ListOfRBetweenTwoSightLines[i]
            if(ListOfN_HIOfTheAnotherLine[i]>20.3):
                Count2_30_40=Count2_30_40+1    

        if(ListOfRBetweenTwoSightLines[i]>40 and ListOfRBetweenTwoSightLines[i]<50):
            Count_40_50=Count_40_50+1
            RSum_40_50=RSum_40_50+ListOfRBetweenTwoSightLines[i]
            if(ListOfN_HIOfTheAnotherLine[i]>20.3):
                Count2_40_50=Count2_40_50+1

        if(ListOfRBetweenTwoSightLines[i]>50 and ListOfRBetweenTwoSightLines[i]<60):
            Count_50_60=Count_50_60+1
            RSum_50_60=RSum_50_60+ListOfRBetweenTwoSightLines[i]
            if(ListOfN_HIOfTheAnotherLine[i]>20.3):
                Count2_50_60=Count2_50_60+1

        if(ListOfRBetweenTwoSightLines[i]>60 and ListOfRBetweenTwoSightLines[i]<70):
            Count_60_70=Count_60_70+1
            RSum_60_70=RSum_60_70+ListOfRBetweenTwoSightLines[i]
            if(ListOfN_HIOfTheAnotherLine[i]>20.3):
                Count2_60_70=Count2_60_70+1

        if(ListOfRBetweenTwoSightLines[i]>70 and ListOfRBetweenTwoSightLines[i]<80):
            Count_70_80=Count_70_80+1
            RSum_70_80=RSum_70_80+ListOfRBetweenTwoSightLines[i]
            if(ListOfN_HIOfTheAnotherLine[i]>20.3):
                Count2_70_80=Count2_70_80+1

        if(ListOfRBetweenTwoSightLines[i]>80 and ListOfRBetweenTwoSightLines[i]<90):
            Count_80_90=Count_80_90+1
            RSum_80_90=RSum_80_90+ListOfRBetweenTwoSightLines[i]
            if(ListOfN_HIOfTheAnotherLine[i]>20.3):
                Count2_80_90=Count2_80_90+1    

        if(ListOfRBetweenTwoSightLines[i]>90 and ListOfRBetweenTwoSightLines[i]<100):
            Count_90_100=Count_90_100+1
            RSum_90_100=RSum_90_100+ListOfRBetweenTwoSightLines[i]
            if(ListOfN_HIOfTheAnotherLine[i]>20.3):
                Count2_90_100=Count2_90_100+1

        if(ListOfRBetweenTwoSightLines[i]>100 and ListOfRBetweenTwoSightLines[i]<120):
            Count_100_120=Count_100_120+1
            RSum_100_120=RSum_100_120+ListOfRBetweenTwoSightLines[i]
            if(ListOfN_HIOfTheAnotherLine[i]>20.3):
                Count2_100_120=Count2_100_120+1

        if(ListOfRBetweenTwoSightLines[i]>120 and ListOfRBetweenTwoSightLines[i]<140):
            Count_120_140=Count_120_140+1
            RSum_120_140=RSum_120_140+ListOfRBetweenTwoSightLines[i]
            if(ListOfN_HIOfTheAnotherLine[i]>20.3):
                Count2_120_140=Count2_120_140+1

        if(ListOfRBetweenTwoSightLines[i]>140 and ListOfRBetweenTwoSightLines[i]<160):
            Count_140_160=Count_140_160+1
            RSum_140_160=RSum_140_160+ListOfRBetweenTwoSightLines[i]
            if(ListOfN_HIOfTheAnotherLine[i]>20.3):
                Count2_140_160=Count2_140_160+1

        if(ListOfRBetweenTwoSightLines[i]>160 and ListOfRBetweenTwoSightLines[i]<180):
            Count_160_180=Count_160_180+1
            RSum_160_180=RSum_160_180+ListOfRBetweenTwoSightLines[i]
            if(ListOfN_HIOfTheAnotherLine[i]>20.3):
                Count2_160_180=Count2_160_180+1    

        if(ListOfRBetweenTwoSightLines[i]>180 and ListOfRBetweenTwoSightLines[i]<200):
            Count_180_200=Count_180_200+1
            RSum_180_200=RSum_180_200+ListOfRBetweenTwoSightLines[i]
            if(ListOfN_HIOfTheAnotherLine[i]>20.3):
                Count2_180_200=Count2_180_200+1

    # This if part calculates the final x (that will be the maen of R in specific area) and y (the covering factor- how many point from all the pints are at least LLAs)
    if(Count_0_10!=0):
        A1x_0_10=RSum_0_10/Count_0_10
        A1y_0_10=Count2_0_10/Count_0_10
    else:
        A1x_0_10=5
        A1y_0_10=0

    if(Count_10_20!=0):
        A1x_10_20=RSum_10_20/Count_10_20
        A1y_10_20=Count2_10_20/Count_10_20
    else:
        A1x_10_20=10+5
        A1y_10_20=0    

    if(Count_20_30!=0):
        A1x_20_30=RSum_20_30/Count_20_30
        A1y_20_30=Count2_20_30/Count_20_30
    else:
        A1x_20_30=20+5
        A1y_20_30=0

    if(Count_30_40!=0):
        A1x_30_40=RSum_30_40/Count_30_40
        A1y_30_40=Count2_30_40/Count_30_40
    else:
        A1x_30_40=30+5
        A1y_30_40=0

    if(Count_40_50!=0):
        A1x_40_50=RSum_40_50/Count_40_50
        A1y_40_50=Count2_40_50/Count_40_50
    else:
        A1x_40_50=40+5
        A1y_40_50=0

    if(Count_50_60!=0):
        A1x_50_60=RSum_50_60/Count_50_60
        A1y_50_60=Count2_50_60/Count_50_60
    else:
        A1x_50_60=50+5
        A1y_50_60=0

    if(Count_60_70!=0):
        A1x_60_70=RSum_60_70/Count_60_70
        A1y_60_70=Count2_60_70/Count_60_70
    else:
        A1x_60_70=60+5
        A1y_60_70=0

    if(Count_70_80!=0):
        A1x_70_80=RSum_70_80/Count_70_80
        A1y_70_80=Count2_70_80/Count_70_80
    else:
        A1x_70_80=70+5
        A1y_70_80=0    

    if(Count_80_90!=0):
        A1x_80_90=RSum_80_90/Count_80_90
        A1y_80_90=Count2_80_90/Count_80_90
    else:
        A1x_80_90=80+5
        A1y_80_90=0

    if(Count_90_100!=0):
        A1x_90_100=RSum_90_100/Count_90_100
        A1y_90_100=Count2_90_100/Count_90_100
    else:
        A1x_90_100=90+5
        A1y_90_100=0

    if(Count_100_120!=0):
        A1x_100_120=RSum_100_120/Count_100_120
        A1y_100_120=Count2_100_120/Count_100_120
    else:
        A1x_100_120=90+10
        A1y_100_120=0 

    if(Count_120_140!=0):
        A1x_120_140=RSum_120_140/Count_120_140
        A1y_120_140=Count2_120_140/Count_120_140
    else:
        A1x_120_140=120+10
        A1y_120_140=0

    if(Count_140_160!=0):
        A1x_140_160=RSum_140_160/Count_140_160
        A1y_140_160=Count2_140_160/Count_140_160
    else:
        A1x_140_160=140+10
        A1y_140_160=0

    if(Count_160_180!=0):
        A1x_160_180=RSum_160_180/Count_160_180
        A1y_160_180=Count2_160_180/Count_160_180
    else:
        A1x_160_180=160+10
        A1y_160_180=0

    if(Count_180_200!=0):
        A1x_180_200=RSum_180_200/Count_180_200
        A1y_180_200=Count2_180_200/Count_180_200
    else:
        A1x_180_200=180+10
        A1y_180_200=0

    # This part prepears the plot axies and labels
    A1xList=[A1x_0_10,A1x_10_20,A1x_20_30,A1x_30_40,A1x_40_50,A1x_50_60,A1x_60_70,A1x_70_80,A1x_80_90,A1x_90_100,
            A1x_100_120,A1x_120_140,A1x_140_160,A1x_160_180,A1x_180_200] # List of x for the covering factor plot

    A1yList=[A1y_0_10,A1y_10_20,A1y_20_30,A1y_30_40,A1y_40_50,A1y_50_60,A1y_60_70,A1y_70_80,A1y_80_90,A1y_90_100,
            A1y_100_120,A1y_120_140,A1y_140_160,A1y_160_180,A1y_180_200] # List of y for the covering factor plot

    #print("X values (the mean of R_perp in each bin) for the covering factor plot are: ",AxList)
    #print("Y values (the covering factor in each bin) for the covering factor plot are ",AyList)
    plt.scatter(A1xList,A1yList,color="black") # Plot the points in the covering factor plot       
    plt.xlim([0,200])
    plt.ylim([0.0,1.0])
    plt.ylabel(r"$C_f$"+" (DLA)",fontsize=12,fontname="serif")
    plt.xticks([25,50,75,100,125,150,175,200], [" "," "," "," "," "," "," "," "])

    # Next part will plot the horizontal and vertical lines in the covering factor graph for the x errors
    plt.hlines(A1y_0_10,xmin=0,xmax=10,color='black',linestyle='-')
    plt.vlines(10,ymin=A1y_0_10-0.1,ymax=A1y_0_10+0.1,color='black',linestyle='-')

    plt.hlines(A1y_10_20,xmin=10,xmax=20,color='black',linestyle='-')
    plt.vlines(10,ymin=A1y_10_20-0.1,ymax=A1y_10_20+0.1,color='black',linestyle='-')
    plt.vlines(20,ymin=A1y_10_20-0.1,ymax=A1y_10_20+0.1,color='black',linestyle='-')

    plt.hlines(A1y_20_30,xmin=20,xmax=30,color='black',linestyle='-')
    plt.vlines(20,ymin=A1y_20_30-0.1,ymax=A1y_20_30+0.1,color='black',linestyle='-')
    plt.vlines(30,ymin=A1y_20_30-0.1,ymax=A1y_20_30+0.1,color='black',linestyle='-')

    plt.hlines(A1y_30_40,xmin=30,xmax=40,color='black',linestyle='-')
    plt.vlines(30,ymin=A1y_30_40-0.1,ymax=A1y_30_40+0.1,color='black',linestyle='-')
    plt.vlines(40,ymin=A1y_30_40-0.1,ymax=A1y_30_40+0.1,color='black',linestyle='-')

    plt.hlines(A1y_40_50,xmin=40,xmax=50,color='black',linestyle='-')
    plt.vlines(40,ymin=A1y_40_50-0.1,ymax=A1y_40_50+0.1,color='black',linestyle='-')
    plt.vlines(50,ymin=A1y_40_50-0.1,ymax=A1y_40_50+0.1,color='black',linestyle='-')

    plt.hlines(A1y_50_60,xmin=50,xmax=60,color='black',linestyle='-')
    plt.vlines(50,ymin=A1y_50_60-0.1,ymax=A1y_50_60+0.1,color='black',linestyle='-')
    plt.vlines(60,ymin=A1y_50_60-0.1,ymax=A1y_50_60+0.1,color='black',linestyle='-')

    plt.hlines(A1y_60_70,xmin=60,xmax=70,color='black',linestyle='-')
    plt.vlines(60,ymin=A1y_60_70-0.1,ymax=A1y_60_70+0.1,color='black',linestyle='-')
    plt.vlines(70,ymin=A1y_60_70-0.1,ymax=A1y_60_70+0.1,color='black',linestyle='-')

    plt.hlines(A1y_70_80,xmin=70,xmax=80,color='black',linestyle='-')
    plt.vlines(70,ymin=A1y_70_80-0.1,ymax=A1y_70_80+0.1,color='black',linestyle='-')
    plt.vlines(80,ymin=A1y_70_80-0.1,ymax=A1y_70_80+0.1,color='black',linestyle='-')

    plt.hlines(A1y_80_90,xmin=80,xmax=90,color='black',linestyle='-')
    plt.vlines(80,ymin=A1y_80_90-0.1,ymax=A1y_80_90+0.1,color='black',linestyle='-')
    plt.vlines(90,ymin=A1y_80_90-0.1,ymax=A1y_80_90+0.1,color='black',linestyle='-')

    plt.hlines(A1y_90_100,xmin=90,xmax=100,color='black',linestyle='-')
    plt.vlines(90,ymin=A1y_90_100-0.1,ymax=A1y_90_100+0.1,color='black',linestyle='-')
    plt.vlines(100,ymin=A1y_90_100-0.1,ymax=A1y_90_100+0.1,color='black',linestyle='-')

    plt.hlines(A1y_100_120,xmin=100,xmax=120,color='black',linestyle='-')
    plt.vlines(100,ymin=A1y_100_120-0.1,ymax=A1y_100_120+0.1,color='black',linestyle='-')
    plt.vlines(120,ymin=A1y_100_120-0.1,ymax=A1y_100_120+0.1,color='black',linestyle='-')

    plt.hlines(A1y_120_140,xmin=120,xmax=140,color='black',linestyle='-')
    plt.vlines(120,ymin=A1y_120_140-0.1,ymax=A1y_120_140+0.1,color='black',linestyle='-')
    plt.vlines(140,ymin=A1y_120_140-0.1,ymax=A1y_120_140+0.1,color='black',linestyle='-')

    plt.hlines(A1y_140_160,xmin=140,xmax=160,color='black',linestyle='-')
    plt.vlines(140,ymin=A1y_140_160-0.1,ymax=A1y_140_160+0.1,color='black',linestyle='-')
    plt.vlines(160,ymin=A1y_140_160-0.1,ymax=A1y_140_160+0.1,color='black',linestyle='-')

    plt.hlines(A1y_160_180,xmin=160,xmax=180,color='black',linestyle='-')
    plt.vlines(160,ymin=A1y_160_180-0.1,ymax=A1y_160_180+0.1,color='black',linestyle='-')
    plt.vlines(180,ymin=A1y_160_180-0.1,ymax=A1y_160_180+0.1,color='black',linestyle='-')

    plt.hlines(A1y_180_200,xmin=180,xmax=200,color='black',linestyle='-')
    plt.vlines(180,ymin=A1y_180_200-0.1,ymax=A1y_180_200+0.1,color='black',linestyle='-')
    plt.vlines(200,ymin=A1y_180_200-0.1,ymax=A1y_180_200+0.1,color='black',linestyle='-')
    #########################################################################################################################################
    # Covering Factor for LLS
    ax1=fig.add_subplot(ax[1,0:3]) # Grid for the covering factor plot
    # This part calculates the covering factor in each range
    Count_0_10=0 # For the total points in the R_perpendicular area
    RSum_0_10=0 # To calculate the mean of the R in this area
    Count2_0_10=0 # Only for the points with HI>10**17.2

    Count_10_20=0 # For the total points in the R_perpendicular area
    RSum_10_20=0 # To calculate the mean of the R in this area
    Count2_10_20=0 # Only for the points with HI>10**17.2

    Count_20_30=0 # For the total points in the R_perpendicular area
    RSum_20_30=0 # To calculate the mean of the R in this area
    Count2_20_30=0 # Only for the points with HI>10**17.2

    Count_30_40=0 # For the total points in the R_perpendicular area
    RSum_30_40=0 # To calculate the mean of the R in this area
    Count2_30_40=0 # Only for the points with HI>10**17.2

    Count_40_50=0 # For the total points in the R_perpendicular area
    RSum_40_50=0 # To calculate the mean of the R in this area
    Count2_40_50=0 # Only for the points with HI>10**17.2

    Count_50_60=0 # For the total points in the R_perpendicular area
    RSum_50_60=0 # To calculate the mean of the R in this area
    Count2_50_60=0 # Only for the points with HI>10**17.2

    Count_60_70=0 # For the total points in the R_perpendicular area
    RSum_60_70=0 # To calculate the mean of the R in this area
    Count2_60_70=0 # Only for the points with HI>10**17.2

    Count_70_80=0 # For the total points in the R_perpendicular area
    RSum_70_80=0 # To calculate the mean of the R in this area
    Count2_70_80=0 # Only for the points with HI>10**17.2

    Count_80_90=0 # For the total points in the R_perpendicular area
    RSum_80_90=0 # To calculate the mean of the R in this area
    Count2_80_90=0 # Only for the points with HI>10**17.2

    Count_90_100=0 # For the total points in the R_perpendicular area
    RSum_90_100=0 # To calculate the mean of the R in this area
    Count2_90_100=0 # Only for the points with HI>10**17.2

    Count_100_120=0 # For the total points in the R_perpendicular area
    RSum_100_120=0 # To calculate the mean of the R in this area
    Count2_100_120=0 # Only for the points with HI>10**17.2

    Count_120_140=0 # For the total points in the R_perpendicular area
    RSum_120_140=0 # To calculate the mean of the R in this area
    Count2_120_140=0 # Only for the points with HI>10**17.2

    Count_140_160=0 # For the total points in the R_perpendicular area
    RSum_140_160=0 # To calculate the mean of the R in this area
    Count2_140_160=0 # Only for the points with HI>10**17.2

    Count_160_180=0 # For the total points in the R_perpendicular area
    RSum_160_180=0 # To calculate the mean of the R in this area
    Count2_160_180=0 # Only for the points with HI>10**17.2

    Count_180_200=0 # For the total points in the R_perpendicular area
    RSum_180_200=0 # To calculate the mean of the R in this area
    Count2_180_200=0 # Only for the points with HI>10**17.2

    # This for loop calculates for each R_perp area how many total points there are, how many points are at least LLAs and what is the sum of R in each area (later to calculate the mean)
    for i in range(0,len(ListOfRBetweenTwoSightLines)):
        if(ListOfRBetweenTwoSightLines[i]<10):
            Count_0_10=Count_0_10+1
            RSum_0_10=RSum_0_10+ListOfRBetweenTwoSightLines[i]
            if(ListOfN_HIOfTheAnotherLine[i]>17.2):
                Count2_0_10=Count2_0_10+1

        if(ListOfRBetweenTwoSightLines[i]>10 and ListOfRBetweenTwoSightLines[i]<20):
            Count_10_20=Count_10_20+1
            RSum_10_20=RSum_10_20+ListOfRBetweenTwoSightLines[i]
            if(ListOfN_HIOfTheAnotherLine[i]>17.2):
                Count2_10_20=Count2_10_20+1

        if(ListOfRBetweenTwoSightLines[i]>20 and ListOfRBetweenTwoSightLines[i]<30):
            Count_20_30=Count_20_30+1
            RSum_20_30=RSum_20_30+ListOfRBetweenTwoSightLines[i]
            if(ListOfN_HIOfTheAnotherLine[i]>17.2):
                Count2_20_30=Count2_20_30+1

        if(ListOfRBetweenTwoSightLines[i]>30 and ListOfRBetweenTwoSightLines[i]<40):
            Count_30_40=Count_30_40+1
            RSum_30_40=RSum_30_40+ListOfRBetweenTwoSightLines[i]
            if(ListOfN_HIOfTheAnotherLine[i]>17.2):
                Count2_30_40=Count2_30_40+1    

        if(ListOfRBetweenTwoSightLines[i]>40 and ListOfRBetweenTwoSightLines[i]<50):
            Count_40_50=Count_40_50+1
            RSum_40_50=RSum_40_50+ListOfRBetweenTwoSightLines[i]
            if(ListOfN_HIOfTheAnotherLine[i]>17.2):
                Count2_40_50=Count2_40_50+1

        if(ListOfRBetweenTwoSightLines[i]>50 and ListOfRBetweenTwoSightLines[i]<60):
            Count_50_60=Count_50_60+1
            RSum_50_60=RSum_50_60+ListOfRBetweenTwoSightLines[i]
            if(ListOfN_HIOfTheAnotherLine[i]>17.2):
                Count2_50_60=Count2_50_60+1

        if(ListOfRBetweenTwoSightLines[i]>60 and ListOfRBetweenTwoSightLines[i]<70):
            Count_60_70=Count_60_70+1
            RSum_60_70=RSum_60_70+ListOfRBetweenTwoSightLines[i]
            if(ListOfN_HIOfTheAnotherLine[i]>17.2):
                Count2_60_70=Count2_60_70+1

        if(ListOfRBetweenTwoSightLines[i]>70 and ListOfRBetweenTwoSightLines[i]<80):
            Count_70_80=Count_70_80+1
            RSum_70_80=RSum_70_80+ListOfRBetweenTwoSightLines[i]
            if(ListOfN_HIOfTheAnotherLine[i]>17.2):
                Count2_70_80=Count2_70_80+1

        if(ListOfRBetweenTwoSightLines[i]>80 and ListOfRBetweenTwoSightLines[i]<90):
            Count_80_90=Count_80_90+1
            RSum_80_90=RSum_80_90+ListOfRBetweenTwoSightLines[i]
            if(ListOfN_HIOfTheAnotherLine[i]>17.2):
                Count2_80_90=Count2_80_90+1    

        if(ListOfRBetweenTwoSightLines[i]>90 and ListOfRBetweenTwoSightLines[i]<100):
            Count_90_100=Count_90_100+1
            RSum_90_100=RSum_90_100+ListOfRBetweenTwoSightLines[i]
            if(ListOfN_HIOfTheAnotherLine[i]>17.2):
                Count2_90_100=Count2_90_100+1

        if(ListOfRBetweenTwoSightLines[i]>100 and ListOfRBetweenTwoSightLines[i]<120):
            Count_100_120=Count_100_120+1
            RSum_100_120=RSum_100_120+ListOfRBetweenTwoSightLines[i]
            if(ListOfN_HIOfTheAnotherLine[i]>17.2):
                Count2_100_120=Count2_100_120+1

        if(ListOfRBetweenTwoSightLines[i]>120 and ListOfRBetweenTwoSightLines[i]<140):
            Count_120_140=Count_120_140+1
            RSum_120_140=RSum_120_140+ListOfRBetweenTwoSightLines[i]
            if(ListOfN_HIOfTheAnotherLine[i]>17.2):
                Count2_120_140=Count2_120_140+1

        if(ListOfRBetweenTwoSightLines[i]>140 and ListOfRBetweenTwoSightLines[i]<160):
            Count_140_160=Count_140_160+1
            RSum_140_160=RSum_140_160+ListOfRBetweenTwoSightLines[i]
            if(ListOfN_HIOfTheAnotherLine[i]>17.2):
                Count2_140_160=Count2_140_160+1

        if(ListOfRBetweenTwoSightLines[i]>160 and ListOfRBetweenTwoSightLines[i]<180):
            Count_160_180=Count_160_180+1
            RSum_160_180=RSum_160_180+ListOfRBetweenTwoSightLines[i]
            if(ListOfN_HIOfTheAnotherLine[i]>17.2):
                Count2_160_180=Count2_160_180+1    

        if(ListOfRBetweenTwoSightLines[i]>180 and ListOfRBetweenTwoSightLines[i]<200):
            Count_180_200=Count_180_200+1
            RSum_180_200=RSum_180_200+ListOfRBetweenTwoSightLines[i]
            if(ListOfN_HIOfTheAnotherLine[i]>17.2):
                Count2_180_200=Count2_180_200+1

    # This if part calculates the final x (that will be the maen of R in specific area) and y (the covering factor- how many point from all the pints are at least LLAs)
    if(Count_0_10!=0):
        A2x_0_10=RSum_0_10/Count_0_10
        A2y_0_10=Count2_0_10/Count_0_10
    else:
        A2x_0_10=5
        A2y_0_10=0

    if(Count_10_20!=0):
        A2x_10_20=RSum_10_20/Count_10_20
        A2y_10_20=Count2_10_20/Count_10_20
    else:
        A2x_10_20=10+5
        A2y_10_20=0    

    if(Count_20_30!=0):
        A2x_20_30=RSum_20_30/Count_20_30
        A2y_20_30=Count2_20_30/Count_20_30
    else:
        A2x_20_30=20+5
        A2y_20_30=0

    if(Count_30_40!=0):
        A2x_30_40=RSum_30_40/Count_30_40
        A2y_30_40=Count2_30_40/Count_30_40
    else:
        A2x_30_40=30+5
        A2y_30_40=0

    if(Count_40_50!=0):
        A2x_40_50=RSum_40_50/Count_40_50
        A2y_40_50=Count2_40_50/Count_40_50
    else:
        A2x_40_50=40+5
        A2y_40_50=0

    if(Count_50_60!=0):
        A2x_50_60=RSum_50_60/Count_50_60
        A2y_50_60=Count2_50_60/Count_50_60
    else:
        A2x_50_60=50+5
        A2y_50_60=0

    if(Count_60_70!=0):
        A2x_60_70=RSum_60_70/Count_60_70
        A2y_60_70=Count2_60_70/Count_60_70
    else:
        A2x_60_70=60+5
        A2y_60_70=0

    if(Count_70_80!=0):
        A2x_70_80=RSum_70_80/Count_70_80
        A2y_70_80=Count2_70_80/Count_70_80
    else:
        A2x_70_80=70+5
        A2y_70_80=0    

    if(Count_80_90!=0):
        A2x_80_90=RSum_80_90/Count_80_90
        A2y_80_90=Count2_80_90/Count_80_90
    else:
        A2x_80_90=80+5
        A2y_80_90=0

    if(Count_90_100!=0):
        A2x_90_100=RSum_90_100/Count_90_100
        A2y_90_100=Count2_90_100/Count_90_100
    else:
        A2x_90_100=90+5
        A2y_90_100=0

    if(Count_100_120!=0):
        A2x_100_120=RSum_100_120/Count_100_120
        A2y_100_120=Count2_100_120/Count_100_120
    else:
        A2x_100_120=90+10
        A2y_100_120=0 

    if(Count_120_140!=0):
        A2x_120_140=RSum_120_140/Count_120_140
        A2y_120_140=Count2_120_140/Count_120_140
    else:
        A2x_120_140=120+10
        A2y_120_140=0

    if(Count_140_160!=0):
        A2x_140_160=RSum_140_160/Count_140_160
        A2y_140_160=Count2_140_160/Count_140_160
    else:
        A2x_140_160=140+10
        A2y_140_160=0

    if(Count_160_180!=0):
        A2x_160_180=RSum_160_180/Count_160_180
        A2y_160_180=Count2_160_180/Count_160_180
    else:
        A2x_160_180=160+10
        A2y_160_180=0

    if(Count_180_200!=0):
        A2x_180_200=RSum_180_200/Count_180_200
        A2y_180_200=Count2_180_200/Count_180_200
    else:
        A2x_180_200=180+10
        A2y_180_200=0

    # This part prepears the plot axies and labels
    A2xList=[A2x_0_10,A2x_10_20,A2x_20_30,A2x_30_40,A2x_40_50,A2x_50_60,A2x_60_70,A2x_70_80,A2x_80_90,A2x_90_100,
            A2x_100_120,A2x_120_140,A2x_140_160,A2x_160_180,A2x_180_200] # List of x for the covering factor plot

    A2yList=[A2y_0_10,A2y_10_20,A2y_20_30,A2y_30_40,A2y_40_50,A2y_50_60,A2y_60_70,A2y_70_80,A2y_80_90,A2y_90_100,
            A2y_100_120,A2y_120_140,A2y_140_160,A2y_160_180,A2y_180_200] # List of y for the covering factor plot

    plt.scatter(A2xList,A2yList,color="black") # Plot the points in the covering factor plot       
    plt.xlim([0,200])
    plt.ylim([0.0,1.0])
    plt.ylabel(r"$C_f$"+" (LLS)",fontsize=12,fontname="serif")
    plt.xticks([25,50,75,100,125,150,175,200], [" "," "," "," "," "," "," "," "])

    # Next part will plot the horizontal and vertical lines in the covering factor graph for the x errors
    plt.hlines(A2y_0_10,xmin=0,xmax=10,color='black',linestyle='-')
    plt.vlines(10,ymin=A2y_0_10-0.1,ymax=A2y_0_10+0.1,color='black',linestyle='-')

    plt.hlines(A2y_10_20,xmin=10,xmax=20,color='black',linestyle='-')
    plt.vlines(10,ymin=A2y_10_20-0.1,ymax=A2y_10_20+0.1,color='black',linestyle='-')
    plt.vlines(20,ymin=A2y_10_20-0.1,ymax=A2y_10_20+0.1,color='black',linestyle='-')

    plt.hlines(A2y_20_30,xmin=20,xmax=30,color='black',linestyle='-')
    plt.vlines(20,ymin=A2y_20_30-0.1,ymax=A2y_20_30+0.1,color='black',linestyle='-')
    plt.vlines(30,ymin=A2y_20_30-0.1,ymax=A2y_20_30+0.1,color='black',linestyle='-')

    plt.hlines(A2y_30_40,xmin=30,xmax=40,color='black',linestyle='-')
    plt.vlines(30,ymin=A2y_30_40-0.1,ymax=A2y_30_40+0.1,color='black',linestyle='-')
    plt.vlines(40,ymin=A2y_30_40-0.1,ymax=A2y_30_40+0.1,color='black',linestyle='-')

    plt.hlines(A2y_40_50,xmin=40,xmax=50,color='black',linestyle='-')
    plt.vlines(40,ymin=A2y_40_50-0.1,ymax=A2y_40_50+0.1,color='black',linestyle='-')
    plt.vlines(50,ymin=A2y_40_50-0.1,ymax=A2y_40_50+0.1,color='black',linestyle='-')

    plt.hlines(A2y_50_60,xmin=50,xmax=60,color='black',linestyle='-')
    plt.vlines(50,ymin=A2y_50_60-0.1,ymax=A2y_50_60+0.1,color='black',linestyle='-')
    plt.vlines(60,ymin=A2y_50_60-0.1,ymax=A2y_50_60+0.1,color='black',linestyle='-')

    plt.hlines(A2y_60_70,xmin=60,xmax=70,color='black',linestyle='-')
    plt.vlines(60,ymin=A2y_60_70-0.1,ymax=A2y_60_70+0.1,color='black',linestyle='-')
    plt.vlines(70,ymin=A2y_60_70-0.1,ymax=A2y_60_70+0.1,color='black',linestyle='-')

    plt.hlines(A2y_70_80,xmin=70,xmax=80,color='black',linestyle='-')
    plt.vlines(70,ymin=A2y_70_80-0.1,ymax=A2y_70_80+0.1,color='black',linestyle='-')
    plt.vlines(80,ymin=A2y_70_80-0.1,ymax=A2y_70_80+0.1,color='black',linestyle='-')

    plt.hlines(A2y_80_90,xmin=80,xmax=90,color='black',linestyle='-')
    plt.vlines(80,ymin=A2y_80_90-0.1,ymax=A2y_80_90+0.1,color='black',linestyle='-')
    plt.vlines(90,ymin=A2y_80_90-0.1,ymax=A2y_80_90+0.1,color='black',linestyle='-')

    plt.hlines(A2y_90_100,xmin=90,xmax=100,color='black',linestyle='-')
    plt.vlines(90,ymin=A2y_90_100-0.1,ymax=A2y_90_100+0.1,color='black',linestyle='-')
    plt.vlines(100,ymin=A2y_90_100-0.1,ymax=A2y_90_100+0.1,color='black',linestyle='-')

    plt.hlines(A2y_100_120,xmin=100,xmax=120,color='black',linestyle='-')
    plt.vlines(100,ymin=A2y_100_120-0.1,ymax=A2y_100_120+0.1,color='black',linestyle='-')
    plt.vlines(120,ymin=A2y_100_120-0.1,ymax=A2y_100_120+0.1,color='black',linestyle='-')

    plt.hlines(A2y_120_140,xmin=120,xmax=140,color='black',linestyle='-')
    plt.vlines(120,ymin=A2y_120_140-0.1,ymax=A2y_120_140+0.1,color='black',linestyle='-')
    plt.vlines(140,ymin=A2y_120_140-0.1,ymax=A2y_120_140+0.1,color='black',linestyle='-')

    plt.hlines(A2y_140_160,xmin=140,xmax=160,color='black',linestyle='-')
    plt.vlines(140,ymin=A2y_140_160-0.1,ymax=A2y_140_160+0.1,color='black',linestyle='-')
    plt.vlines(160,ymin=A2y_140_160-0.1,ymax=A2y_140_160+0.1,color='black',linestyle='-')

    plt.hlines(A2y_160_180,xmin=160,xmax=180,color='black',linestyle='-')
    plt.vlines(160,ymin=A2y_160_180-0.1,ymax=A2y_160_180+0.1,color='black',linestyle='-')
    plt.vlines(180,ymin=A2y_160_180-0.1,ymax=A2y_160_180+0.1,color='black',linestyle='-')

    plt.hlines(A2y_180_200,xmin=180,xmax=200,color='black',linestyle='-')
    plt.vlines(180,ymin=A2y_180_200-0.1,ymax=A2y_180_200+0.1,color='black',linestyle='-')
    plt.vlines(200,ymin=A2y_180_200-0.1,ymax=A2y_180_200+0.1,color='black',linestyle='-')

    # Next part will handle the main graph of N_HI
    ax1 = fig.add_subplot(ax[2:,0:3]) # Plot for the main graph
    plt.scatter(ListOfRBetweenTwoSightLines,ListOfN_HIOfTheAnotherLine) # Plot the N_HI against R⊥
    plt.xlabel(r"$R_⊥$"+" [kpc]"+" ("+r"$R_\perp$"+" from DLA)",fontsize=12,fontname="serif")
    plt.ylabel("log "+r"$N_{\rm HI}$"+" "+"[cm"+r"$^{-2}$"+"]",fontsize=12,fontname="serif")
    #plt.yticks([14,15,16,17,18,19,20,21,22],["14","15","16","17","18","19","20","21","22"])
    plt.autoscale(enable=True,axis='y',tight=True) # Allows the axhspan to go up till the edge
    plt.axhspan(20.3,22,color='gray',alpha=0.5,label="DLA") # Mark the area of the DLAs
    plt.axhspan(17.2,20.3,color='lightgray',alpha=0.5,label="LLS") # Mark the area of the LLSs
    lightgray_patch=mpatches.Patch(color='lightgray',label='LLS') # Creates patch for the LLS
    gray_patch=mpatches.Patch(color='gray',label='DLA') # Creates patch for the DLA
    plt.legend(handles=[gray_patch,lightgray_patch],loc='lower right')
    plt.xlim([0,200])
    plt.ylim([13.5,21.5])
    #plt.savefig('/home1/08289/tg875885/radial_to_rotating_flows/Aharon/Outputs2/'+SnapDirect+"~log(N_HI)[R⊥].png")
    plt.show()
    plt.clf()
    plt.cla()
    plt.close()
    print("DONE 2")
    ######################################################################################################################################
    ######################################################################################################################################
    # log(N_HI)[R⊥] - R⊥ is the distance between a random sightline and the center of the halo (N_HI is of the random sightline)
    plt.rcParams['mathtext.fontset']='dejavuserif'
    plt.clf()
    hist=NHImap
    x_bins=Xs
    y_bins=Ys
    def CalculateTheDistanceBetweenTwoPoints(x1,y1,x2,y2): # Function that calculates the distance between 2 points (x1,y1,x2,y2)
            r1=np.sqrt(x1**2+y1**2)
            r2=np.sqrt(x2**2+y2**2)
            Distance=np.abs(r1-r2)
            return Distance
    ListOfRBetweenSightlineAndTheCenter=[] # Will be the x values for the N_HI against R⊥ plot
    ListOfN_HIOfTheSightline=[] # Will be the y values for the N_HI against R⊥ plot

    # This part will random 100 pairs in each one of the bins: [0,10], [10,20], [20,30], [30,40] ...
    def RandomSightinesForSpecificBin(R_perpLower,R_perpUpper): # Function that generate 100 pairs of sightlines for a specific given bin 
        CountOfRandom100=0
        while(CountOfRandom100<100):
            xSightlineIndex=random.randint(0,hist.shape[0]-1) # Find the relevant x index for the sighline
            ySightlineIndex=random.randint(0,hist.shape[1]-1) # Find the relevant y index for the sighline
            while(hist[xSightlineIndex][ySightlineIndex]==0): # Make sure that the random sighline has no 0 value for the colmun density
                xSightlineIndex=random.randint(0,hist.shape[0]-1) # Find the relevant x index for the sighline
                ySightlineIndex=random.randint(0,hist.shape[1]-1) # Find the relevant y index for the sighline
            R_perp=CalculateTheDistanceBetweenTwoPoints(0,0,x_bins[xSightlineIndex],y_bins[ySightlineIndex])
            if(R_perp<R_perpUpper and R_perp>R_perpLower):
                ListOfN_HIOfTheSightline.append(hist[xSightlineIndex][ySightlineIndex]) # Insert the N_HI of the sightline to the final y list
                ListOfRBetweenSightlineAndTheCenter.append(R_perp) # Insert the R⊥ between the sightline and the center to the final x list
                CountOfRandom100=CountOfRandom100+1       
    ProjectionLimit=200            
    R_perpLower=0
    R_perpUpper=10
    count=0
    while(R_perpUpper<ProjectionLimit+10): # Generate all of the sightlines in the projection limit (100 pairs for each bin, each bin=10kpc)
        RandomSightinesForSpecificBin(R_perpLower,R_perpUpper)
        print("done",count)
        count=count+1
        R_perpLower=R_perpLower+10
        R_perpUpper=R_perpUpper+10

    fig=plt.figure(constrained_layout=True) # Creates the figure
    ax=fig.add_gridspec(5,3) # Creates the figure structure
    #########################################################################################################################################
    # Covering Factor for DLA
    ax2=fig.add_subplot(ax[0,0:3]) # Grid for the covering factor plot
    # This part calculates the covering factor in each range
    Count_0_10=0 # For the total points in the R_perpendicular area
    RSum_0_10=0 # To calculate the mean of the R in this area
    Count2_0_10=0 # Only for the points with HI>10**17.2

    Count_10_20=0 # For the total points in the R_perpendicular area
    RSum_10_20=0 # To calculate the mean of the R in this area
    Count2_10_20=0 # Only for the points with HI>10**17.2

    Count_20_30=0 # For the total points in the R_perpendicular area
    RSum_20_30=0 # To calculate the mean of the R in this area
    Count2_20_30=0 # Only for the points with HI>10**17.2

    Count_30_40=0 # For the total points in the R_perpendicular area
    RSum_30_40=0 # To calculate the mean of the R in this area
    Count2_30_40=0 # Only for the points with HI>10**17.2

    Count_40_50=0 # For the total points in the R_perpendicular area
    RSum_40_50=0 # To calculate the mean of the R in this area
    Count2_40_50=0 # Only for the points with HI>10**17.2

    Count_50_60=0 # For the total points in the R_perpendicular area
    RSum_50_60=0 # To calculate the mean of the R in this area
    Count2_50_60=0 # Only for the points with HI>10**17.2

    Count_60_70=0 # For the total points in the R_perpendicular area
    RSum_60_70=0 # To calculate the mean of the R in this area
    Count2_60_70=0 # Only for the points with HI>10**17.2

    Count_70_80=0 # For the total points in the R_perpendicular area
    RSum_70_80=0 # To calculate the mean of the R in this area
    Count2_70_80=0 # Only for the points with HI>10**17.2

    Count_80_90=0 # For the total points in the R_perpendicular area
    RSum_80_90=0 # To calculate the mean of the R in this area
    Count2_80_90=0 # Only for the points with HI>10**17.2

    Count_90_100=0 # For the total points in the R_perpendicular area
    RSum_90_100=0 # To calculate the mean of the R in this area
    Count2_90_100=0 # Only for the points with HI>10**17.2

    Count_100_120=0 # For the total points in the R_perpendicular area
    RSum_100_120=0 # To calculate the mean of the R in this area
    Count2_100_120=0 # Only for the points with HI>10**17.2

    Count_120_140=0 # For the total points in the R_perpendicular area
    RSum_120_140=0 # To calculate the mean of the R in this area
    Count2_120_140=0 # Only for the points with HI>10**17.2

    Count_140_160=0 # For the total points in the R_perpendicular area
    RSum_140_160=0 # To calculate the mean of the R in this area
    Count2_140_160=0 # Only for the points with HI>10**17.2

    Count_160_180=0 # For the total points in the R_perpendicular area
    RSum_160_180=0 # To calculate the mean of the R in this area
    Count2_160_180=0 # Only for the points with HI>10**17.2

    Count_180_200=0 # For the total points in the R_perpendicular area
    RSum_180_200=0 # To calculate the mean of the R in this area
    Count2_180_200=0 # Only for the points with HI>10**17.2

    # This for loop calculates for each R_perp area how many total points there are, how many points are at least DLAs and what is the sum of R in each area (later to calculate the mean)
    for i in range(0,len(ListOfRBetweenSightlineAndTheCenter)):
        if(ListOfRBetweenSightlineAndTheCenter[i]<10):
            Count_0_10=Count_0_10+1
            RSum_0_10=RSum_0_10+ListOfRBetweenSightlineAndTheCenter[i]
            if(ListOfN_HIOfTheSightline[i]>20.3):
                Count2_0_10=Count2_0_10+1

        if(ListOfRBetweenSightlineAndTheCenter[i]>10 and ListOfRBetweenSightlineAndTheCenter[i]<20):
            Count_10_20=Count_10_20+1
            RSum_10_20=RSum_10_20+ListOfRBetweenSightlineAndTheCenter[i]
            if(ListOfN_HIOfTheSightline[i]>20.3):
                Count2_10_20=Count2_10_20+1

        if(ListOfRBetweenSightlineAndTheCenter[i]>20 and ListOfRBetweenSightlineAndTheCenter[i]<30):
            Count_20_30=Count_20_30+1
            RSum_20_30=RSum_20_30+ListOfRBetweenSightlineAndTheCenter[i]
            if(ListOfN_HIOfTheSightline[i]>20.3):
                Count2_20_30=Count2_20_30+1

        if(ListOfRBetweenSightlineAndTheCenter[i]>30 and ListOfRBetweenSightlineAndTheCenter[i]<40):
            Count_30_40=Count_30_40+1
            RSum_30_40=RSum_30_40+ListOfRBetweenSightlineAndTheCenter[i]
            if(ListOfN_HIOfTheSightline[i]>20.3):
                Count2_30_40=Count2_30_40+1    

        if(ListOfRBetweenSightlineAndTheCenter[i]>40 and ListOfRBetweenSightlineAndTheCenter[i]<50):
            Count_40_50=Count_40_50+1
            RSum_40_50=RSum_40_50+ListOfRBetweenSightlineAndTheCenter[i]
            if(ListOfN_HIOfTheSightline[i]>20.3):
                Count2_40_50=Count2_40_50+1

        if(ListOfRBetweenSightlineAndTheCenter[i]>50 and ListOfRBetweenSightlineAndTheCenter[i]<60):
            Count_50_60=Count_50_60+1
            RSum_50_60=RSum_50_60+ListOfRBetweenSightlineAndTheCenter[i]
            if(ListOfN_HIOfTheSightline[i]>20.3):
                Count2_50_60=Count2_50_60+1

        if(ListOfRBetweenSightlineAndTheCenter[i]>60 and ListOfRBetweenSightlineAndTheCenter[i]<70):
            Count_60_70=Count_60_70+1
            RSum_60_70=RSum_60_70+ListOfRBetweenSightlineAndTheCenter[i]
            if(ListOfN_HIOfTheSightline[i]>20.3):
                Count2_60_70=Count2_60_70+1

        if(ListOfRBetweenSightlineAndTheCenter[i]>70 and ListOfRBetweenSightlineAndTheCenter[i]<80):
            Count_70_80=Count_70_80+1
            RSum_70_80=RSum_70_80+ListOfRBetweenSightlineAndTheCenter[i]
            if(ListOfN_HIOfTheSightline[i]>20.3):
                Count2_70_80=Count2_70_80+1

        if(ListOfRBetweenSightlineAndTheCenter[i]>80 and ListOfRBetweenSightlineAndTheCenter[i]<90):
            Count_80_90=Count_80_90+1
            RSum_80_90=RSum_80_90+ListOfRBetweenSightlineAndTheCenter[i]
            if(ListOfN_HIOfTheSightline[i]>20.3):
                Count2_80_90=Count2_80_90+1    

        if(ListOfRBetweenSightlineAndTheCenter[i]>90 and ListOfRBetweenSightlineAndTheCenter[i]<100):
            Count_90_100=Count_90_100+1
            RSum_90_100=RSum_90_100+ListOfRBetweenSightlineAndTheCenter[i]
            if(ListOfN_HIOfTheSightline[i]>20.3):
                Count2_90_100=Count2_90_100+1

        if(ListOfRBetweenSightlineAndTheCenter[i]>100 and ListOfRBetweenSightlineAndTheCenter[i]<120):
            Count_100_120=Count_100_120+1
            RSum_100_120=RSum_100_120+ListOfRBetweenSightlineAndTheCenter[i]
            if(ListOfN_HIOfTheSightline[i]>20.3):
                Count2_100_120=Count2_100_120+1       

        if(ListOfRBetweenSightlineAndTheCenter[i]>120 and ListOfRBetweenSightlineAndTheCenter[i]<140):
            Count_120_140=Count_120_140+1
            RSum_120_140=RSum_120_140+ListOfRBetweenSightlineAndTheCenter[i]
            if(ListOfN_HIOfTheSightline[i]>20.3):
                Count2_120_140=Count2_120_140+1    

        if(ListOfRBetweenSightlineAndTheCenter[i]>140 and ListOfRBetweenSightlineAndTheCenter[i]<160):
            Count_140_160=Count_140_160+1
            RSum_140_160=RSum_140_160+ListOfRBetweenSightlineAndTheCenter[i]
            if(ListOfN_HIOfTheSightline[i]>20.3):
                Count2_140_160=Count2_140_160+1

        if(ListOfRBetweenSightlineAndTheCenter[i]>160 and ListOfRBetweenSightlineAndTheCenter[i]<180):
            Count_160_180=Count_160_180+1
            RSum_160_180=RSum_160_180+ListOfRBetweenSightlineAndTheCenter[i]
            if(ListOfN_HIOfTheSightline[i]>20.3):
                Count2_160_180=Count2_160_180+1

        if(ListOfRBetweenSightlineAndTheCenter[i]>180 and ListOfRBetweenSightlineAndTheCenter[i]<200):
            Count_180_200=Count_180_200+1
            RSum_180_200=RSum_180_200+ListOfRBetweenSightlineAndTheCenter[i]
            if(ListOfN_HIOfTheSightline[i]>20.3):
                Count2_180_200=Count2_180_200+1       

    # This if part calculates the final x (that will be the maen of R in specific area) and y (the covering factor- how many point from all the pints are at least LLAs)
    if(Count_0_10!=0):
        B1x_0_10=RSum_0_10/Count_0_10
        B1y_0_10=Count2_0_10/Count_0_10
    else:
        B1x_0_10=5
        B1y_0_10=0

    if(Count_10_20!=0):
        B1x_10_20=RSum_10_20/Count_10_20
        B1y_10_20=Count2_10_20/Count_10_20
    else:
        B1x_10_20=10+5
        B1y_10_20=0    

    if(Count_20_30!=0):
        B1x_20_30=RSum_20_30/Count_20_30
        B1y_20_30=Count2_20_30/Count_20_30
    else:
        B1x_20_30=20+5
        B1y_20_30=0

    if(Count_30_40!=0):
        B1x_30_40=RSum_30_40/Count_30_40
        B1y_30_40=Count2_30_40/Count_30_40
    else:
        B1x_30_40=30+5
        B1y_30_40=0

    if(Count_40_50!=0):
        B1x_40_50=RSum_40_50/Count_40_50
        B1y_40_50=Count2_40_50/Count_40_50
    else:
        B1x_40_50=40+5
        B1y_40_50=0

    if(Count_50_60!=0):
        B1x_50_60=RSum_50_60/Count_50_60
        B1y_50_60=Count2_50_60/Count_50_60
    else:
        B1x_50_60=50+5
        B1y_50_60=0

    if(Count_60_70!=0):
        B1x_60_70=RSum_60_70/Count_60_70
        B1y_60_70=Count2_60_70/Count_60_70
    else:
        B1x_60_70=60+5
        B1y_60_70=0

    if(Count_70_80!=0):
        B1x_70_80=RSum_70_80/Count_70_80
        B1y_70_80=Count2_70_80/Count_70_80
    else:
        B1x_70_80=70+5
        B1y_70_80=0    

    if(Count_80_90!=0):
        B1x_80_90=RSum_80_90/Count_80_90
        B1y_80_90=Count2_80_90/Count_80_90
    else:
        B1x_80_90=80+5
        B1y_80_90=0

    if(Count_90_100!=0):
        B1x_90_100=RSum_90_100/Count_90_100
        B1y_90_100=Count2_90_100/Count_90_100
    else:
        B1x_90_100=90+5
        B1y_90_100=0

    if(Count_100_120!=0):
        B1x_100_120=RSum_100_120/Count_100_120
        B1y_100_120=Count2_100_120/Count_100_120
    else:
        B1x_100_120=90+10
        B1y_100_120=0 

    if(Count_120_140!=0):
        B1x_120_140=RSum_120_140/Count_120_140
        B1y_120_140=Count2_120_140/Count_120_140
    else:
        B1x_120_140=120+10
        B1y_120_140=0

    if(Count_140_160!=0):
        B1x_140_160=RSum_140_160/Count_140_160
        B1y_140_160=Count2_140_160/Count_140_160
    else:
        B1x_140_160=140+10
        B1y_140_160=0

    if(Count_160_180!=0):
        B1x_160_180=RSum_160_180/Count_160_180
        B1y_160_180=Count2_160_180/Count_160_180
    else:
        B1x_160_180=160+10
        B1y_160_180=0

    if(Count_180_200!=0):
        B1x_180_200=RSum_180_200/Count_180_200
        B1y_180_200=Count2_180_200/Count_180_200
    else:
        B1x_180_200=180+10
        B1y_180_200=0

    # This part prepears the plot axies and labels
    B1xList=[B1x_0_10,B1x_10_20,B1x_20_30,B1x_30_40,B1x_40_50,B1x_50_60,B1x_60_70,B1x_70_80,B1x_80_90,B1x_90_100,
            B1x_100_120,B1x_120_140,B1x_140_160,B1x_160_180,B1x_180_200] # List of x for the covering factor plot

    B1yList=[B1y_0_10,B1y_10_20,B1y_20_30,B1y_30_40,B1y_40_50,B1y_50_60,B1y_60_70,B1y_70_80,B1y_80_90,B1y_90_100,
            B1y_100_120,B1y_120_140,B1y_140_160,B1y_160_180,B1y_180_200] # List of y for the covering factor plot

    #print("X values (the mean of R_perp in each bin) for the covering factor plot are: ",AxList)
    #print("Y values (the covering factor in each bin) for the covering factor plot are ",AyList)
    plt.scatter(B1xList,B1yList,color="black") # Plot the points in the covering factor plot       
    plt.xlim([0,200])
    plt.ylim([0.0,1.0])
    plt.ylabel(r"$C_f$"+" (DLA)",fontsize=12,fontname="serif")
    plt.xticks([25,50,75,100,125,150,175,200], [" "," "," "," "," "," "," "," "])

    # Next part will plot the horizontal and vertical lines in the covering factor graph for the x errors
    plt.hlines(B1y_0_10,xmin=0,xmax=10,color='black',linestyle='-')
    plt.vlines(10,ymin=B1y_0_10-0.1,ymax=B1y_0_10+0.1,color='black',linestyle='-')

    plt.hlines(B1y_10_20,xmin=10,xmax=20,color='black',linestyle='-')
    plt.vlines(10,ymin=B1y_10_20-0.1,ymax=B1y_10_20+0.1,color='black',linestyle='-')
    plt.vlines(20,ymin=B1y_10_20-0.1,ymax=B1y_10_20+0.1,color='black',linestyle='-')

    plt.hlines(B1y_20_30,xmin=20,xmax=30,color='black',linestyle='-')
    plt.vlines(20,ymin=B1y_20_30-0.1,ymax=B1y_20_30+0.1,color='black',linestyle='-')
    plt.vlines(30,ymin=B1y_20_30-0.1,ymax=B1y_20_30+0.1,color='black',linestyle='-')

    plt.hlines(B1y_30_40,xmin=30,xmax=40,color='black',linestyle='-')
    plt.vlines(30,ymin=B1y_30_40-0.1,ymax=B1y_30_40+0.1,color='black',linestyle='-')
    plt.vlines(40,ymin=B1y_30_40-0.1,ymax=B1y_30_40+0.1,color='black',linestyle='-')

    plt.hlines(B1y_40_50,xmin=40,xmax=50,color='black',linestyle='-')
    plt.vlines(40,ymin=B1y_40_50-0.1,ymax=B1y_40_50+0.1,color='black',linestyle='-')
    plt.vlines(50,ymin=B1y_40_50-0.1,ymax=B1y_40_50+0.1,color='black',linestyle='-')

    plt.hlines(B1y_50_60,xmin=50,xmax=60,color='black',linestyle='-')
    plt.vlines(50,ymin=B1y_50_60-0.1,ymax=B1y_50_60+0.1,color='black',linestyle='-')
    plt.vlines(60,ymin=B1y_50_60-0.1,ymax=B1y_50_60+0.1,color='black',linestyle='-')

    plt.hlines(B1y_60_70,xmin=60,xmax=70,color='black',linestyle='-')
    plt.vlines(60,ymin=B1y_60_70-0.1,ymax=B1y_60_70+0.1,color='black',linestyle='-')
    plt.vlines(70,ymin=B1y_60_70-0.1,ymax=B1y_60_70+0.1,color='black',linestyle='-')

    plt.hlines(B1y_70_80,xmin=70,xmax=80,color='black',linestyle='-')
    plt.vlines(70,ymin=B1y_70_80-0.1,ymax=B1y_70_80+0.1,color='black',linestyle='-')
    plt.vlines(80,ymin=B1y_70_80-0.1,ymax=B1y_70_80+0.1,color='black',linestyle='-')

    plt.hlines(B1y_80_90,xmin=80,xmax=90,color='black',linestyle='-')
    plt.vlines(80,ymin=B1y_80_90-0.1,ymax=B1y_80_90+0.1,color='black',linestyle='-')
    plt.vlines(90,ymin=B1y_80_90-0.1,ymax=B1y_80_90+0.1,color='black',linestyle='-')

    plt.hlines(B1y_90_100,xmin=90,xmax=100,color='black',linestyle='-')
    plt.vlines(90,ymin=B1y_90_100-0.1,ymax=B1y_90_100+0.1,color='black',linestyle='-')
    plt.vlines(100,ymin=B1y_90_100-0.1,ymax=B1y_90_100+0.1,color='black',linestyle='-')

    plt.hlines(B1y_100_120,xmin=100,xmax=120,color='black',linestyle='-')
    plt.vlines(100,ymin=B1y_100_120-0.1,ymax=B1y_100_120+0.1,color='black',linestyle='-')
    plt.vlines(120,ymin=B1y_100_120-0.1,ymax=B1y_100_120+0.1,color='black',linestyle='-')

    plt.hlines(B1y_120_140,xmin=120,xmax=140,color='black',linestyle='-')
    plt.vlines(120,ymin=B1y_120_140-0.1,ymax=B1y_120_140+0.1,color='black',linestyle='-')
    plt.vlines(140,ymin=B1y_120_140-0.1,ymax=B1y_120_140+0.1,color='black',linestyle='-')

    plt.hlines(B1y_140_160,xmin=140,xmax=160,color='black',linestyle='-')
    plt.vlines(140,ymin=B1y_140_160-0.1,ymax=B1y_140_160+0.1,color='black',linestyle='-')
    plt.vlines(160,ymin=B1y_140_160-0.1,ymax=B1y_140_160+0.1,color='black',linestyle='-')

    plt.hlines(B1y_160_180,xmin=160,xmax=180,color='black',linestyle='-')
    plt.vlines(160,ymin=B1y_160_180-0.1,ymax=B1y_160_180+0.1,color='black',linestyle='-')
    plt.vlines(180,ymin=B1y_160_180-0.1,ymax=B1y_160_180+0.1,color='black',linestyle='-')

    plt.hlines(B1y_180_200,xmin=180,xmax=200,color='black',linestyle='-')
    plt.vlines(180,ymin=B1y_180_200-0.1,ymax=B1y_180_200+0.1,color='black',linestyle='-')
    plt.vlines(200,ymin=B1y_180_200-0.1,ymax=B1y_180_200+0.1,color='black',linestyle='-')
    #########################################################################################################################################
    # Covering Factor for LLS
    ax1=fig.add_subplot(ax[1,0:3]) # Grid for the covering factor plot
    # This part calculates the covering factor in each range
    Count_0_10=0 # For the total points in the R_perpendicular area
    RSum_0_10=0 # To calculate the mean of the R in this area
    Count2_0_10=0 # Only for the points with HI>10**17.2

    Count_10_20=0 # For the total points in the R_perpendicular area
    RSum_10_20=0 # To calculate the mean of the R in this area
    Count2_10_20=0 # Only for the points with HI>10**17.2

    Count_20_30=0 # For the total points in the R_perpendicular area
    RSum_20_30=0 # To calculate the mean of the R in this area
    Count2_20_30=0 # Only for the points with HI>10**17.2

    Count_30_40=0 # For the total points in the R_perpendicular area
    RSum_30_40=0 # To calculate the mean of the R in this area
    Count2_30_40=0 # Only for the points with HI>10**17.2

    Count_40_50=0 # For the total points in the R_perpendicular area
    RSum_40_50=0 # To calculate the mean of the R in this area
    Count2_40_50=0 # Only for the points with HI>10**17.2

    Count_50_60=0 # For the total points in the R_perpendicular area
    RSum_50_60=0 # To calculate the mean of the R in this area
    Count2_50_60=0 # Only for the points with HI>10**17.2

    Count_60_70=0 # For the total points in the R_perpendicular area
    RSum_60_70=0 # To calculate the mean of the R in this area
    Count2_60_70=0 # Only for the points with HI>10**17.2

    Count_70_80=0 # For the total points in the R_perpendicular area
    RSum_70_80=0 # To calculate the mean of the R in this area
    Count2_70_80=0 # Only for the points with HI>10**17.2

    Count_80_90=0 # For the total points in the R_perpendicular area
    RSum_80_90=0 # To calculate the mean of the R in this area
    Count2_80_90=0 # Only for the points with HI>10**17.2

    Count_90_100=0 # For the total points in the R_perpendicular area
    RSum_90_100=0 # To calculate the mean of the R in this area
    Count2_90_100=0 # Only for the points with HI>10**17.2

    Count_100_120=0 # For the total points in the R_perpendicular area
    RSum_100_120=0 # To calculate the mean of the R in this area
    Count2_100_120=0 # Only for the points with HI>10**17.2

    Count_120_140=0 # For the total points in the R_perpendicular area
    RSum_120_140=0 # To calculate the mean of the R in this area
    Count2_120_140=0 # Only for the points with HI>10**17.2

    Count_140_160=0 # For the total points in the R_perpendicular area
    RSum_140_160=0 # To calculate the mean of the R in this area
    Count2_140_160=0 # Only for the points with HI>10**17.2

    Count_160_180=0 # For the total points in the R_perpendicular area
    RSum_160_180=0 # To calculate the mean of the R in this area
    Count2_160_180=0 # Only for the points with HI>10**17.2

    Count_180_200=0 # For the total points in the R_perpendicular area
    RSum_180_200=0 # To calculate the mean of the R in this area
    Count2_180_200=0 # Only for the points with HI>10**17.2

    # This for loop calculates for each R_perp area how many total points there are, how many points are at least LLAs and what is the sum of R in each area (later to calculate the mean)
    for i in range(0,len(ListOfRBetweenSightlineAndTheCenter)):
        if(ListOfRBetweenSightlineAndTheCenter[i]<10):
            Count_0_10=Count_0_10+1
            RSum_0_10=RSum_0_10+ListOfRBetweenSightlineAndTheCenter[i]
            if(ListOfN_HIOfTheSightline[i]>17.2):
                Count2_0_10=Count2_0_10+1

        if(ListOfRBetweenSightlineAndTheCenter[i]>10 and ListOfRBetweenSightlineAndTheCenter[i]<20):
            Count_10_20=Count_10_20+1
            RSum_10_20=RSum_10_20+ListOfRBetweenSightlineAndTheCenter[i]
            if(ListOfN_HIOfTheSightline[i]>17.2):
                Count2_10_20=Count2_10_20+1

        if(ListOfRBetweenSightlineAndTheCenter[i]>20 and ListOfRBetweenSightlineAndTheCenter[i]<30):
            Count_20_30=Count_20_30+1
            RSum_20_30=RSum_20_30+ListOfRBetweenSightlineAndTheCenter[i]
            if(ListOfN_HIOfTheSightline[i]>17.2):
                Count2_20_30=Count2_20_30+1

        if(ListOfRBetweenSightlineAndTheCenter[i]>30 and ListOfRBetweenSightlineAndTheCenter[i]<40):
            Count_30_40=Count_30_40+1
            RSum_30_40=RSum_30_40+ListOfRBetweenSightlineAndTheCenter[i]
            if(ListOfN_HIOfTheSightline[i]>17.2):
                Count2_30_40=Count2_30_40+1    

        if(ListOfRBetweenSightlineAndTheCenter[i]>40 and ListOfRBetweenSightlineAndTheCenter[i]<50):
            Count_40_50=Count_40_50+1
            RSum_40_50=RSum_40_50+ListOfRBetweenSightlineAndTheCenter[i]
            if(ListOfN_HIOfTheSightline[i]>17.2):
                Count2_40_50=Count2_40_50+1

        if(ListOfRBetweenSightlineAndTheCenter[i]>50 and ListOfRBetweenSightlineAndTheCenter[i]<60):
            Count_50_60=Count_50_60+1
            RSum_50_60=RSum_50_60+ListOfRBetweenSightlineAndTheCenter[i]
            if(ListOfN_HIOfTheSightline[i]>17.2):
                Count2_50_60=Count2_50_60+1

        if(ListOfRBetweenSightlineAndTheCenter[i]>60 and ListOfRBetweenSightlineAndTheCenter[i]<70):
            Count_60_70=Count_60_70+1
            RSum_60_70=RSum_60_70+ListOfRBetweenSightlineAndTheCenter[i]
            if(ListOfN_HIOfTheSightline[i]>17.2):
                Count2_60_70=Count2_60_70+1

        if(ListOfRBetweenSightlineAndTheCenter[i]>70 and ListOfRBetweenSightlineAndTheCenter[i]<80):
            Count_70_80=Count_70_80+1
            RSum_70_80=RSum_70_80+ListOfRBetweenSightlineAndTheCenter[i]
            if(ListOfN_HIOfTheSightline[i]>17.2):
                Count2_70_80=Count2_70_80+1

        if(ListOfRBetweenSightlineAndTheCenter[i]>80 and ListOfRBetweenSightlineAndTheCenter[i]<90):
            Count_80_90=Count_80_90+1
            RSum_80_90=RSum_80_90+ListOfRBetweenSightlineAndTheCenter[i]
            if(ListOfN_HIOfTheSightline[i]>17.2):
                Count2_80_90=Count2_80_90+1    

        if(ListOfRBetweenSightlineAndTheCenter[i]>90 and ListOfRBetweenSightlineAndTheCenter[i]<100):
            Count_90_100=Count_90_100+1
            RSum_90_100=RSum_90_100+ListOfRBetweenSightlineAndTheCenter[i]
            if(ListOfN_HIOfTheSightline[i]>17.2):
                Count2_90_100=Count2_90_100+1

        if(ListOfRBetweenSightlineAndTheCenter[i]>100 and ListOfRBetweenSightlineAndTheCenter[i]<120):
            Count_100_120=Count_100_120+1
            RSum_100_120=RSum_100_120+ListOfRBetweenSightlineAndTheCenter[i]
            if(ListOfN_HIOfTheSightline[i]>17.2):
                Count2_100_120=Count2_100_120+1       

        if(ListOfRBetweenSightlineAndTheCenter[i]>120 and ListOfRBetweenSightlineAndTheCenter[i]<140):
            Count_120_140=Count_120_140+1
            RSum_120_140=RSum_120_140+ListOfRBetweenSightlineAndTheCenter[i]
            if(ListOfN_HIOfTheSightline[i]>17.2):
                Count2_120_140=Count2_120_140+1    

        if(ListOfRBetweenSightlineAndTheCenter[i]>140 and ListOfRBetweenSightlineAndTheCenter[i]<160):
            Count_140_160=Count_140_160+1
            RSum_140_160=RSum_140_160+ListOfRBetweenSightlineAndTheCenter[i]
            if(ListOfN_HIOfTheSightline[i]>17.2):
                Count2_140_160=Count2_140_160+1

        if(ListOfRBetweenSightlineAndTheCenter[i]>160 and ListOfRBetweenSightlineAndTheCenter[i]<180):
            Count_160_180=Count_160_180+1
            RSum_160_180=RSum_160_180+ListOfRBetweenSightlineAndTheCenter[i]
            if(ListOfN_HIOfTheSightline[i]>17.2):
                Count2_160_180=Count2_160_180+1

        if(ListOfRBetweenSightlineAndTheCenter[i]>180 and ListOfRBetweenSightlineAndTheCenter[i]<200):
            Count_180_200=Count_180_200+1
            RSum_180_200=RSum_180_200+ListOfRBetweenSightlineAndTheCenter[i]
            if(ListOfN_HIOfTheSightline[i]>17.2):
                Count2_180_200=Count2_180_200+1     

    # This if part calculates the final x (that will be the maen of R in specific area) and y (the covering factor- how many point from all the pints are at least LLAs)
    if(Count_0_10!=0):
        B2x_0_10=RSum_0_10/Count_0_10
        B2y_0_10=Count2_0_10/Count_0_10
    else:
        B2x_0_10=5
        B2y_0_10=0

    if(Count_10_20!=0):
        B2x_10_20=RSum_10_20/Count_10_20
        B2y_10_20=Count2_10_20/Count_10_20
    else:
        B2x_10_20=10+5
        B2y_10_20=0    

    if(Count_20_30!=0):
        B2x_20_30=RSum_20_30/Count_20_30
        B2y_20_30=Count2_20_30/Count_20_30
    else:
        B2x_20_30=20+5
        B2y_20_30=0

    if(Count_30_40!=0):
        B2x_30_40=RSum_30_40/Count_30_40
        B2y_30_40=Count2_30_40/Count_30_40
    else:
        B2x_30_40=30+5
        B2y_30_40=0

    if(Count_40_50!=0):
        B2x_40_50=RSum_40_50/Count_40_50
        B2y_40_50=Count2_40_50/Count_40_50
    else:
        B2x_40_50=40+5
        B2y_40_50=0

    if(Count_50_60!=0):
        B2x_50_60=RSum_50_60/Count_50_60
        B2y_50_60=Count2_50_60/Count_50_60
    else:
        B2x_50_60=50+5
        B2y_50_60=0

    if(Count_60_70!=0):
        B2x_60_70=RSum_60_70/Count_60_70
        B2y_60_70=Count2_60_70/Count_60_70
    else:
        B2x_60_70=60+5
        B2y_60_70=0

    if(Count_70_80!=0):
        B2x_70_80=RSum_70_80/Count_70_80
        B2y_70_80=Count2_70_80/Count_70_80
    else:
        B2x_70_80=70+5
        B2y_70_80=0    

    if(Count_80_90!=0):
        B2x_80_90=RSum_80_90/Count_80_90
        B2y_80_90=Count2_80_90/Count_80_90
    else:
        B2x_80_90=80+5
        B2y_80_90=0

    if(Count_90_100!=0):
        B2x_90_100=RSum_90_100/Count_90_100
        B2y_90_100=Count2_90_100/Count_90_100
    else:
        B2x_90_100=90+5
        B2y_90_100=0

    if(Count_100_120!=0):
        B2x_100_120=RSum_100_120/Count_100_120
        B2y_100_120=Count2_100_120/Count_100_120
    else:
        B2x_100_120=90+10
        B2y_100_120=0 

    if(Count_120_140!=0):
        B2x_120_140=RSum_120_140/Count_120_140
        B2y_120_140=Count2_120_140/Count_120_140
    else:
        B2x_120_140=120+10
        B2y_120_140=0

    if(Count_140_160!=0):
        B2x_140_160=RSum_140_160/Count_140_160
        B2y_140_160=Count2_140_160/Count_140_160
    else:
        B2x_140_160=140+10
        B2y_140_160=0

    if(Count_160_180!=0):
        B2x_160_180=RSum_160_180/Count_160_180
        B2y_160_180=Count2_160_180/Count_160_180
    else:
        B2x_160_180=160+10
        B2y_160_180=0

    if(Count_180_200!=0):
        B2x_180_200=RSum_180_200/Count_180_200
        B2y_180_200=Count2_180_200/Count_180_200
    else:
        B2x_180_200=180+10
        B2y_180_200=0

    # This part prepears the plot axies and labels
    B2xList=[B2x_0_10,B2x_10_20,B2x_20_30,B2x_30_40,B2x_40_50,B2x_50_60,B2x_60_70,B2x_70_80,B2x_80_90,B2x_90_100,
            B2x_100_120,B2x_120_140,B2x_140_160,B2x_160_180,B2x_180_200] # List of x for the covering factor plot

    B2yList=[B2y_0_10,B2y_10_20,B2y_20_30,B2y_30_40,B2y_40_50,B2y_50_60,B2y_60_70,B2y_70_80,B2y_80_90,B2y_90_100,
            B2y_100_120,B2y_120_140,B2y_140_160,B2y_160_180,B2y_180_200] # List of y for the covering factor plot

    #print("X values (the mean of R_perp in each bin) for the covering factor plot are: ",AxList)
    #print("Y values (the covering factor in each bin) for the covering factor plot are ",AyList)
    plt.scatter(B2xList,B2yList,color="black") # Plot the points in the covering factor plot       
    plt.xlim([0,200])
    plt.ylim([0.0,1.0])
    plt.ylabel(r"$C_f$"+" (LLS)",fontsize=12,fontname="serif")
    plt.xticks([25,50,75,100,125,150,175,200], [" "," "," "," "," "," "," "," "])

    # Next part will plot the horizontal and vertical lines in the covering factor graph for the x errors
    plt.hlines(B2y_0_10,xmin=0,xmax=10,color='black',linestyle='-')
    plt.vlines(10,ymin=B2y_0_10-0.1,ymax=B2y_0_10+0.1,color='black',linestyle='-')

    plt.hlines(B2y_10_20,xmin=10,xmax=20,color='black',linestyle='-')
    plt.vlines(10,ymin=B2y_10_20-0.1,ymax=B2y_10_20+0.1,color='black',linestyle='-')
    plt.vlines(20,ymin=B2y_10_20-0.1,ymax=B2y_10_20+0.1,color='black',linestyle='-')

    plt.hlines(B2y_20_30,xmin=20,xmax=30,color='black',linestyle='-')
    plt.vlines(20,ymin=B2y_20_30-0.1,ymax=B2y_20_30+0.1,color='black',linestyle='-')
    plt.vlines(30,ymin=B2y_20_30-0.1,ymax=B2y_20_30+0.1,color='black',linestyle='-')

    plt.hlines(B2y_30_40,xmin=30,xmax=40,color='black',linestyle='-')
    plt.vlines(30,ymin=B2y_30_40-0.1,ymax=B2y_30_40+0.1,color='black',linestyle='-')
    plt.vlines(40,ymin=B2y_30_40-0.1,ymax=B2y_30_40+0.1,color='black',linestyle='-')

    plt.hlines(B2y_40_50,xmin=40,xmax=50,color='black',linestyle='-')
    plt.vlines(40,ymin=B2y_40_50-0.1,ymax=B2y_40_50+0.1,color='black',linestyle='-')
    plt.vlines(50,ymin=B2y_40_50-0.1,ymax=B2y_40_50+0.1,color='black',linestyle='-')

    plt.hlines(B2y_50_60,xmin=50,xmax=60,color='black',linestyle='-')
    plt.vlines(50,ymin=B2y_50_60-0.1,ymax=B2y_50_60+0.1,color='black',linestyle='-')
    plt.vlines(60,ymin=B2y_50_60-0.1,ymax=B2y_50_60+0.1,color='black',linestyle='-')

    plt.hlines(B2y_60_70,xmin=60,xmax=70,color='black',linestyle='-')
    plt.vlines(60,ymin=B2y_60_70-0.1,ymax=B2y_60_70+0.1,color='black',linestyle='-')
    plt.vlines(70,ymin=B2y_60_70-0.1,ymax=B2y_60_70+0.1,color='black',linestyle='-')

    plt.hlines(B2y_70_80,xmin=70,xmax=80,color='black',linestyle='-')
    plt.vlines(70,ymin=B2y_70_80-0.1,ymax=B2y_70_80+0.1,color='black',linestyle='-')
    plt.vlines(80,ymin=B2y_70_80-0.1,ymax=B2y_70_80+0.1,color='black',linestyle='-')

    plt.hlines(B2y_80_90,xmin=80,xmax=90,color='black',linestyle='-')
    plt.vlines(80,ymin=B2y_80_90-0.1,ymax=B2y_80_90+0.1,color='black',linestyle='-')
    plt.vlines(90,ymin=B2y_80_90-0.1,ymax=B2y_80_90+0.1,color='black',linestyle='-')

    plt.hlines(B2y_90_100,xmin=90,xmax=100,color='black',linestyle='-')
    plt.vlines(90,ymin=B2y_90_100-0.1,ymax=B2y_90_100+0.1,color='black',linestyle='-')
    plt.vlines(100,ymin=B2y_90_100-0.1,ymax=B2y_90_100+0.1,color='black',linestyle='-')

    plt.hlines(B2y_100_120,xmin=100,xmax=120,color='black',linestyle='-')
    plt.vlines(100,ymin=B2y_100_120-0.1,ymax=B2y_100_120+0.1,color='black',linestyle='-')
    plt.vlines(120,ymin=B2y_100_120-0.1,ymax=B2y_100_120+0.1,color='black',linestyle='-')

    plt.hlines(B2y_120_140,xmin=120,xmax=140,color='black',linestyle='-')
    plt.vlines(120,ymin=B2y_120_140-0.1,ymax=B2y_120_140+0.1,color='black',linestyle='-')
    plt.vlines(140,ymin=B2y_120_140-0.1,ymax=B2y_120_140+0.1,color='black',linestyle='-')

    plt.hlines(B2y_140_160,xmin=140,xmax=160,color='black',linestyle='-')
    plt.vlines(140,ymin=B2y_140_160-0.1,ymax=B2y_140_160+0.1,color='black',linestyle='-')
    plt.vlines(160,ymin=B2y_140_160-0.1,ymax=B2y_140_160+0.1,color='black',linestyle='-')

    plt.hlines(B2y_160_180,xmin=160,xmax=180,color='black',linestyle='-')
    plt.vlines(160,ymin=B2y_160_180-0.1,ymax=B2y_160_180+0.1,color='black',linestyle='-')
    plt.vlines(180,ymin=B2y_160_180-0.1,ymax=B2y_160_180+0.1,color='black',linestyle='-')

    plt.hlines(B2y_180_200,xmin=180,xmax=200,color='black',linestyle='-')
    plt.vlines(180,ymin=B2y_180_200-0.1,ymax=B2y_180_200+0.1,color='black',linestyle='-')
    plt.vlines(200,ymin=B2y_180_200-0.1,ymax=B2y_180_200+0.1,color='black',linestyle='-')

    # Next part will handle the main graph of N_HI
    ax1 = fig.add_subplot(ax[2:,0:3]) # Plot for the main graph
    plt.scatter(ListOfRBetweenSightlineAndTheCenter,ListOfN_HIOfTheSightline) # Plot the N_HI against R⊥
    plt.xlabel(r"$R_⊥$"+" [kpc]"+" ("+r"$R_\perp$"+" from center)",fontsize=12,fontname="serif")
    plt.ylabel("log "+r"$N_{\rm HI}$"+" "+"[cm"+r"$^{-2}$"+"]",fontsize=12,fontname="serif")
    plt.autoscale(enable=True,axis='y',tight=True) # Allows the axhspan to go up till the edge
    plt.axhspan(20.3,23,color='gray',alpha=0.5,label="DLA") # Mark the area of the DLAs
    plt.axhspan(17.2,20.3,color='lightgray',alpha=0.5,label="LLS") # Mark the area of the LLSs
    lightgray_patch=mpatches.Patch(color='lightgray',label='LLS') # Creates patch for the LLS
    gray_patch=mpatches.Patch(color='gray',label='DLA') # Creates patch for the DLA
    plt.legend(handles=[gray_patch,lightgray_patch],loc='lower right')
    plt.xlim([0,200])
    plt.ylim([13.5,21.5])
    #plt.savefig('/home1/08289/tg875885/radial_to_rotating_flows/Aharon/Outputs2/'+SnapDirect+"~log(N_HI)[R⊥]~FromCenter.png")
    plt.show()
    plt.clf()
    plt.cla()
    plt.close()
    print("DONE 3")
    ######################################################################################################################################
    ######################################################################################################################################
    # This block plot median and 16th, 18th percentile for N_HI agains R_perp for both types of R_perp and C_f factor above it
    plt.clf()
    # Special graph colors
    rperp_dla_color1=palettable.cartocolors.qualitative.Safe_2.mpl_colors[0]
    rperp_dla_color2=palettable.cartocolors.qualitative.Safe_2.mpl_colors[1]
    fig=plt.figure(constrained_layout=True) # Creates the figure
    ax=fig.add_gridspec(5,3) # Creates the figure structure
    ########################################################################################################################################
    # C_f (of DLA) Graph for both R_perp
    ax2=fig.add_subplot(ax[0,0:3]) # Grid for the covering factor plot
    plt.xlim([0,200])
    plt.ylim([0.0,1.0])
    plt.ylabel(r"$C_f$"+" (DLA)",fontsize=12,fontname="serif")
    plt.xticks([25,50,75,100,125,150,175,200], [" "," "," "," "," "," "," "," "])

    # For A1- R_perp from DLA
    plt.scatter(A1xList,A1yList,color=rperp_dla_color1) # Plot the points in the covering factor plot     
    # Next part will plot the horizontal and vertical lines in the covering factor graph for the x errors
    plt.hlines(A1y_0_10,xmin=0,xmax=10,color=rperp_dla_color1,linestyle='-')
    plt.vlines(10,ymin=A1y_0_10-0.1,ymax=A1y_0_10+0.1,color=rperp_dla_color1,linestyle='-')

    plt.hlines(A1y_10_20,xmin=10,xmax=20,color=rperp_dla_color1,linestyle='-')
    plt.vlines(10,ymin=A1y_10_20-0.1,ymax=A1y_10_20+0.1,color=rperp_dla_color1,linestyle='-')
    plt.vlines(20,ymin=A1y_10_20-0.1,ymax=A1y_10_20+0.1,color=rperp_dla_color1,linestyle='-')

    plt.hlines(A1y_20_30,xmin=20,xmax=30,color=rperp_dla_color1,linestyle='-')
    plt.vlines(20,ymin=A1y_20_30-0.1,ymax=A1y_20_30+0.1,color=rperp_dla_color1,linestyle='-')
    plt.vlines(30,ymin=A1y_20_30-0.1,ymax=A1y_20_30+0.1,color=rperp_dla_color1,linestyle='-')

    plt.hlines(A1y_30_40,xmin=30,xmax=40,color=rperp_dla_color1,linestyle='-')
    plt.vlines(30,ymin=A1y_30_40-0.1,ymax=A1y_30_40+0.1,color=rperp_dla_color1,linestyle='-')
    plt.vlines(40,ymin=A1y_30_40-0.1,ymax=A1y_30_40+0.1,color=rperp_dla_color1,linestyle='-')

    plt.hlines(A1y_40_50,xmin=40,xmax=50,color=rperp_dla_color1,linestyle='-')
    plt.vlines(40,ymin=A1y_40_50-0.1,ymax=A1y_40_50+0.1,color=rperp_dla_color1,linestyle='-')
    plt.vlines(50,ymin=A1y_40_50-0.1,ymax=A1y_40_50+0.1,color=rperp_dla_color1,linestyle='-')

    plt.hlines(A1y_50_60,xmin=50,xmax=60,color=rperp_dla_color1,linestyle='-')
    plt.vlines(50,ymin=A1y_50_60-0.1,ymax=A1y_50_60+0.1,color=rperp_dla_color1,linestyle='-')
    plt.vlines(60,ymin=A1y_50_60-0.1,ymax=A1y_50_60+0.1,color=rperp_dla_color1,linestyle='-')

    plt.hlines(A1y_60_70,xmin=60,xmax=70,color=rperp_dla_color1,linestyle='-')
    plt.vlines(60,ymin=A1y_60_70-0.1,ymax=A1y_60_70+0.1,color=rperp_dla_color1,linestyle='-')
    plt.vlines(70,ymin=A1y_60_70-0.1,ymax=A1y_60_70+0.1,color=rperp_dla_color1,linestyle='-')

    plt.hlines(A1y_70_80,xmin=70,xmax=80,color=rperp_dla_color1,linestyle='-')
    plt.vlines(70,ymin=A1y_70_80-0.1,ymax=A1y_70_80+0.1,color=rperp_dla_color1,linestyle='-')
    plt.vlines(80,ymin=A1y_70_80-0.1,ymax=A1y_70_80+0.1,color=rperp_dla_color1,linestyle='-')

    plt.hlines(A1y_80_90,xmin=80,xmax=90,color=rperp_dla_color1,linestyle='-')
    plt.vlines(80,ymin=A1y_80_90-0.1,ymax=A1y_80_90+0.1,color=rperp_dla_color1,linestyle='-')
    plt.vlines(90,ymin=A1y_80_90-0.1,ymax=A1y_80_90+0.1,color=rperp_dla_color1,linestyle='-')

    plt.hlines(A1y_90_100,xmin=90,xmax=100,color=rperp_dla_color1,linestyle='-')
    plt.vlines(90,ymin=A1y_90_100-0.1,ymax=A1y_90_100+0.1,color=rperp_dla_color1,linestyle='-')
    plt.vlines(100,ymin=A1y_90_100-0.1,ymax=A1y_90_100+0.1,color=rperp_dla_color1,linestyle='-')

    plt.hlines(A1y_100_120,xmin=100,xmax=120,color=rperp_dla_color1,linestyle='-')
    plt.vlines(100,ymin=A1y_100_120-0.1,ymax=A1y_100_120+0.1,color=rperp_dla_color1,linestyle='-')
    plt.vlines(120,ymin=A1y_100_120-0.1,ymax=A1y_100_120+0.1,color=rperp_dla_color1,linestyle='-')

    plt.hlines(A1y_120_140,xmin=120,xmax=140,color=rperp_dla_color1,linestyle='-')
    plt.vlines(120,ymin=A1y_120_140-0.1,ymax=A1y_120_140+0.1,color=rperp_dla_color1,linestyle='-')
    plt.vlines(140,ymin=A1y_120_140-0.1,ymax=A1y_120_140+0.1,color=rperp_dla_color1,linestyle='-')

    plt.hlines(A1y_140_160,xmin=140,xmax=160,color=rperp_dla_color1,linestyle='-')
    plt.vlines(140,ymin=A1y_140_160-0.1,ymax=A1y_140_160+0.1,color=rperp_dla_color1,linestyle='-')
    plt.vlines(160,ymin=A1y_140_160-0.1,ymax=A1y_140_160+0.1,color=rperp_dla_color1,linestyle='-')

    plt.hlines(A1y_160_180,xmin=160,xmax=180,color=rperp_dla_color1,linestyle='-')
    plt.vlines(160,ymin=A1y_160_180-0.1,ymax=A1y_160_180+0.1,color=rperp_dla_color1,linestyle='-')
    plt.vlines(180,ymin=A1y_160_180-0.1,ymax=A1y_160_180+0.1,color=rperp_dla_color1,linestyle='-')

    plt.hlines(A1y_180_200,xmin=180,xmax=200,color=rperp_dla_color1,linestyle='-')
    plt.vlines(180,ymin=A1y_180_200-0.1,ymax=A1y_180_200+0.1,color=rperp_dla_color1,linestyle='-')
    plt.vlines(200,ymin=A1y_180_200-0.1,ymax=A1y_180_200+0.1,color=rperp_dla_color1,linestyle='-')

    # For B1- R_perp from center
    plt.scatter(B1xList,B1yList,color=rperp_dla_color2) # Plot the points in the covering factor plot 
    # Next part will plot the horizontal and vertical lines in the covering factor graph for the x errors
    plt.hlines(B1y_0_10,xmin=0,xmax=10,color=rperp_dla_color2,linestyle='-')
    plt.vlines(10,ymin=B1y_0_10-0.1,ymax=B1y_0_10+0.1,color=rperp_dla_color2,linestyle='-')

    plt.hlines(B1y_10_20,xmin=10,xmax=20,color=rperp_dla_color2,linestyle='-')
    plt.vlines(10,ymin=B1y_10_20-0.1,ymax=B1y_10_20+0.1,color=rperp_dla_color2,linestyle='-')
    plt.vlines(20,ymin=B1y_10_20-0.1,ymax=B1y_10_20+0.1,color=rperp_dla_color2,linestyle='-')

    plt.hlines(B1y_20_30,xmin=20,xmax=30,color=rperp_dla_color2,linestyle='-')
    plt.vlines(20,ymin=B1y_20_30-0.1,ymax=B1y_20_30+0.1,color=rperp_dla_color2,linestyle='-')
    plt.vlines(30,ymin=B1y_20_30-0.1,ymax=B1y_20_30+0.1,color=rperp_dla_color2,linestyle='-')

    plt.hlines(B1y_30_40,xmin=30,xmax=40,color=rperp_dla_color2,linestyle='-')
    plt.vlines(30,ymin=B1y_30_40-0.1,ymax=B1y_30_40+0.1,color=rperp_dla_color2,linestyle='-')
    plt.vlines(40,ymin=B1y_30_40-0.1,ymax=B1y_30_40+0.1,color=rperp_dla_color2,linestyle='-')

    plt.hlines(B1y_40_50,xmin=40,xmax=50,color=rperp_dla_color2,linestyle='-')
    plt.vlines(40,ymin=B1y_40_50-0.1,ymax=B1y_40_50+0.1,color=rperp_dla_color2,linestyle='-')
    plt.vlines(50,ymin=B1y_40_50-0.1,ymax=B1y_40_50+0.1,color=rperp_dla_color2,linestyle='-')

    plt.hlines(B1y_50_60,xmin=50,xmax=60,color=rperp_dla_color2,linestyle='-')
    plt.vlines(50,ymin=B1y_50_60-0.1,ymax=B1y_50_60+0.1,color=rperp_dla_color2,linestyle='-')
    plt.vlines(60,ymin=B1y_50_60-0.1,ymax=B1y_50_60+0.1,color=rperp_dla_color2,linestyle='-')

    plt.hlines(B1y_60_70,xmin=60,xmax=70,color=rperp_dla_color2,linestyle='-')
    plt.vlines(60,ymin=B1y_60_70-0.1,ymax=B1y_60_70+0.1,color=rperp_dla_color2,linestyle='-')
    plt.vlines(70,ymin=B1y_60_70-0.1,ymax=B1y_60_70+0.1,color=rperp_dla_color2,linestyle='-')

    plt.hlines(B1y_70_80,xmin=70,xmax=80,color=rperp_dla_color2,linestyle='-')
    plt.vlines(70,ymin=B1y_70_80-0.1,ymax=B1y_70_80+0.1,color=rperp_dla_color2,linestyle='-')
    plt.vlines(80,ymin=B1y_70_80-0.1,ymax=B1y_70_80+0.1,color=rperp_dla_color2,linestyle='-')

    plt.hlines(B1y_80_90,xmin=80,xmax=90,color=rperp_dla_color2,linestyle='-')
    plt.vlines(80,ymin=B1y_80_90-0.1,ymax=B1y_80_90+0.1,color=rperp_dla_color2,linestyle='-')
    plt.vlines(90,ymin=B1y_80_90-0.1,ymax=B1y_80_90+0.1,color=rperp_dla_color2,linestyle='-')

    plt.hlines(B1y_90_100,xmin=90,xmax=100,color=rperp_dla_color2,linestyle='-')
    plt.vlines(90,ymin=B1y_90_100-0.1,ymax=B1y_90_100+0.1,color=rperp_dla_color2,linestyle='-')
    plt.vlines(100,ymin=B1y_90_100-0.1,ymax=B1y_90_100+0.1,color=rperp_dla_color2,linestyle='-')

    plt.hlines(B1y_100_120,xmin=100,xmax=120,color=rperp_dla_color2,linestyle='-')
    plt.vlines(100,ymin=B1y_100_120-0.1,ymax=B1y_100_120+0.1,color=rperp_dla_color2,linestyle='-')
    plt.vlines(120,ymin=B1y_100_120-0.1,ymax=B1y_100_120+0.1,color=rperp_dla_color2,linestyle='-')

    plt.hlines(B1y_120_140,xmin=120,xmax=140,color=rperp_dla_color2,linestyle='-')
    plt.vlines(120,ymin=B1y_120_140-0.1,ymax=B1y_120_140+0.1,color=rperp_dla_color2,linestyle='-')
    plt.vlines(140,ymin=B1y_120_140-0.1,ymax=B1y_120_140+0.1,color=rperp_dla_color2,linestyle='-')

    plt.hlines(B1y_140_160,xmin=140,xmax=160,color=rperp_dla_color2,linestyle='-')
    plt.vlines(140,ymin=B1y_140_160-0.1,ymax=B1y_140_160+0.1,color=rperp_dla_color2,linestyle='-')
    plt.vlines(160,ymin=B1y_140_160-0.1,ymax=B1y_140_160+0.1,color=rperp_dla_color2,linestyle='-')

    plt.hlines(B1y_160_180,xmin=160,xmax=180,color=rperp_dla_color2,linestyle='-')
    plt.vlines(160,ymin=B1y_160_180-0.1,ymax=B1y_160_180+0.1,color=rperp_dla_color2,linestyle='-')
    plt.vlines(180,ymin=B1y_160_180-0.1,ymax=B1y_160_180+0.1,color=rperp_dla_color2,linestyle='-')

    plt.hlines(B1y_180_200,xmin=180,xmax=200,color=rperp_dla_color2,linestyle='-')
    plt.vlines(180,ymin=B1y_180_200-0.1,ymax=B1y_180_200+0.1,color=rperp_dla_color2,linestyle='-')
    plt.vlines(200,ymin=B1y_180_200-0.1,ymax=B1y_180_200+0.1,color=rperp_dla_color2,linestyle='-')
    ########################################################################################################################################
    # C_f (of LLS) Graph for both R_perp
    ax1=fig.add_subplot(ax[1,0:3]) # Grid for the covering factor plot
    plt.xlim([0,200])
    plt.ylim([0.0,1.0])
    plt.ylabel(r"$C_f$"+" (LLS)",fontsize=12,fontname="serif")
    plt.xticks([25,50,75,100,125,150,175,200], [" "," "," "," "," "," "," "," "])

    # For A2- R_perp from DLA
    plt.scatter(A2xList,A2yList,color=rperp_dla_color1) # Plot the points in the covering factor plot       
    # Next part will plot the horizontal and vertical lines in the covering factor graph for the x errors
    plt.hlines(A2y_0_10,xmin=0,xmax=10,color=rperp_dla_color1,linestyle='-')
    plt.vlines(10,ymin=A2y_0_10-0.1,ymax=A2y_0_10+0.1,color=rperp_dla_color1,linestyle='-')

    plt.hlines(A2y_10_20,xmin=10,xmax=20,color=rperp_dla_color1,linestyle='-')
    plt.vlines(10,ymin=A2y_10_20-0.1,ymax=A2y_10_20+0.1,color=rperp_dla_color1,linestyle='-')
    plt.vlines(20,ymin=A2y_10_20-0.1,ymax=A2y_10_20+0.1,color=rperp_dla_color1,linestyle='-')

    plt.hlines(A2y_20_30,xmin=20,xmax=30,color=rperp_dla_color1,linestyle='-')
    plt.vlines(20,ymin=A2y_20_30-0.1,ymax=A2y_20_30+0.1,color=rperp_dla_color1,linestyle='-')
    plt.vlines(30,ymin=A2y_20_30-0.1,ymax=A2y_20_30+0.1,color=rperp_dla_color1,linestyle='-')

    plt.hlines(A2y_30_40,xmin=30,xmax=40,color=rperp_dla_color1,linestyle='-')
    plt.vlines(30,ymin=A2y_30_40-0.1,ymax=A2y_30_40+0.1,color=rperp_dla_color1,linestyle='-')
    plt.vlines(40,ymin=A2y_30_40-0.1,ymax=A2y_30_40+0.1,color=rperp_dla_color1,linestyle='-')

    plt.hlines(A2y_40_50,xmin=40,xmax=50,color=rperp_dla_color1,linestyle='-')
    plt.vlines(40,ymin=A2y_40_50-0.1,ymax=A2y_40_50+0.1,color=rperp_dla_color1,linestyle='-')
    plt.vlines(50,ymin=A2y_40_50-0.1,ymax=A2y_40_50+0.1,color=rperp_dla_color1,linestyle='-')

    plt.hlines(A2y_50_60,xmin=50,xmax=60,color=rperp_dla_color1,linestyle='-')
    plt.vlines(50,ymin=A2y_50_60-0.1,ymax=A2y_50_60+0.1,color=rperp_dla_color1,linestyle='-')
    plt.vlines(60,ymin=A2y_50_60-0.1,ymax=A2y_50_60+0.1,color=rperp_dla_color1,linestyle='-')

    plt.hlines(A2y_60_70,xmin=60,xmax=70,color=rperp_dla_color1,linestyle='-')
    plt.vlines(60,ymin=A2y_60_70-0.1,ymax=A2y_60_70+0.1,color=rperp_dla_color1,linestyle='-')
    plt.vlines(70,ymin=A2y_60_70-0.1,ymax=A2y_60_70+0.1,color=rperp_dla_color1,linestyle='-')

    plt.hlines(A2y_70_80,xmin=70,xmax=80,color=rperp_dla_color1,linestyle='-')
    plt.vlines(70,ymin=A2y_70_80-0.1,ymax=A2y_70_80+0.1,color=rperp_dla_color1,linestyle='-')
    plt.vlines(80,ymin=A2y_70_80-0.1,ymax=A2y_70_80+0.1,color=rperp_dla_color1,linestyle='-')

    plt.hlines(A2y_80_90,xmin=80,xmax=90,color=rperp_dla_color1,linestyle='-')
    plt.vlines(80,ymin=A2y_80_90-0.1,ymax=A2y_80_90+0.1,color=rperp_dla_color1,linestyle='-')
    plt.vlines(90,ymin=A2y_80_90-0.1,ymax=A2y_80_90+0.1,color=rperp_dla_color1,linestyle='-')

    plt.hlines(A2y_90_100,xmin=90,xmax=100,color=rperp_dla_color1,linestyle='-')
    plt.vlines(90,ymin=A2y_90_100-0.1,ymax=A2y_90_100+0.1,color=rperp_dla_color1,linestyle='-')
    plt.vlines(100,ymin=A2y_90_100-0.1,ymax=A2y_90_100+0.1,color=rperp_dla_color1,linestyle='-')

    plt.hlines(A2y_100_120,xmin=100,xmax=120,color=rperp_dla_color1,linestyle='-')
    plt.vlines(100,ymin=A2y_100_120-0.1,ymax=A2y_100_120+0.1,color=rperp_dla_color1,linestyle='-')
    plt.vlines(120,ymin=A2y_100_120-0.1,ymax=A2y_100_120+0.1,color=rperp_dla_color1,linestyle='-')

    plt.hlines(A2y_120_140,xmin=120,xmax=140,color=rperp_dla_color1,linestyle='-')
    plt.vlines(120,ymin=A2y_120_140-0.1,ymax=A2y_120_140+0.1,color=rperp_dla_color1,linestyle='-')
    plt.vlines(140,ymin=A2y_120_140-0.1,ymax=A2y_120_140+0.1,color=rperp_dla_color1,linestyle='-')

    plt.hlines(A2y_140_160,xmin=140,xmax=160,color=rperp_dla_color1,linestyle='-')
    plt.vlines(140,ymin=A2y_140_160-0.1,ymax=A2y_140_160+0.1,color=rperp_dla_color1,linestyle='-')
    plt.vlines(160,ymin=A2y_140_160-0.1,ymax=A2y_140_160+0.1,color=rperp_dla_color1,linestyle='-')

    plt.hlines(A2y_160_180,xmin=160,xmax=180,color=rperp_dla_color1,linestyle='-')
    plt.vlines(160,ymin=A2y_160_180-0.1,ymax=A2y_160_180+0.1,color=rperp_dla_color1,linestyle='-')
    plt.vlines(180,ymin=A2y_160_180-0.1,ymax=A2y_160_180+0.1,color=rperp_dla_color1,linestyle='-')

    # For B2- R_perp from center
    plt.scatter(B2xList,B2yList,color=rperp_dla_color2) # Plot the points in the covering factor plot       
    # Next part will plot the horizontal and vertical lines in the covering factor graph for the x errors
    plt.hlines(B2y_0_10,xmin=0,xmax=10,color=rperp_dla_color2,linestyle='-')
    plt.vlines(10,ymin=B2y_0_10-0.1,ymax=B2y_0_10+0.1,color=rperp_dla_color2,linestyle='-')

    plt.hlines(B2y_10_20,xmin=10,xmax=20,color=rperp_dla_color2,linestyle='-')
    plt.vlines(10,ymin=B2y_10_20-0.1,ymax=B2y_10_20+0.1,color=rperp_dla_color2,linestyle='-')
    plt.vlines(20,ymin=B2y_10_20-0.1,ymax=B2y_10_20+0.1,color=rperp_dla_color2,linestyle='-')

    plt.hlines(B2y_20_30,xmin=20,xmax=30,color=rperp_dla_color2,linestyle='-')
    plt.vlines(20,ymin=B2y_20_30-0.1,ymax=B2y_20_30+0.1,color=rperp_dla_color2,linestyle='-')
    plt.vlines(30,ymin=B2y_20_30-0.1,ymax=B2y_20_30+0.1,color=rperp_dla_color2,linestyle='-')

    plt.hlines(B2y_30_40,xmin=30,xmax=40,color=rperp_dla_color2,linestyle='-')
    plt.vlines(30,ymin=B2y_30_40-0.1,ymax=B2y_30_40+0.1,color=rperp_dla_color2,linestyle='-')
    plt.vlines(40,ymin=B2y_30_40-0.1,ymax=B2y_30_40+0.1,color=rperp_dla_color2,linestyle='-')

    plt.hlines(B2y_40_50,xmin=40,xmax=50,color=rperp_dla_color2,linestyle='-')
    plt.vlines(40,ymin=B2y_40_50-0.1,ymax=B2y_40_50+0.1,color=rperp_dla_color2,linestyle='-')
    plt.vlines(50,ymin=B2y_40_50-0.1,ymax=B2y_40_50+0.1,color=rperp_dla_color2,linestyle='-')

    plt.hlines(B2y_50_60,xmin=50,xmax=60,color=rperp_dla_color2,linestyle='-')
    plt.vlines(50,ymin=B2y_50_60-0.1,ymax=B2y_50_60+0.1,color=rperp_dla_color2,linestyle='-')
    plt.vlines(60,ymin=B2y_50_60-0.1,ymax=B2y_50_60+0.1,color=rperp_dla_color2,linestyle='-')

    plt.hlines(B2y_60_70,xmin=60,xmax=70,color=rperp_dla_color2,linestyle='-')
    plt.vlines(60,ymin=B2y_60_70-0.1,ymax=B2y_60_70+0.1,color=rperp_dla_color2,linestyle='-')
    plt.vlines(70,ymin=B2y_60_70-0.1,ymax=B2y_60_70+0.1,color=rperp_dla_color2,linestyle='-')

    plt.hlines(B2y_70_80,xmin=70,xmax=80,color=rperp_dla_color2,linestyle='-')
    plt.vlines(70,ymin=B2y_70_80-0.1,ymax=B2y_70_80+0.1,color=rperp_dla_color2,linestyle='-')
    plt.vlines(80,ymin=B2y_70_80-0.1,ymax=B2y_70_80+0.1,color=rperp_dla_color2,linestyle='-')

    plt.hlines(B2y_80_90,xmin=80,xmax=90,color=rperp_dla_color2,linestyle='-')
    plt.vlines(80,ymin=B2y_80_90-0.1,ymax=B2y_80_90+0.1,color=rperp_dla_color2,linestyle='-')
    plt.vlines(90,ymin=B2y_80_90-0.1,ymax=B2y_80_90+0.1,color=rperp_dla_color2,linestyle='-')

    plt.hlines(B2y_90_100,xmin=90,xmax=100,color=rperp_dla_color2,linestyle='-')
    plt.vlines(90,ymin=B2y_90_100-0.1,ymax=B2y_90_100+0.1,color=rperp_dla_color2,linestyle='-')
    plt.vlines(100,ymin=B2y_90_100-0.1,ymax=B2y_90_100+0.1,color=rperp_dla_color2,linestyle='-')

    plt.hlines(B2y_100_120,xmin=100,xmax=120,color=rperp_dla_color2,linestyle='-')
    plt.vlines(100,ymin=B2y_100_120-0.1,ymax=B2y_100_120+0.1,color=rperp_dla_color2,linestyle='-')
    plt.vlines(120,ymin=B2y_100_120-0.1,ymax=B2y_100_120+0.1,color=rperp_dla_color2,linestyle='-')

    plt.hlines(B2y_120_140,xmin=120,xmax=140,color=rperp_dla_color2,linestyle='-')
    plt.vlines(120,ymin=B2y_120_140-0.1,ymax=B2y_120_140+0.1,color=rperp_dla_color2,linestyle='-')
    plt.vlines(140,ymin=B2y_120_140-0.1,ymax=B2y_120_140+0.1,color=rperp_dla_color2,linestyle='-')

    plt.hlines(B2y_140_160,xmin=140,xmax=160,color=rperp_dla_color2,linestyle='-')
    plt.vlines(140,ymin=B2y_140_160-0.1,ymax=B2y_140_160+0.1,color=rperp_dla_color2,linestyle='-')
    plt.vlines(160,ymin=B2y_140_160-0.1,ymax=B2y_140_160+0.1,color=rperp_dla_color2,linestyle='-')

    plt.hlines(B2y_160_180,xmin=160,xmax=180,color=rperp_dla_color2,linestyle='-')
    plt.vlines(160,ymin=B2y_160_180-0.1,ymax=B2y_160_180+0.1,color=rperp_dla_color2,linestyle='-')
    plt.vlines(180,ymin=B2y_160_180-0.1,ymax=B2y_160_180+0.1,color=rperp_dla_color2,linestyle='-')

    plt.hlines(B2y_180_200,xmin=180,xmax=200,color=rperp_dla_color2,linestyle='-')
    plt.vlines(180,ymin=B2y_180_200-0.1,ymax=B2y_180_200+0.1,color=rperp_dla_color2,linestyle='-')
    plt.vlines(200,ymin=B2y_180_200-0.1,ymax=B2y_180_200+0.1,color=rperp_dla_color2,linestyle='-')

    # Next part will handle the main graph of Median, 16th, 18th
    ax1 = fig.add_subplot(ax[2:,0:3]) # Plot for the main graph
    low_interval=lambda x: np.nanpercentile(x,16,axis=0)
    high_interval=lambda x: np.nanpercentile(x,84,axis=0)

    Abinsmin=np.min(ListOfRBetweenTwoSightLines)
    Abinsmax=np.max(ListOfRBetweenTwoSightLines)
    Bbinsmin=np.min(ListOfRBetweenSightlineAndTheCenter)
    Bbinsmax=np.max(ListOfRBetweenSightlineAndTheCenter)

    binsmin=np.min([Abinsmin,Bbinsmin])
    binsmax=np.max([Abinsmax,Bbinsmax])
    #bins=np.linspace(binsmin,binsmax,12)
    bins=np.array([5,15,25,35,45,55,65,75,85,95,110,130,150,170,190,210])

    #Abins=np.linspace(np.min(ListOfRBetweenTwoSightLines),np.max(ListOfRBetweenTwoSightLines),12)
    Abin_means,Abin_edges,Abinnumber=scipy.stats.binned_statistic(ListOfRBetweenTwoSightLines,ListOfN_HIOfTheAnotherLine,statistic='mean',bins=bins)
    Abin_means2,Abin_edges2,Abinnumber2=scipy.stats.binned_statistic(ListOfRBetweenTwoSightLines,ListOfN_HIOfTheAnotherLine,statistic=low_interval,bins=bins)
    Abin_means3,Abin_edges3,Abinnumber3=scipy.stats.binned_statistic(ListOfRBetweenTwoSightLines,ListOfN_HIOfTheAnotherLine,statistic=high_interval,bins=bins)
    print("Abin_means shape is: ",Abin_means.shape)
    print("Abin_means list is: ",Abin_means)
    print("Abin_edges shape is: ",Abin_edges.shape)
    print("Abin_edges list is: ",Abin_edges)
    plt.plot(Abin_edges[0:15],Abin_means,color=rperp_dla_color1,label=r"$R_\perp$"+" from DLA",linewidth=3)
    plt.fill_between(Abin_edges[0:15],Abin_means2,Abin_means3,color=rperp_dla_color1,alpha=0.5)
    Amax=np.max(Abin_edges[0:15])

    #Bbins=np.linspace(np.min(ListOfRBetweenSightlineAndTheCenter),np.max(ListOfRBetweenSightlineAndTheCenter),12)
    Bbin_means,Bbin_edges,Bbinnumber=scipy.stats.binned_statistic(ListOfRBetweenSightlineAndTheCenter,ListOfN_HIOfTheSightline,statistic='mean',bins=bins)
    Bbin_means2,Bbin_edges2,Bbinnumber2=scipy.stats.binned_statistic(ListOfRBetweenSightlineAndTheCenter,ListOfN_HIOfTheSightline,statistic=low_interval,bins=bins)
    Bbin_means3,Bbin_edges3,Bbinnumber3=scipy.stats.binned_statistic(ListOfRBetweenSightlineAndTheCenter,ListOfN_HIOfTheSightline,statistic=high_interval,bins=bins)
    print("Bbin_means shape is: ",Bbin_means.shape)
    print("Bbin_means list is: ",Bbin_means)
    print("Bbin_edges shape is: ",Bbin_edges.shape)
    print("Bbin_edges list is: ",Bbin_edges)
    plt.plot(Bbin_edges[0:15],Bbin_means,color=rperp_dla_color2,label=r"$R_\perp$"+" from center",linewidth=3)
    plt.fill_between(Bbin_edges[0:15],Bbin_means2,Bbin_means3,color=rperp_dla_color2,alpha=0.25)
    Bmax=np.max(Bbin_edges[0:15])

    xmax=np.max([Amax,Bmax])
    plt.xlim([0,200])
    plt.xlabel(r"$R_⊥$"+" [kpc]",fontsize=12,fontname="serif")
    plt.ylabel(r"$\log N_{\rm HI}$"+" "+"[cm"+r"$^{-2}$"+"]",fontsize=12,fontname="serif")
    plt.hlines(20.3,xmin=0,xmax=200,color='red',linestyle='--',label="DLA")
    plt.hlines(17.2,xmin=0,xmax=200,color='black',linestyle='--',label="LLS")
    plt.legend(loc='upper right')
    plt.ylim([13.5,21.5])
    plt.savefig('/home1/08289/tg875885/radial_to_rotating_flows/Aharon/Outputs2/'+SnapDirect+"~log(N_HI)[R⊥]~MedianOfBothRperp")
    print("DONE 4")
    
    # Prepare text file with all plots data (will be useful in the analysis)
    import pandas as pd
    MainGraph=pd.DataFrame({'X values of R perp from DLA':ListOfRBetweenTwoSightLines,
                        'Y values of R perp from DLA':ListOfN_HIOfTheAnotherLine,
                        'X values of R perp from Center':ListOfRBetweenSightlineAndTheCenter,
                        'Y values of R perp from Center':ListOfN_HIOfTheSightline})
    DLACoveringFactor=pd.DataFrame({'Cf(DLA) X values of R perp from DLA':A1xList,
                                'Cf(DLA) Y values of R perp from DLA':A1yList,
                                'Cf(DLA) X values of R perp from Center':B1xList,
                                'Cf(DLA) Y values of R perp from Center':B1yList})
    LLSCoveringFactor=pd.DataFrame({'Cf(LLS) X values of R perp from DLA':A2xList,
                                'Cf(LLS) Y values of R perp from DLA':A2yList,
                                'Cf(LLS) X values of R perp from Center':B2xList,
                                'Cf(LLS) Y values of R perp from Center':B2yList})
    MoreData=pd.DataFrame({'Abin_edges[0:5]':Abin_edges[0:15],
                       'Abin_means':Abin_means,
                       'Abin_means2':Abin_means2,
                       'Abin_means3':Abin_means3,
                       'Bbin_edges[0:15]':Bbin_edges[0:15],
                       'Bbin_means':Bbin_means,
                       'Bbin_means2':Bbin_means2,
                       'Bbin_means3':Bbin_means3})
    np.savetxt(r'/home1/08289/tg875885/radial_to_rotating_flows/Aharon/Outputs2/'+SnapDirect+'_MainGraph.txt',MainGraph,header="X values of R perp from DLA,Y values of R perp from DLA,X values of R perp from Center,Y values of R perp from Center")
    np.savetxt(r'/home1/08289/tg875885/radial_to_rotating_flows/Aharon/Outputs2/'+SnapDirect+'_Cf(DLA).txt',DLACoveringFactor,header="X values of R perp from DLA,Y values of R perp from DLA,X values of R perp from Center,Y values of R perp from Center")
    np.savetxt(r'/home1/08289/tg875885/radial_to_rotating_flows/Aharon/Outputs2/'+SnapDirect+'_Cf(LLS).txt',LLSCoveringFactor,header="X values of R perp from DLA,Y values of R perp from DLA,X values of R perp from Center,Y values of R perp from Center")
    np.savetxt(r'/home1/08289/tg875885/radial_to_rotating_flows/Aharon/Outputs2/'+SnapDirect+'_MoreData.txt',MoreData,header="Abin_edges[0:15],Abin_means,Abin_means2,Abin_means3,Bbin_edges[0:15],Bbin_means,Bbin_means2,Bbin_means3")

In [ ]:
# Prepare text file with all plots data (will be useful in the analysis)
import pandas as pd
MainGraph=pd.DataFrame({'X values of R perp from DLA':ListOfRBetweenTwoSightLines,
                        'Y values of R perp from DLA':ListOfN_HIOfTheAnotherLine,
                        'X values of R perp from Center':ListOfRBetweenSightlineAndTheCenter,
                        'Y values of R perp from Center':ListOfN_HIOfTheSightline})
DLACoveringFactor=pd.DataFrame({'Cf(DLA) X values of R perp from DLA':A1xList,
                                'Cf(DLA) Y values of R perp from DLA':A1yList,
                                'Cf(DLA) X values of R perp from Center':B1xList,
                                'Cf(DLA) Y values of R perp from Center':B1yList})
LLSCoveringFactor=pd.DataFrame({'Cf(LLS) X values of R perp from DLA':A2xList,
                                'Cf(LLS) Y values of R perp from DLA':A2yList,
                                'Cf(LLS) X values of R perp from Center':B2xList,
                                'Cf(LLS) Y values of R perp from Center':B2yList})
MoreData=pd.DataFrame({'Abin_edges[0:15]':Abin_edges[0:15],
                       'Abin_means':Abin_means,
                       'Abin_means2':Abin_means2,
                       'Abin_means3':Abin_means3,
                       'Bbin_edges[0:15]':Bbin_edges[0:15],
                       'Bbin_means':Bbin_means,
                       'Bbin_means2':Bbin_means2,
                       'Bbin_means3':Bbin_means3})
np.savetxt(r'/home1/08289/tg875885/radial_to_rotating_flows/Aharon/Outputs2/'+SnapDirect+'_MainGraph.txt',MainGraph,header="X values of R perp from DLA,Y values of R perp from DLA,X values of R perp from Center,Y values of R perp from Center")
np.savetxt(r'/home1/08289/tg875885/radial_to_rotating_flows/Aharon/Outputs2/'+SnapDirect+'_Cf(DLA).txt',DLACoveringFactor,header="X values of R perp from DLA,Y values of R perp from DLA,X values of R perp from Center,Y values of R perp from Center")
np.savetxt(r'/home1/08289/tg875885/radial_to_rotating_flows/Aharon/Outputs2/'+SnapDirect+'_Cf(LLS).txt',LLSCoveringFactor,header="X values of R perp from DLA,Y values of R perp from DLA,X values of R perp from Center,Y values of R perp from Center")
np.savetxt(r'/home1/08289/tg875885/radial_to_rotating_flows/Aharon/Outputs2/'+SnapDirect+'_MoreData.txt',MoreData,header="Abin_edges[0:15],Abin_means,Abin_means2,Abin_means3,Bbin_edges[0:15],Bbin_means,Bbin_means2,Bbin_means3")

In [ ]:
List=[]
for i in range(102,215):
    List.append(str(i))
print(List)

In [ ]:
DataOfHalos=np.loadtxt('/home1/08289/tg875885/radial_to_rotating_flows/Aharon/HalosData/m12w.dat')
print(DataOfHalos[len(DataOfHalos)-1][1])